# Azure Document Intelligence 문서 처리 파이프라인 테스트

Azure Document Intelligence (구 Form Recognizer)를 사용한 문서 처리 시스템의 핵심 기능을 단계별로 테스트합니다.

## 파이프라인 개요
1. **환경 설정**: Azure 서비스 연결 및 초기화
2. **문서 업로드**: 다양한 형식의 문서 처리
3. **텍스트 추출**: Azure Document Intelligence 기반 OCR 및 구조 분석
4. **레이아웃 분석**: 다중 컬럼 및 페이지별 읽기 순서 최적화
5. **구조적 요소 추출**: 테이블, 문단, 제목 등 문서 구조 분석
6. **검색 테스트**: 추출된 텍스트 기반 검색 성능 검증

**주요 기능**:
- ✅ 정확한 텍스트 추출
- ✅ 고성능 레이아웃 분석
- ✅ 표/폼 구조 인식
- ✅ 페이지별 처리 최적화

## 1. 환경 설정 및 Azure 서비스 구성

In [1]:
# Azure Document Intelligence 설정 및 연결 테스트
import os
from pathlib import Path
from dotenv import load_dotenv

# 백엔드 .env 파일 로드
backend_env_path = Path("/home/admin/wkms-aws/backend/.env")
if backend_env_path.exists():
    load_dotenv(backend_env_path)
    print(f"✅ 백엔드 환경 파일 로드 완료: {backend_env_path}")
else:
    print(f"❌ 백엔드 환경 파일을 찾을 수 없습니다: {backend_env_path}")

# Azure Document Intelligence 설정 (백엔드 .env에서 로드)
AZURE_DOCUMENT_INTELLIGENCE_ENDPOINT = os.getenv('AZURE_DOCUMENT_INTELLIGENCE_ENDPOINT')
AZURE_DOCUMENT_INTELLIGENCE_KEY = os.getenv('AZURE_DOCUMENT_INTELLIGENCE_API_KEY')
AZURE_DOC_MODEL = os.getenv('AZURE_DOCUMENT_INTELLIGENCE_DEFAULT_MODEL', 'prebuilt-read')
CONFIDENCE_THRESHOLD = float(os.getenv('AZURE_DOCUMENT_INTELLIGENCE_CONFIDENCE_THRESHOLD', '0.75'))

# 기타 설정
BACKEND_URL = "http://localhost:8000"
USE_AZURE_DOCUMENT_INTELLIGENCE = True
ENABLE_PAGE_SPECIFIC_LAYOUT = True
MAX_PAGES = int(os.getenv('AZURE_DOCUMENT_INTELLIGENCE_MAX_ASYNC_PAGES', '50'))
FIRST_PAGE_LAYOUT = "full_page"
SAVE_PAGE_RESULTS = True

# Azure 리소스 설정 (선택사항)
AZURE_SUBSCRIPTION_ID = os.getenv('AZURE_SUBSCRIPTION_ID')
AZURE_RESOURCE_GROUP = os.getenv('AZURE_RESOURCE_GROUP')

print("🔧 Azure Document Intelligence 설정 확인:")
print(f"🔧 백엔드 URL: {BACKEND_URL}")

if AZURE_DOCUMENT_INTELLIGENCE_ENDPOINT:
    print(f"✅ Azure 엔드포인트: {AZURE_DOCUMENT_INTELLIGENCE_ENDPOINT}")
else:
    print("❌ Azure 엔드포인트 미설정")

if AZURE_DOCUMENT_INTELLIGENCE_KEY:
    masked_key = AZURE_DOCUMENT_INTELLIGENCE_KEY[:8] + "*" * 20 + AZURE_DOCUMENT_INTELLIGENCE_KEY[-8:] if len(AZURE_DOCUMENT_INTELLIGENCE_KEY) > 16 else "***"
    print(f"🔐 Azure API 키: ✅ 설정됨 ({masked_key})")
else:
    print("🔐 Azure API 키: ❌ 미설정")

print(f"📄 기본 모델: {AZURE_DOC_MODEL}")
print(f"🎯 신뢰도 임계값: {CONFIDENCE_THRESHOLD*100:.1f}%")
print(f"📋 페이지별 레이아웃: {'✅ 활성화' if ENABLE_PAGE_SPECIFIC_LAYOUT else '❌ 비활성화'}")

# Azure Document Intelligence 클라이언트 초기화 시도
azure_client = None
azure_client_status = "❌ 미초기화"

try:
    if AZURE_DOCUMENT_INTELLIGENCE_ENDPOINT and AZURE_DOCUMENT_INTELLIGENCE_KEY:
        from azure.ai.formrecognizer import DocumentAnalysisClient
        from azure.core.credentials import AzureKeyCredential
        
        azure_client = DocumentAnalysisClient(
            endpoint=AZURE_DOCUMENT_INTELLIGENCE_ENDPOINT,
            credential=AzureKeyCredential(AZURE_DOCUMENT_INTELLIGENCE_KEY)
        )
        azure_client_status = "✅ 초기화 완료"
        print(f"\n🎉 Azure Document Intelligence 클라이언트: {azure_client_status}")
        print("📡 클라이언트 연결 상태: ✅ 준비됨")
        
    else:
        print(f"\n❌ Azure Document Intelligence 클라이언트 초기화 실패: 엔드포인트 또는 API 키 누락")
        
except ImportError as e:
    print(f"\n❌ Azure 라이브러리 임포트 실패: {e}")
    print("   다음 명령어로 설치하세요: pip install azure-ai-formrecognizer azure-identity")
except Exception as e:
    print(f"\n❌ Azure Document Intelligence 클라이언트 초기화 실패: {e}")

print(f"\n📋 설정 요약:")
print(f"   • Azure 클라이언트: {azure_client_status}")
print(f"   • 환경 설정 파일: ✅ 로드됨")
print(f"   • 백엔드 연동: ✅ 설정됨")

✅ 백엔드 환경 파일 로드 완료: /home/admin/wkms-aws/backend/.env
🔧 Azure Document Intelligence 설정 확인:
🔧 백엔드 URL: http://localhost:8000
✅ Azure 엔드포인트: https://pre-pro-doc.cognitiveservices.azure.com/
🔐 Azure API 키: ✅ 설정됨 (Ch3M6JK1********************ACOGYcq0)
📄 기본 모델: prebuilt-read
🎯 신뢰도 임계값: 80.0%
📋 페이지별 레이아웃: ✅ 활성화

🎉 Azure Document Intelligence 클라이언트: ✅ 초기화 완료
📡 클라이언트 연결 상태: ✅ 준비됨

📋 설정 요약:
   • Azure 클라이언트: ✅ 초기화 완료
   • 환경 설정 파일: ✅ 로드됨
   • 백엔드 연동: ✅ 설정됨

🎉 Azure Document Intelligence 클라이언트: ✅ 초기화 완료
📡 클라이언트 연결 상태: ✅ 준비됨

📋 설정 요약:
   • Azure 클라이언트: ✅ 초기화 완료
   • 환경 설정 파일: ✅ 로드됨
   • 백엔드 연동: ✅ 설정됨


## 2. Azure Document Intelligence 클라이언트 초기화

In [2]:
# Azure Document Intelligence 클라이언트 초기화 및 연결 테스트
def init_azure_document_intelligence():
    """Azure Document Intelligence 클라이언트 초기화 및 연결 테스트"""
    try:
        # 올바른 패키지 import (azure-ai-formrecognizer)
        from azure.ai.formrecognizer import DocumentAnalysisClient
        from azure.core.credentials import AzureKeyCredential
        
        if not AZURE_DOCUMENT_INTELLIGENCE_ENDPOINT or not AZURE_DOCUMENT_INTELLIGENCE_KEY:
            print("❌ Azure Document Intelligence 자격증명이 없습니다")
            print("🔧 다음 환경변수를 설정해주세요:")
            print("   AZURE_DOCUMENT_INTELLIGENCE_ENDPOINT=https://your-resource.cognitiveservices.azure.com/")
            print("   AZURE_DOCUMENT_INTELLIGENCE_KEY=your-api-key")
            return None
            
        # 클라이언트 생성
        credential = AzureKeyCredential(AZURE_DOCUMENT_INTELLIGENCE_KEY)
        client = DocumentAnalysisClient(
            endpoint=AZURE_DOCUMENT_INTELLIGENCE_ENDPOINT,
            credential=credential
        )
        
        print("✅ Azure Document Intelligence 클라이언트 초기화 완료")
        print(f"   🌐 엔드포인트: {AZURE_DOCUMENT_INTELLIGENCE_ENDPOINT}")
        print(f"   🔐 인증: AzureKeyCredential 사용")
        print(f"   📦 SDK: azure-ai-formrecognizer (Azure Document Intelligence)")
        
        # 간단한 연결 테스트
        try:
            print("📋 사용 가능한 모델:")
            available_models = [
                "prebuilt-read",
                "prebuilt-layout", 
                "prebuilt-document",
                "prebuilt-invoice",
                "prebuilt-receipt"
            ]
            for model in available_models:
                print(f"   • {model}")
            
            print("✅ Azure Document Intelligence 서비스 연결 준비 완료")
            return client
            
        except Exception as e:
            print(f"⚠️ 서비스 연결 테스트 중 오류: {e}")
            print("클라이언트는 생성되었지만 서비스 연결을 확인할 수 없습니다")
            return client
            
    except ImportError as e:
        print("❌ Azure Document Intelligence SDK가 설치되지 않았습니다")
        print("🔧 설치 명령어:")
        print("   pip install azure-ai-formrecognizer azure-identity")
        print(f"   상세 오류: {e}")
        return None
    except Exception as e:
        print(f"❌ Azure Document Intelligence 클라이언트 초기화 실패: {e}")
        return None

def test_azure_client_connection(client):
    """Azure 클라이언트 연결 상태 테스트"""
    if not client:
        return False
    
    try:
        # 클라이언트 객체가 올바르게 생성되었는지 확인
        endpoint = client._endpoint
        print(f"🔗 클라이언트 엔드포인트: {endpoint}")
        print("✅ 클라이언트 객체 검증 완료")
        return True
    except Exception as e:
        print(f"❌ 클라이언트 검증 실패: {e}")
        return False

# Azure Document Intelligence 클라이언트 초기화
print("🚀 Azure Document Intelligence 클라이언트 초기화 중...")
azure_client = init_azure_document_intelligence()

if azure_client:
    client_ok = test_azure_client_connection(azure_client)
    
    print(f"\n🎯 Azure Document Intelligence 준비 완료!")
    print(f"   기본 모델: {AZURE_DOC_MODEL}")
    print(f"   신뢰도 임계값: {CONFIDENCE_THRESHOLD*100:.1f}%")
    print(f"   최대 페이지: {MAX_PAGES}페이지")
    print(f"   클라이언트 상태: {'✅ 정상' if client_ok else '⚠️ 검증 필요'}")
    
    # 전역 변수로 설정
    globals()['azure_client'] = azure_client
else:
    print(f"\n❌ Azure Document Intelligence 사용 불가")
    print("환경 설정을 확인하고 다시 시도해주세요")
    globals()['azure_client'] = None

🚀 Azure Document Intelligence 클라이언트 초기화 중...
✅ Azure Document Intelligence 클라이언트 초기화 완료
   🌐 엔드포인트: https://pre-pro-doc.cognitiveservices.azure.com/
   🔐 인증: AzureKeyCredential 사용
   📦 SDK: azure-ai-formrecognizer (Azure Document Intelligence)
📋 사용 가능한 모델:
   • prebuilt-read
   • prebuilt-layout
   • prebuilt-document
   • prebuilt-invoice
   • prebuilt-receipt
✅ Azure Document Intelligence 서비스 연결 준비 완료
🔗 클라이언트 엔드포인트: https://pre-pro-doc.cognitiveservices.azure.com
✅ 클라이언트 객체 검증 완료

🎯 Azure Document Intelligence 준비 완료!
   기본 모델: prebuilt-read
   신뢰도 임계값: 80.0%
   최대 페이지: 10페이지
   클라이언트 상태: ✅ 정상


## 3. 문서 업로드 및 처리 함수

In [3]:
# 문서 복잡도 및 처리 전략 판별
def detect_document_complexity_azure(file_path):
    """Azure Document Intelligence를 위한 문서 복잡도 분석"""
    try:
        file_size = os.path.getsize(file_path)
        file_extension = os.path.splitext(file_path.lower())[1]
        
        complexity_info = {
            'file_size': file_size,
            'file_size_mb': file_size / (1024 * 1024),
            'file_extension': file_extension,
            'complexity': 'simple',
            'recommended_model': 'prebuilt-read',
            'processing_strategy': 'sync'
        }
        
        # 파일 크기 기준
        if file_size > 20 * 1024 * 1024:  # 20MB 이상
            complexity_info['complexity'] = 'complex'
            complexity_info['recommended_model'] = 'prebuilt-layout'
            complexity_info['processing_strategy'] = 'async'
        elif file_size > 5 * 1024 * 1024:  # 5-20MB
            complexity_info['complexity'] = 'medium'
            complexity_info['recommended_model'] = 'prebuilt-layout'
            complexity_info['processing_strategy'] = 'sync'
        
        # PDF의 경우 페이지 수 확인
        if file_extension == '.pdf':
            try:
                import fitz
                doc = fitz.open(file_path)
                page_count = len(doc)
                doc.close()
                
                complexity_info['page_count'] = page_count
                
                if page_count > 50:
                    complexity_info['complexity'] = 'complex'
                    complexity_info['recommended_model'] = 'prebuilt-layout'
                    complexity_info['processing_strategy'] = 'async'
                elif page_count > 10:
                    complexity_info['complexity'] = 'medium'
                    complexity_info['recommended_model'] = 'prebuilt-layout'
                    
            except Exception as e:
                print(f"⚠️ PDF 페이지 수 확인 실패: {e}")
                complexity_info['page_count'] = 'unknown'
        
        # 일반적으로 layout 모델이 더 안정적이므로 기본값으로 설정
        if complexity_info['recommended_model'] == 'prebuilt-read':
            complexity_info['recommended_model'] = 'prebuilt-layout'
            complexity_info['reason'] = 'layout_optimization'
        
        return complexity_info
        
    except Exception as e:
        print(f"⚠️ 복잡도 감지 실패: {e}")
        return {
            'complexity': 'unknown',
            'recommended_model': 'prebuilt-layout',  # 안전한 기본값
            'processing_strategy': 'sync',
            'error': str(e)
        }

def determine_azure_processing_strategy(complexity_info):
    """Azure Document Intelligence 처리 전략 결정"""
    strategy = {
        'model_id': complexity_info.get('recommended_model', 'prebuilt-layout'),
        'is_async': complexity_info.get('processing_strategy') == 'async',
        'max_pages': MAX_PAGES,
        'confidence_threshold': CONFIDENCE_THRESHOLD,
        'features': []
    }
    
    # 복잡도에 따른 기능 추가
    complexity = complexity_info.get('complexity', 'simple')
    
    if complexity in ['medium', 'complex']:
        strategy['features'].extend(['tables', 'keyValuePairs'])
    
    if complexity == 'complex':
        strategy['features'].extend(['paragraphs', 'styles'])
    
    # 기본적으로 자연스러운 읽기 순서 적용
    strategy['reading_order'] = 'natural'
    
    return strategy

# 테스트 파일로 복잡도 검사
test_file = "/home/admin/wkms-aws/jupyter_notebook/data/input_docs/test.pdf"
if os.path.exists(test_file):
    complexity_info = detect_document_complexity_azure(test_file)
    strategy = determine_azure_processing_strategy(complexity_info)
    
    print(f"📄 문서 분석 결과: {test_file}")
    print(f"   파일 크기: {complexity_info['file_size_mb']:.2f}MB")
    print(f"   복잡도: {complexity_info['complexity']}")
    print(f"   권장 모델: {strategy['model_id']}")
    print(f"   처리 방식: {'비동기' if strategy['is_async'] else '동기'}")
    print(f"   추가 기능: {', '.join(strategy['features']) if strategy['features'] else '기본'}")
    if 'page_count' in complexity_info:
        print(f"   페이지 수: {complexity_info['page_count']}")
else:
    print(f"❌ 테스트 파일을 찾을 수 없습니다: {test_file}")
    print("💡 /home/admin/wkms-aws/jupyter_notebook/data/input_docs/ 폴더에 test.pdf를 배치해주세요")

📄 문서 분석 결과: /home/admin/wkms-aws/jupyter_notebook/data/input_docs/test.pdf
   파일 크기: 11.85MB
   복잡도: medium
   권장 모델: prebuilt-layout
   처리 방식: 동기
   추가 기능: tables, keyValuePairs
   페이지 수: 27


## 4. 한국어 텍스트 레이아웃 분석 및 재구성

In [4]:
# 한국어 텍스트 레이아웃 분석 및 재구성 함수들
def analyze_korean_layout(azure_result, page_number=1):
    """Azure Document Intelligence 결과에서 한국어 레이아웃 분석"""
    layout_info = {
        'page_number': page_number,
        'total_elements': 0,
        'text_elements': [],
        'table_elements': [],
        'paragraph_elements': [],
        'reading_order': [],
        'column_structure': 'unknown',
        'layout_type': 'unknown'
    }
    
    try:
        # Azure Document Intelligence 결과 구조 분석
        if hasattr(azure_result, 'pages') and azure_result.pages:
            page = azure_result.pages[page_number - 1] if len(azure_result.pages) >= page_number else azure_result.pages[0]
            
            # 텍스트 라인 수집
            if hasattr(page, 'lines') and page.lines:
                for line in page.lines:
                    if line.content and line.polygon:
                        # 바운딩 박스 계산
                        x_coords = [p.x for p in line.polygon]
                        y_coords = [p.y for p in line.polygon]
                        
                        element = {
                            'text': line.content,
                            'bbox': {
                                'left': min(x_coords),
                                'top': min(y_coords),
                                'right': max(x_coords),
                                'bottom': max(y_coords),
                                'center_x': sum(x_coords) / len(x_coords),
                                'center_y': sum(y_coords) / len(y_coords),
                                'width': max(x_coords) - min(x_coords),
                                'height': max(y_coords) - min(y_coords)
                            },
                            'confidence': getattr(line, 'confidence', 1.0),
                            'type': 'line'
                        }
                        layout_info['text_elements'].append(element)
        
        # 단락 정보 수집 (사용 가능한 경우)
        if hasattr(azure_result, 'paragraphs') and azure_result.paragraphs:
            for para in azure_result.paragraphs:
                if para.content and hasattr(para, 'bounding_regions'):
                    for region in para.bounding_regions:
                        if region.page_number == page_number:
                            # 단락 레벨 정보
                            para_element = {
                                'text': para.content,
                                'page': region.page_number,
                                'polygon': region.polygon,
                                'role': getattr(para, 'role', 'paragraph'),
                                'type': 'paragraph'
                            }
                            layout_info['paragraph_elements'].append(para_element)
        
        # 테이블 정보 수집
        if hasattr(azure_result, 'tables') and azure_result.tables:
            for table in azure_result.tables:
                if hasattr(table, 'bounding_regions'):
                    for region in table.bounding_regions:
                        if region.page_number == page_number:
                            table_element = {
                                'row_count': table.row_count,
                                'column_count': table.column_count,
                                'page': region.page_number,
                                'polygon': region.polygon,
                                'type': 'table'
                            }
                            layout_info['table_elements'].append(table_element)
        
        layout_info['total_elements'] = len(layout_info['text_elements'])
        
        # 레이아웃 타입 결정
        layout_info['layout_type'] = determine_layout_type_korean(layout_info, page_number)
        
        return layout_info
        
    except Exception as e:
        print(f"⚠️ 레이아웃 분석 실패: {e}")
        return layout_info

def determine_layout_type_korean(layout_info, page_number=1):
    """한국어 문서의 레이아웃 타입 결정"""
    text_elements = layout_info['text_elements']
    
    if not text_elements:
        return 'empty'
    
    # 첫 페이지는 제목/저자/초록 구조로 간주
    if page_number == 1 and FIRST_PAGE_LAYOUT == 'full_page':
        return 'title_page'
    
    # 중심점 분포로 컬럼 구조 판단
    center_x_coords = [elem['bbox']['center_x'] for elem in text_elements]
    
    if len(set([round(x, 1) for x in center_x_coords])) <= 2:
        return 'single_column'
    
    # X 좌표 클러스터링으로 2열 구조 감지
    center_x_sorted = sorted(center_x_coords)
    
    if len(center_x_sorted) >= 4:
        # 가장 큰 간격 찾기
        gaps = []
        for i in range(len(center_x_sorted) - 1):
            gap = center_x_sorted[i + 1] - center_x_sorted[i]
            gaps.append(gap)
        
        max_gap = max(gaps) if gaps else 0
        avg_gap = sum(gaps) / len(gaps) if gaps else 0
        
        # 큰 간격이 있으면 2열 구조
        if max_gap > avg_gap * 3:
            return 'two_column'
    
    return 'single_column'

def reconstruct_korean_reading_order(layout_info):
    """한국어 읽기 순서에 맞춘 텍스트 재구성"""
    text_elements = layout_info['text_elements']
    layout_type = layout_info['layout_type']
    page_number = layout_info['page_number']
    
    if not text_elements:
        return []
    
    if layout_type == 'title_page':
        # 첫 페이지: 상하 순서 (제목 → 저자 → 초록 → 내용)
        sorted_elements = sorted(text_elements, key=lambda x: (x['bbox']['top'], x['bbox']['left']))
        return sorted_elements
    
    elif layout_type == 'two_column':
        # 2열 구조: 좌측 열 → 우측 열
        center_x_coords = [elem['bbox']['center_x'] for elem in text_elements]
        
        # 중간점 계산으로 좌우 분할
        if center_x_coords:
            # 클러스터링을 통한 분할점 결정
            center_x_sorted = sorted(set(center_x_coords))
            if len(center_x_sorted) >= 2:
                # 가장 큰 간격의 중점을 분할점으로 사용
                gaps = [(center_x_sorted[i+1] - center_x_sorted[i], (center_x_sorted[i] + center_x_sorted[i+1]) / 2) 
                       for i in range(len(center_x_sorted) - 1)]
                gaps.sort(reverse=True)
                threshold = gaps[0][1] if gaps else 0.5
            else:
                threshold = sum(center_x_coords) / len(center_x_coords)
            
            # 좌우 분할
            left_elements = [elem for elem in text_elements if elem['bbox']['center_x'] <= threshold]
            right_elements = [elem for elem in text_elements if elem['bbox']['center_x'] > threshold]
            
            # 각 열 내에서 상하 정렬
            left_sorted = sorted(left_elements, key=lambda x: (x['bbox']['top'], x['bbox']['left']))
            right_sorted = sorted(right_elements, key=lambda x: (x['bbox']['top'], x['bbox']['left']))
            
            # 좌열 → 우열 순서로 결합
            return left_sorted + right_sorted
    
    else:
        # 단일 열: 상하 순서
        sorted_elements = sorted(text_elements, key=lambda x: (x['bbox']['top'], x['bbox']['left']))
        return sorted_elements

print("📦 한국어 레이아웃 분석 함수 로드 완료")
print("   ✅ analyze_korean_layout: Azure 결과에서 레이아웃 분석")
print("   ✅ determine_layout_type_korean: 한국어 문서 레이아웃 타입 결정") 
print("   ✅ reconstruct_korean_reading_order: 한국어 읽기 순서 재구성")

📦 한국어 레이아웃 분석 함수 로드 완료
   ✅ analyze_korean_layout: Azure 결과에서 레이아웃 분석
   ✅ determine_layout_type_korean: 한국어 문서 레이아웃 타입 결정
   ✅ reconstruct_korean_reading_order: 한국어 읽기 순서 재구성


## 5. 다중 페이지 지원 텍스트 추출

In [5]:
# Azure Document Intelligence 텍스트 추출 함수들
import time
from datetime import datetime

def extract_text_with_azure(file_path, strategy=None):
    """Azure Document Intelligence를 사용한 텍스트 추출"""
    if not azure_client:
        return {
            'success': False,
            'error': 'Azure Document Intelligence 클라이언트가 초기화되지 않았습니다',
            'extraction_method': 'azure_failed'
        }
    
    try:
        print(f"🚀 Azure Document Intelligence 텍스트 추출 시작")
        print(f"   📄 파일: {os.path.basename(file_path)}")
        
        # 기본 전략 설정
        if not strategy:
            complexity_info = detect_document_complexity_azure(file_path)
            strategy = determine_azure_processing_strategy(complexity_info)
        
        print(f"   🔧 모델: {strategy['model_id']}")
        print(f"   ⚡ 처리: {'비동기' if strategy['is_async'] else '동기'}")
        
        start_time = time.time()
        
        # 파일 읽기
        with open(file_path, 'rb') as file:
            file_content = file.read()
        
        if strategy['is_async']:
            # 비동기 처리
            result = extract_text_async_azure(azure_client, file_content, strategy)
        else:
            # 동기 처리  
            result = extract_text_sync_azure(azure_client, file_content, strategy)
        
        processing_time = time.time() - start_time
        
        if result['success']:
            print(f"✅ 추출 완료: {len(result['text'])}자 ({processing_time:.1f}초)")
            result['processing_time'] = processing_time
            result['strategy'] = strategy
        else:
            print(f"❌ 추출 실패: {result['error']}")
        
        return result
        
    except Exception as e:
        print(f"❌ Azure 텍스트 추출 오류: {e}")
        return {
            'success': False,
            'error': str(e),
            'extraction_method': 'azure_exception'
        }

def extract_text_sync_azure(client, file_content, strategy):
    """Azure Document Intelligence 동기 텍스트 추출"""
    try:
        from azure.ai.documentintelligence.models import AnalyzeDocumentRequest
        
        print("   🔄 동기 처리 중...")
        
        # 문서 분석 요청
        analyze_request = AnalyzeDocumentRequest(bytes_source=file_content)
        
        poller = client.begin_analyze_document(
            model_id=strategy['model_id'],
            analyze_request=analyze_request
        )
        
        # 결과 대기
        azure_result = poller.result()
        
        # 결과 처리
        return process_azure_result(azure_result, strategy)
        
    except Exception as e:
        return {
            'success': False,
            'error': f'동기 처리 실패: {str(e)}',
            'extraction_method': 'azure_sync_failed'
        }

def extract_text_async_azure(client, file_content, strategy):
    """Azure Document Intelligence 비동기 텍스트 추출"""
    try:
        from azure.ai.documentintelligence.models import AnalyzeDocumentRequest
        
        print("   🔄 비동기 처리 시작...")
        
        # 문서 분석 요청
        analyze_request = AnalyzeDocumentRequest(bytes_source=file_content)
        
        poller = client.begin_analyze_document(
            model_id=strategy['model_id'],
            analyze_request=analyze_request
        )
        
        # 폴링으로 완료 대기
        max_wait_time = 300  # 5분
        wait_interval = 5
        total_waited = 0
        
        while not poller.done():
            if total_waited >= max_wait_time:
                return {
                    'success': False,
                    'error': f'처리 시간 초과 ({max_wait_time}초)',
                    'extraction_method': 'azure_async_timeout'
                }
            
            print(f"   ⏳ 처리 중... ({total_waited}초 경과)")
            time.sleep(wait_interval)
            total_waited += wait_interval
        
        # 결과 가져오기
        azure_result = poller.result()
        
        print(f"   ✅ 비동기 처리 완료 ({total_waited}초)")
        
        # 결과 처리
        return process_azure_result(azure_result, strategy)
        
    except Exception as e:
        return {
            'success': False,
            'error': f'비동기 처리 실패: {str(e)}',
            'extraction_method': 'azure_async_failed'
        }

def process_azure_result(azure_result, strategy):
    """Azure Document Intelligence 결과 처리"""
    try:
        print("   📊 결과 분석 중...")
        
        result = {
            'success': True,
            'text': '',
            'page_results': [],
            'tables': [],
            'key_value_pairs': [],
            'metadata': {
                'extraction_method': 'azure_document_intelligence',
                'model_used': strategy['model_id'],
                'page_count': 0,
                'confidence_stats': {},
                'layout_analysis': {}
            }
        }
        
        # 전체 텍스트 추출
        if hasattr(azure_result, 'content'):
            result['text'] = azure_result.content
        
        # 페이지별 분석
        if hasattr(azure_result, 'pages') and azure_result.pages:
            result['metadata']['page_count'] = len(azure_result.pages)
            
            for page_idx, page in enumerate(azure_result.pages):
                page_number = page_idx + 1
                
                # 페이지별 레이아웃 분석
                layout_info = analyze_korean_layout(azure_result, page_number)
                
                # 한국어 읽기 순서 재구성
                ordered_elements = reconstruct_korean_reading_order(layout_info)
                
                # 페이지 텍스트 생성
                page_text = '\\n'.join([elem['text'] for elem in ordered_elements])
                
                page_result = {
                    'page': page_number,
                    'text': page_text,
                    'layout_type': layout_info['layout_type'],
                    'element_count': len(ordered_elements),
                    'table_count': len(layout_info['table_elements']),
                    'paragraph_count': len(layout_info['paragraph_elements'])
                }
                
                # 신뢰도 통계
                confidences = [elem['confidence'] for elem in ordered_elements if elem.get('confidence')]
                if confidences:
                    page_result['avg_confidence'] = sum(confidences) / len(confidences)
                    page_result['min_confidence'] = min(confidences)
                    page_result['max_confidence'] = max(confidences)
                
                result['page_results'].append(page_result)
        
        # 테이블 추출
        if hasattr(azure_result, 'tables') and azure_result.tables:
            for table in azure_result.tables:
                table_data = extract_table_data_azure(table)
                result['tables'].append(table_data)
        
        # 키-값 쌍 추출
        if hasattr(azure_result, 'key_value_pairs') and azure_result.key_value_pairs:
            for kv in azure_result.key_value_pairs:
                if kv.key and kv.value:
                    kv_data = {
                        'key': kv.key.content if hasattr(kv.key, 'content') else str(kv.key),
                        'value': kv.value.content if hasattr(kv.value, 'content') else str(kv.value),
                        'confidence': getattr(kv, 'confidence', 1.0)
                    }
                    result['key_value_pairs'].append(kv_data)
        
        # 전체 신뢰도 통계
        all_confidences = []
        for page_result in result['page_results']:
            if 'avg_confidence' in page_result:
                all_confidences.append(page_result['avg_confidence'])
        
        if all_confidences:
            result['metadata']['confidence_stats'] = {
                'average': sum(all_confidences) / len(all_confidences),
                'min': min(all_confidences),
                'max': max(all_confidences)
            }
        
        print(f"   📄 페이지: {result['metadata']['page_count']}개")
        print(f"   📊 테이블: {len(result['tables'])}개")
        print(f"   🔑 키-값 쌍: {len(result['key_value_pairs'])}개")
        
        return result
        
    except Exception as e:
        return {
            'success': False,
            'error': f'결과 처리 실패: {str(e)}',
            'extraction_method': 'azure_result_processing_failed'
        }

def extract_table_data_azure(table):
    """Azure Document Intelligence 테이블 데이터 추출"""
    try:
        table_data = {
            'row_count': table.row_count,
            'column_count': table.column_count,
            'cells': [],
            'tsv': '',
            'csv': ''
        }
        
        # 셀 데이터 수집
        cells_grid = {}
        for cell in table.cells:
            row_idx = cell.row_index
            col_idx = cell.column_index
            
            if row_idx not in cells_grid:
                cells_grid[row_idx] = {}
            
            cells_grid[row_idx][col_idx] = {
                'content': cell.content,
                'row_span': getattr(cell, 'row_span', 1),
                'column_span': getattr(cell, 'column_span', 1),
                'confidence': getattr(cell, 'confidence', 1.0)
            }
        
        # TSV/CSV 생성
        tsv_rows = []
        csv_rows = []
        
        for row_idx in sorted(cells_grid.keys()):
            row_data = cells_grid[row_idx]
            
            tsv_row = []
            csv_row = []
            
            for col_idx in range(table.column_count):
                cell_content = row_data.get(col_idx, {}).get('content', '')
                tsv_row.append(cell_content)
                csv_row.append(f'\"{cell_content}\"' if ',' in cell_content else cell_content)
            
            tsv_rows.append('\\t'.join(tsv_row))
            csv_rows.append(','.join(csv_row))
        
        table_data['tsv'] = '\\n'.join(tsv_rows)
        table_data['csv'] = '\\n'.join(csv_rows)
        table_data['cells'] = cells_grid
        
        return table_data
        
    except Exception as e:
        return {
            'error': f'테이블 데이터 추출 실패: {str(e)}',
            'cells': [],
            'tsv': '',
            'csv': ''
        }

print("📦 Azure Document Intelligence 텍스트 추출 함수 로드 완료")
print("   ✅ extract_text_with_azure: 메인 추출 함수")
print("   ✅ extract_text_sync_azure: 동기 처리")
print("   ✅ extract_text_async_azure: 비동기 처리")
print("   ✅ process_azure_result: 결과 처리 및 한국어 최적화")
print("   ✅ extract_table_data_azure: 테이블 데이터 추출")

📦 Azure Document Intelligence 텍스트 추출 함수 로드 완료
   ✅ extract_text_with_azure: 메인 추출 함수
   ✅ extract_text_sync_azure: 동기 처리
   ✅ extract_text_async_azure: 비동기 처리
   ✅ process_azure_result: 결과 처리 및 한국어 최적화
   ✅ extract_table_data_azure: 테이블 데이터 추출


## 6. 한국어 최적화 텍스트 청킹 및 RAG 준비

Azure Document Intelligence로 추출된 텍스트를 한국어 특성에 맞게 청킹하고 RAG(Retrieval-Augmented Generation) 파이프라인에 적합하도록 준비합니다.

### 기능:
- 한국어 형태소 분석 기반 의미 단위 청킹
- 문서 구조 인식 기반 컨텍스트 보존
- 페이지별/섹션별 메타데이터 유지
- 검색 최적화를 위한 키워드 추출
- RAG 백엔드 연동을 위한 데이터 포맷팅

In [6]:
# 한국어 최적화 텍스트 청킹 및 RAG 준비 함수들
import re
from collections import defaultdict

def prepare_text_for_rag_azure(extraction_result, chunk_size=512, chunk_overlap=50):
    """Azure 추출 결과를 RAG용으로 청킹 및 준비"""
    if not extraction_result.get('success'):
        return {
            'success': False,
            'error': 'Azure 추출 결과가 유효하지 않습니다',
            'chunks': []
        }
    
    try:
        print("🔧 RAG용 텍스트 준비 시작")
        print(f"   📊 청크 크기: {chunk_size} 토큰")
        print(f"   🔗 오버랩: {chunk_overlap} 토큰")
        
        chunks = []
        total_tokens = 0
        
        # 페이지별 청킹
        for page_result in extraction_result.get('page_results', []):
            page_chunks = create_korean_chunks_azure(
                page_result, 
                chunk_size, 
                chunk_overlap,
                extraction_result.get('metadata', {})
            )
            chunks.extend(page_chunks)
            total_tokens += sum(chunk['token_count'] for chunk in page_chunks)
        
        # 테이블 별도 처리
        if extraction_result.get('tables'):
            table_chunks = process_tables_for_rag_azure(
                extraction_result['tables'], 
                chunk_size
            )
            chunks.extend(table_chunks)
            total_tokens += sum(chunk['token_count'] for chunk in table_chunks)
        
        # 키-값 쌍 별도 처리
        if extraction_result.get('key_value_pairs'):
            kv_chunks = process_key_values_for_rag_azure(
                extraction_result['key_value_pairs']
            )
            chunks.extend(kv_chunks)
            total_tokens += sum(chunk['token_count'] for chunk in kv_chunks)
        
        # 청크 품질 검증 및 개선
        validated_chunks = validate_and_improve_chunks_azure(chunks)
        
        print(f"✅ 청킹 완료: {len(validated_chunks)}개 청크 ({total_tokens} 토큰)")
        
        return {
            'success': True,
            'chunks': validated_chunks,
            'total_chunks': len(validated_chunks),
            'total_tokens': total_tokens,
            'avg_chunk_size': total_tokens / len(validated_chunks) if validated_chunks else 0,
            'processing_metadata': {
                'source': 'azure_document_intelligence',
                'chunk_strategy': 'korean_morphological_aware',
                'chunk_size': chunk_size,
                'chunk_overlap': chunk_overlap
            }
        }
        
    except Exception as e:
        print(f"❌ RAG 준비 실패: {e}")
        return {
            'success': False,
            'error': str(e),
            'chunks': []
        }

def create_korean_chunks_azure(page_result, chunk_size, chunk_overlap, document_metadata):
    """한국어 특성을 고려한 페이지별 청킹"""
    try:
        page_text = page_result.get('text', '')
        page_number = page_result.get('page', 1)
        
        if not page_text.strip():
            return []
        
        print(f"   📄 페이지 {page_number} 청킹 중...")
        
        # 한국어 문장 단위 분할
        sentences = split_korean_sentences_azure(page_text)
        
        # 형태소 분석을 통한 의미 단위 식별
        morpheme_groups = analyze_morpheme_groups_azure(sentences)
        
        # 청크 생성
        chunks = []
        current_chunk = {
            'text': '',
            'sentences': [],
            'token_count': 0,
            'morpheme_count': 0
        }
        
        for group in morpheme_groups:
            group_tokens = count_tokens_korean(group['text'])
            
            # 청크 크기 체크
            if current_chunk['token_count'] + group_tokens > chunk_size and current_chunk['text']:
                # 현재 청크 완성
                chunk_data = finalize_chunk_azure(
                    current_chunk, 
                    page_number, 
                    page_result,
                    document_metadata,
                    len(chunks) + 1
                )
                chunks.append(chunk_data)
                
                # 오버랩을 위한 문장 유지
                overlap_sentences = current_chunk['sentences'][-2:] if len(current_chunk['sentences']) > 1 else []
                overlap_text = ' '.join(overlap_sentences)
                overlap_tokens = count_tokens_korean(overlap_text)
                
                # 새 청크 시작 (오버랩 포함)
                current_chunk = {
                    'text': overlap_text,
                    'sentences': overlap_sentences,
                    'token_count': overlap_tokens,
                    'morpheme_count': len(overlap_sentences)
                }
            
            # 그룹 추가
            current_chunk['text'] += (' ' if current_chunk['text'] else '') + group['text']
            current_chunk['sentences'].append(group['text'])
            current_chunk['token_count'] += group_tokens
            current_chunk['morpheme_count'] += group['morpheme_count']
        
        # 마지막 청크 처리
        if current_chunk['text'].strip():
            chunk_data = finalize_chunk_azure(
                current_chunk, 
                page_number, 
                page_result,
                document_metadata,
                len(chunks) + 1
            )
            chunks.append(chunk_data)
        
        print(f"   ✅ 페이지 {page_number}: {len(chunks)}개 청크 생성")
        return chunks
        
    except Exception as e:
        print(f"   ❌ 페이지 {page_number} 청킹 실패: {e}")
        return []

def split_korean_sentences_azure(text):
    """한국어 문장 분할 (Azure 결과 최적화)"""
    try:
        # 기본 문장 분할 패턴
        sentence_endings = r'[.!?。]\s+'
        
        # 한국어 특수 패턴 처리
        text = re.sub(r'([.!?。])\s*([가-힣A-Z])', r'\\1\\n\\2', text)
        
        # 줄바꿈 기반 분할
        sentences = []
        for line in text.split('\\n'):
            line = line.strip()
            if line:
                # 문장 부호 기반 세부 분할
                sub_sentences = re.split(sentence_endings, line)
                sentences.extend([s.strip() for s in sub_sentences if s.strip()])
        
        return sentences
        
    except Exception as e:
        print(f"문장 분할 실패: {e}")
        return [text]

def analyze_morpheme_groups_azure(sentences):
    """형태소 분석을 통한 의미 단위 그룹화"""
    try:
        groups = []
        
        for sentence in sentences:
            if not sentence.strip():
                continue
                
            # 형태소 분석
            morphemes = kiwi.analyze(sentence)
            
            # 의미 있는 형태소 추출
            meaningful_morphemes = []
            for token, pos, _, _ in morphemes[0]:
                if pos in ['NNG', 'NNP', 'VV', 'VA', 'MAG']:  # 명사, 동사, 형용사, 부사
                    meaningful_morphemes.append(token)
            
            group = {
                'text': sentence,
                'morpheme_count': len(meaningful_morphemes),
                'key_morphemes': meaningful_morphemes[:5],  # 상위 5개
                'sentence_type': classify_sentence_type_azure(sentence),
                'importance_score': calculate_importance_score_azure(sentence, meaningful_morphemes)
            }
            
            groups.append(group)
        
        return groups
        
    except Exception as e:
        print(f"형태소 분석 실패: {e}")
        return [{'text': ' '.join(sentences), 'morpheme_count': 0, 'key_morphemes': []}]

def classify_sentence_type_azure(sentence):
    """문장 유형 분류"""
    try:
        if re.search(r'[0-9]+[.]', sentence):
            return 'numbered_list'
        elif re.search(r'^[•▪▫-]', sentence.strip()):
            return 'bullet_point'
        elif '?' in sentence:
            return 'question'
        elif re.search(r'표[\\s]*[0-9]+|그림[\\s]*[0-9]+|도[\\s]*[0-9]+', sentence):
            return 'figure_reference'
        elif len(sentence) > 100:
            return 'paragraph'
        else:
            return 'sentence'
    except:
        return 'unknown'

def calculate_importance_score_azure(sentence, morphemes):
    """문장 중요도 점수 계산"""
    try:
        score = 0
        
        # 길이 기반 점수
        score += min(len(sentence) / 50, 2.0)
        
        # 형태소 다양성 점수
        score += len(set(morphemes)) * 0.1
        
        # 특수 키워드 점수
        important_patterns = [
            r'결론|요약|정리',
            r'중요|핵심|주요',
            r'첫째|둘째|셋째',
            r'따라서|그러므로|결국'
        ]
        
        for pattern in important_patterns:
            if re.search(pattern, sentence):
                score += 1.0
        
        return min(score, 5.0)  # 최대 5점
        
    except:
        return 1.0

def finalize_chunk_azure(chunk_data, page_number, page_result, document_metadata, chunk_index):
    """청크 데이터 최종 처리"""
    try:
        # 키워드 추출
        keywords = extract_keywords_from_chunk_azure(chunk_data['text'])
        
        # 메타데이터 생성
        metadata = {
            'page_number': page_number,
            'chunk_index': chunk_index,
            'layout_type': page_result.get('layout_type', 'unknown'),
            'confidence': page_result.get('avg_confidence', 1.0),
            'extraction_method': 'azure_document_intelligence',
            'document_metadata': document_metadata,
            'processing_timestamp': datetime.now().isoformat()
        }
        
        return {
            'text': chunk_data['text'].strip(),
            'token_count': chunk_data['token_count'],
            'morpheme_count': chunk_data['morpheme_count'],
            'sentence_count': len(chunk_data['sentences']),
            'keywords': keywords,
            'metadata': metadata,
            'chunk_type': 'text_content'
        }
        
    except Exception as e:
        print(f"청크 완성 실패: {e}")
        return {
            'text': chunk_data['text'],
            'token_count': chunk_data['token_count'],
            'error': str(e)
        }

def extract_keywords_from_chunk_azure(text):
    """청크에서 한국어 키워드 추출"""
    try:
        # 형태소 분석
        morphemes = kiwi.analyze(text)
        
        # 명사 추출
        nouns = []
        for token, pos, _, _ in morphemes[0]:
            if pos in ['NNG', 'NNP'] and len(token) > 1:
                nouns.append(token)
        
        # 빈도 계산
        noun_freq = defaultdict(int)
        for noun in nouns:
            noun_freq[noun] += 1
        
        # 상위 키워드 선별 (빈도 + 길이 고려)
        keywords = sorted(
            noun_freq.items(),
            key=lambda x: x[1] * len(x[0]),
            reverse=True
        )
        
        return [word for word, freq in keywords[:10]]  # 상위 10개
        
    except Exception as e:
        print(f"키워드 추출 실패: {e}")
        return []

print("📦 한국어 최적화 텍스트 청킹 함수 로드 완료")
print("   ✅ prepare_text_for_rag_azure: RAG용 텍스트 준비")
print("   ✅ create_korean_chunks_azure: 한국어 청킹")
print("   ✅ analyze_morpheme_groups_azure: 형태소 분석 그룹화")
print("   ✅ extract_keywords_from_chunk_azure: 키워드 추출")

📦 한국어 최적화 텍스트 청킹 함수 로드 완료
   ✅ prepare_text_for_rag_azure: RAG용 텍스트 준비
   ✅ create_korean_chunks_azure: 한국어 청킹
   ✅ analyze_morpheme_groups_azure: 형태소 분석 그룹화
   ✅ extract_keywords_from_chunk_azure: 키워드 추출


In [7]:
# 테이블 및 키-값 쌍 RAG 처리 함수들

def process_tables_for_rag_azure(tables, chunk_size):
    """테이블을 RAG용 청크로 변환"""
    try:
        chunks = []
        
        for table_idx, table in enumerate(tables):
            print(f"   📊 테이블 {table_idx + 1} 처리 중...")
            
            # 테이블 요약 생성
            table_summary = create_table_summary_azure(table)
            
            # 테이블 데이터를 텍스트로 변환
            table_text = convert_table_to_text_azure(table)
            
            # 큰 테이블은 행 단위로 분할
            if count_tokens_korean(table_text) > chunk_size:
                row_chunks = split_large_table_azure(table, chunk_size)
                chunks.extend(row_chunks)
            else:
                # 전체 테이블을 하나의 청크로
                chunk = {
                    'text': table_text,
                    'token_count': count_tokens_korean(table_text),
                    'chunk_type': 'table',
                    'metadata': {
                        'table_index': table_idx,
                        'row_count': table.get('row_count', 0),
                        'column_count': table.get('column_count', 0),
                        'table_summary': table_summary,
                        'extraction_method': 'azure_table'
                    },
                    'keywords': extract_keywords_from_chunk_azure(table_text)
                }
                chunks.append(chunk)
        
        print(f"   ✅ 테이블 처리 완료: {len(chunks)}개 청크")
        return chunks
        
    except Exception as e:
        print(f"   ❌ 테이블 처리 실패: {e}")
        return []

def create_table_summary_azure(table):
    """테이블 요약 생성"""
    try:
        summary = f"표 정보: {table.get('row_count', 0)}행 {table.get('column_count', 0)}열"
        
        # 첫 번째 행을 헤더로 가정하고 요약
        cells = table.get('cells', {})
        if 0 in cells:  # 첫 번째 행 존재
            headers = []
            for col_idx in range(table.get('column_count', 0)):
                if col_idx in cells[0]:
                    headers.append(cells[0][col_idx]['content'])
            
            if headers:
                summary += f", 컬럼: {', '.join(headers[:5])}"  # 상위 5개 컬럼만
        
        return summary
        
    except Exception as e:
        return f"표 요약 생성 실패: {str(e)}"

def convert_table_to_text_azure(table):
    """테이블을 한국어 자연어 텍스트로 변환"""
    try:
        cells = table.get('cells', {})
        if not cells:
            return "빈 테이블"
        
        text_parts = []
        text_parts.append(f"다음은 {table.get('row_count', 0)}행 {table.get('column_count', 0)}열의 표입니다.")
        
        # 첫 번째 행을 헤더로 처리
        if 0 in cells:
            headers = []
            for col_idx in range(table.get('column_count', 0)):
                if col_idx in cells[0]:
                    headers.append(cells[0][col_idx]['content'])
            
            if headers:
                text_parts.append(f"컬럼 제목: {', '.join(headers)}")
        
        # 데이터 행들을 자연어로 변환
        for row_idx in range(1, table.get('row_count', 0)):  # 헤더 제외
            if row_idx in cells:
                row_data = []
                for col_idx in range(table.get('column_count', 0)):
                    if col_idx in cells[row_idx]:
                        content = cells[row_idx][col_idx]['content']
                        if content.strip():
                            row_data.append(content)
                
                if row_data:
                    text_parts.append(f"{row_idx}행: {', '.join(row_data)}")
        
        return '\\n'.join(text_parts)
        
    except Exception as e:
        return f"테이블 텍스트 변환 실패: {str(e)}"

def split_large_table_azure(table, chunk_size):
    """큰 테이블을 행 단위로 분할"""
    try:
        chunks = []
        cells = table.get('cells', {})
        
        # 헤더 정보
        headers = []
        if 0 in cells:
            for col_idx in range(table.get('column_count', 0)):
                if col_idx in cells[0]:
                    headers.append(cells[0][col_idx]['content'])
        
        header_text = f"컬럼 제목: {', '.join(headers)}" if headers else ""
        header_tokens = count_tokens_korean(header_text)
        
        current_chunk_rows = []
        current_tokens = header_tokens
        
        # 데이터 행들을 청크로 분할
        for row_idx in range(1, table.get('row_count', 0)):
            if row_idx in cells:
                row_data = []
                for col_idx in range(table.get('column_count', 0)):
                    if col_idx in cells[row_idx]:
                        content = cells[row_idx][col_idx]['content']
                        if content.strip():
                            row_data.append(content)
                
                if row_data:
                    row_text = f"{row_idx}행: {', '.join(row_data)}"
                    row_tokens = count_tokens_korean(row_text)
                    
                    # 청크 크기 체크
                    if current_tokens + row_tokens > chunk_size and current_chunk_rows:
                        # 현재 청크 완성
                        chunk_text = header_text + '\\n' + '\\n'.join(current_chunk_rows)
                        chunk = {
                            'text': chunk_text,
                            'token_count': current_tokens,
                            'chunk_type': 'table_partial',
                            'metadata': {
                                'table_rows': len(current_chunk_rows),
                                'is_table_continuation': len(chunks) > 0
                            }
                        }
                        chunks.append(chunk)
                        
                        # 새 청크 시작
                        current_chunk_rows = [row_text]
                        current_tokens = header_tokens + row_tokens
                    else:
                        current_chunk_rows.append(row_text)
                        current_tokens += row_tokens
        
        # 마지막 청크 처리
        if current_chunk_rows:
            chunk_text = header_text + '\\n' + '\\n'.join(current_chunk_rows)
            chunk = {
                'text': chunk_text,
                'token_count': current_tokens,
                'chunk_type': 'table_partial',
                'metadata': {
                    'table_rows': len(current_chunk_rows),
                    'is_table_continuation': len(chunks) > 0
                }
            }
            chunks.append(chunk)
        
        return chunks
        
    except Exception as e:
        print(f"큰 테이블 분할 실패: {e}")
        return []

def process_key_values_for_rag_azure(key_value_pairs):
    """키-값 쌍을 RAG용 청크로 변환"""
    try:
        if not key_value_pairs:
            return []
        
        print(f"   🔑 키-값 쌍 {len(key_value_pairs)}개 처리 중...")
        
        # 키-값 쌍들을 의미별로 그룹화
        grouped_kvs = group_key_values_by_context_azure(key_value_pairs)
        
        chunks = []
        for group_name, kv_group in grouped_kvs.items():
            # 그룹을 텍스트로 변환
            group_text = convert_key_values_to_text_azure(kv_group, group_name)
            
            chunk = {
                'text': group_text,
                'token_count': count_tokens_korean(group_text),
                'chunk_type': 'key_value_pairs',
                'metadata': {
                    'kv_group': group_name,
                    'kv_count': len(kv_group),
                    'extraction_method': 'azure_form_recognizer'
                },
                'keywords': extract_keywords_from_chunk_azure(group_text)
            }
            chunks.append(chunk)
        
        print(f"   ✅ 키-값 쌍 처리 완료: {len(chunks)}개 그룹")
        return chunks
        
    except Exception as e:
        print(f"   ❌ 키-값 쌍 처리 실패: {e}")
        return []

def group_key_values_by_context_azure(key_value_pairs):
    """키-값 쌍을 의미적 컨텍스트로 그룹화"""
    try:
        groups = defaultdict(list)
        
        for kv in key_value_pairs:
            key = kv.get('key', '').lower()
            
            # 키 패턴 기반 그룹화
            if any(pattern in key for pattern in ['이름', '성명', '명칭']):
                groups['개인정보'].append(kv)
            elif any(pattern in key for pattern in ['날짜', '일시', '시간']):
                groups['날짜정보'].append(kv)
            elif any(pattern in key for pattern in ['금액', '가격', '비용', '원']):
                groups['금액정보'].append(kv)
            elif any(pattern in key for pattern in ['주소', '위치', '장소']):
                groups['위치정보'].append(kv)
            elif any(pattern in key for pattern in ['전화', '번호', '연락처']):
                groups['연락처정보'].append(kv)
            else:
                groups['기타정보'].append(kv)
        
        return dict(groups)
        
    except Exception as e:
        print(f"키-값 그룹화 실패: {e}")
        return {'전체': key_value_pairs}

def convert_key_values_to_text_azure(kv_group, group_name):
    """키-값 쌍 그룹을 자연어 텍스트로 변환"""
    try:
        text_parts = [f"다음은 {group_name} 관련 정보입니다:"]
        
        for kv in kv_group:
            key = kv.get('key', '')
            value = kv.get('value', '')
            confidence = kv.get('confidence', 1.0)
            
            if key and value:
                confidence_text = f" (신뢰도: {confidence:.2f})" if confidence < 0.9 else ""
                text_parts.append(f"- {key}: {value}{confidence_text}")
        
        return '\\n'.join(text_parts)
        
    except Exception as e:
        return f"{group_name} 정보 변환 실패: {str(e)}"

def validate_and_improve_chunks_azure(chunks):
    """청크 품질 검증 및 개선"""
    try:
        validated_chunks = []
        
        for chunk in chunks:
            # 최소 길이 체크
            if chunk.get('token_count', 0) < 10:
                continue
                
            # 텍스트 정제
            cleaned_text = clean_chunk_text_azure(chunk['text'])
            if not cleaned_text.strip():
                continue
                
            # 청크 개선
            improved_chunk = improve_chunk_quality_azure(chunk, cleaned_text)
            validated_chunks.append(improved_chunk)
        
        print(f"   ✅ 청크 검증 완료: {len(validated_chunks)}/{len(chunks)}개 유효")
        return validated_chunks
        
    except Exception as e:
        print(f"   ❌ 청크 검증 실패: {e}")
        return chunks

def clean_chunk_text_azure(text):
    """청크 텍스트 정제"""
    try:
        # 불필요한 공백 제거
        text = re.sub(r'\\s+', ' ', text)
        
        # 반복되는 특수문자 제거
        text = re.sub(r'[\\-_=]{3,}', '', text)
        
        # 의미없는 짧은 단어 제거
        text = re.sub(r'\\b[a-zA-Z]{1,2}\\b', '', text)
        
        return text.strip()
        
    except Exception as e:
        return text

def improve_chunk_quality_azure(chunk, cleaned_text):
    """청크 품질 개선"""
    try:
        improved_chunk = chunk.copy()
        improved_chunk['text'] = cleaned_text
        improved_chunk['token_count'] = count_tokens_korean(cleaned_text)
        
        # 품질 점수 계산
        quality_score = calculate_chunk_quality_score_azure(improved_chunk)
        improved_chunk['quality_score'] = quality_score
        
        return improved_chunk
        
    except Exception as e:
        return chunk

def calculate_chunk_quality_score_azure(chunk):
    """청크 품질 점수 계산"""
    try:
        score = 0.0
        
        # 길이 점수 (적절한 길이)
        token_count = chunk.get('token_count', 0)
        if 50 <= token_count <= 800:
            score += 2.0
        elif token_count > 10:
            score += 1.0
        
        # 키워드 다양성 점수
        keywords = chunk.get('keywords', [])
        score += min(len(set(keywords)), 3) * 0.5
        
        # 메타데이터 완성도 점수
        metadata = chunk.get('metadata', {})
        if metadata.get('page_number'):
            score += 0.5
        if metadata.get('confidence', 0) > 0.8:
            score += 0.5
        
        return min(score, 5.0)
        
    except Exception as e:
        return 1.0

print("📦 테이블 및 키-값 쌍 RAG 처리 함수 로드 완료")
print("   ✅ process_tables_for_rag_azure: 테이블 RAG 처리")
print("   ✅ process_key_values_for_rag_azure: 키-값 쌍 RAG 처리")
print("   ✅ validate_and_improve_chunks_azure: 청크 품질 검증")

📦 테이블 및 키-값 쌍 RAG 처리 함수 로드 완료
   ✅ process_tables_for_rag_azure: 테이블 RAG 처리
   ✅ process_key_values_for_rag_azure: 키-값 쌍 RAG 처리
   ✅ validate_and_improve_chunks_azure: 청크 품질 검증


## 7. 백엔드 시스템 연동 및 인증

Azure Document Intelligence로 처리된 문서 데이터를 기존 백엔드 시스템과 연동하고, 인증 및 세션 관리를 처리합니다.

### 기능:
- 기존 백엔드 API 엔드포인트 연동
- 사용자 인증 및 세션 관리
- 문서 업로드 및 처리 상태 추적
- RAG 데이터 저장 및 검색 인덱스 업데이트
- 에러 처리 및 재시도 로직

In [8]:
# 백엔드 시스템 연동 및 인증 함수들
import requests
import json
from urllib.parse import urljoin

# 백엔드 설정
BACKEND_BASE_URL = "http://localhost:8000"  # 백엔드 서버 URL
DEFAULT_HEADERS = {
    'Content-Type': 'application/json',
    'Accept': 'application/json'
}

# 세션 관리
session_manager = {
    'token': None,
    'user_id': None,
    'session_id': None,
    'authenticated': False
}

def authenticate_user_azure(username, password):
    """사용자 인증 및 토큰 획득"""
    try:
        print(f"🔐 사용자 인증 시도: {username}")
        
        login_data = {
            'username': username,
            'password': password
        }
        
        response = requests.post(
            urljoin(BACKEND_BASE_URL, '/api/auth/login'),
            json=login_data,
            headers=DEFAULT_HEADERS,
            timeout=30
        )
        
        if response.status_code == 200:
            auth_data = response.json()
            
            # 세션 정보 저장
            session_manager['token'] = auth_data.get('access_token')
            session_manager['user_id'] = auth_data.get('user_id')
            session_manager['session_id'] = auth_data.get('session_id')
            session_manager['authenticated'] = True
            
            print(f"✅ 인증 성공: 사용자 ID {session_manager['user_id']}")
            return {
                'success': True,
                'token': session_manager['token'],
                'user_id': session_manager['user_id']
            }
        else:
            print(f"❌ 인증 실패: {response.status_code} - {response.text}")
            return {
                'success': False,
                'error': f'인증 실패: {response.status_code}',
                'details': response.text
            }
            
    except Exception as e:
        print(f"❌ 인증 오류: {e}")
        return {
            'success': False,
            'error': str(e)
        }

def get_authenticated_headers():
    """인증 헤더 생성"""
    headers = DEFAULT_HEADERS.copy()
    if session_manager['token']:
        headers['Authorization'] = f"Bearer {session_manager['token']}"
    return headers

def upload_document_to_backend_azure(file_path, document_metadata=None):
    """문서를 백엔드에 업로드"""
    if not session_manager['authenticated']:
        return {
            'success': False,
            'error': '인증되지 않은 사용자입니다'
        }
    
    try:
        print(f"📤 문서 업로드 시작: {os.path.basename(file_path)}")
        
        # 파일 준비
        with open(file_path, 'rb') as file:
            files = {
                'file': (os.path.basename(file_path), file, 'application/pdf')
            }
            
            # 메타데이터 준비
            data = {
                'user_id': session_manager['user_id'],
                'processing_method': 'azure_document_intelligence',
                'metadata': json.dumps(document_metadata or {})
            }
            
            # 업로드 요청
            response = requests.post(
                urljoin(BACKEND_BASE_URL, '/api/documents/upload'),
                files=files,
                data=data,
                headers={'Authorization': f"Bearer {session_manager['token']}"},
                timeout=120
            )
        
        if response.status_code == 201:
            upload_result = response.json()
            print(f"✅ 업로드 성공: 문서 ID {upload_result.get('document_id')}")
            return {
                'success': True,
                'document_id': upload_result.get('document_id'),
                'upload_id': upload_result.get('upload_id'),
                'status': upload_result.get('status', 'uploaded')
            }
        else:
            print(f"❌ 업로드 실패: {response.status_code}")
            return {
                'success': False,
                'error': f'업로드 실패: {response.status_code}',
                'details': response.text
            }
            
    except Exception as e:
        print(f"❌ 업로드 오류: {e}")
        return {
            'success': False,
            'error': str(e)
        }

def send_chunks_to_backend_azure(chunks, document_id):
    """청크 데이터를 백엔드로 전송"""
    if not session_manager['authenticated']:
        return {
            'success': False,
            'error': '인증되지 않은 사용자입니다'
        }
    
    try:
        print(f"📊 청크 데이터 전송 시작: {len(chunks)}개 청크")
        
        chunk_data = {
            'document_id': document_id,
            'user_id': session_manager['user_id'],
            'chunks': chunks,
            'processing_metadata': {
                'extraction_method': 'azure_document_intelligence',
                'chunk_count': len(chunks),
                'processing_timestamp': datetime.now().isoformat()
            }
        }
        
        response = requests.post(
            urljoin(BACKEND_BASE_URL, '/api/documents/chunks'),
            json=chunk_data,
            headers=get_authenticated_headers(),
            timeout=120
        )
        
        if response.status_code == 201:
            result = response.json()
            print(f"✅ 청크 전송 성공: {result.get('stored_chunks')}개 저장됨")
            return {
                'success': True,
                'stored_chunks': result.get('stored_chunks'),
                'chunk_ids': result.get('chunk_ids', []),
                'indexing_status': result.get('indexing_status')
            }
        else:
            print(f"❌ 청크 전송 실패: {response.status_code}")
            return {
                'success': False,
                'error': f'청크 전송 실패: {response.status_code}',
                'details': response.text
            }
            
    except Exception as e:
        print(f"❌ 청크 전송 오류: {e}")
        return {
            'success': False,
            'error': str(e)
        }

def update_document_status_azure(document_id, status, processing_result=None):
    """문서 처리 상태 업데이트"""
    if not session_manager['authenticated']:
        return {
            'success': False,
            'error': '인증되지 않은 사용자입니다'
        }
    
    try:
        print(f"📝 문서 상태 업데이트: {document_id} -> {status}")
        
        update_data = {
            'status': status,
            'user_id': session_manager['user_id']
        }
        
        if processing_result:
            update_data['processing_result'] = processing_result
        
        response = requests.patch(
            urljoin(BACKEND_BASE_URL, f'/api/documents/{document_id}/status'),
            json=update_data,
            headers=get_authenticated_headers(),
            timeout=30
        )
        
        if response.status_code == 200:
            print(f"✅ 상태 업데이트 성공")
            return {
                'success': True,
                'updated_status': status
            }
        else:
            print(f"❌ 상태 업데이트 실패: {response.status_code}")
            return {
                'success': False,
                'error': f'상태 업데이트 실패: {response.status_code}'
            }
            
    except Exception as e:
        print(f"❌ 상태 업데이트 오류: {e}")
        return {
            'success': False,
            'error': str(e)
        }

def get_document_processing_status_azure(document_id):
    """문서 처리 상태 조회"""
    if not session_manager['authenticated']:
        return {
            'success': False,
            'error': '인증되지 않은 사용자입니다'
        }
    
    try:
        response = requests.get(
            urljoin(BACKEND_BASE_URL, f'/api/documents/{document_id}/status'),
            headers=get_authenticated_headers(),
            timeout=30
        )
        
        if response.status_code == 200:
            status_data = response.json()
            return {
                'success': True,
                'status': status_data.get('status'),
                'progress': status_data.get('progress', 0),
                'processing_time': status_data.get('processing_time'),
                'error_message': status_data.get('error_message')
            }
        else:
            return {
                'success': False,
                'error': f'상태 조회 실패: {response.status_code}'
            }
            
    except Exception as e:
        return {
            'success': False,
            'error': str(e)
        }

def test_search_integration_azure(query, document_id=None):
    """검색 기능 테스트"""
    if not session_manager['authenticated']:
        return {
            'success': False,
            'error': '인증되지 않은 사용자입니다'
        }
    
    try:
        print(f"🔍 검색 테스트: '{query}'")
        
        search_data = {
            'query': query,
            'user_id': session_manager['user_id'],
            'search_type': 'hybrid',
            'max_results': 10
        }
        
        if document_id:
            search_data['document_filter'] = [document_id]
        
        response = requests.post(
            urljoin(BACKEND_BASE_URL, '/api/search'),
            json=search_data,
            headers=get_authenticated_headers(),
            timeout=60
        )
        
        if response.status_code == 200:
            search_results = response.json()
            print(f"✅ 검색 완료: {len(search_results.get('results', []))}개 결과")
            return {
                'success': True,
                'results': search_results.get('results', []),
                'total_results': search_results.get('total', 0),
                'search_time': search_results.get('search_time')
            }
        else:
            print(f"❌ 검색 실패: {response.status_code}")
            return {
                'success': False,
                'error': f'검색 실패: {response.status_code}',
                'details': response.text
            }
            
    except Exception as e:
        print(f"❌ 검색 오류: {e}")
        return {
            'success': False,
            'error': str(e)
        }

def complete_document_processing_azure(file_path, username="testuser", password="testpass"):
    """완전한 문서 처리 워크플로우"""
    try:
        print("🚀 Azure Document Intelligence 완전 처리 워크플로우 시작")
        
        # 1. 사용자 인증
        auth_result = authenticate_user_azure(username, password)
        if not auth_result['success']:
            return auth_result
        
        # 2. 문서 업로드
        upload_result = upload_document_to_backend_azure(file_path)
        if not upload_result['success']:
            return upload_result
        
        document_id = upload_result['document_id']
        
        # 3. Azure Document Intelligence로 텍스트 추출
        extraction_result = extract_text_with_azure(file_path)
        if not extraction_result['success']:
            update_document_status_azure(document_id, 'extraction_failed', extraction_result)
            return extraction_result
        
        # 4. RAG용 청킹
        chunking_result = prepare_text_for_rag_azure(extraction_result)
        if not chunking_result['success']:
            update_document_status_azure(document_id, 'chunking_failed', chunking_result)
            return chunking_result
        
        # 5. 청크 데이터 백엔드 전송
        chunks_result = send_chunks_to_backend_azure(chunking_result['chunks'], document_id)
        if not chunks_result['success']:
            update_document_status_azure(document_id, 'storage_failed', chunks_result)
            return chunks_result
        
        # 6. 처리 완료 상태 업데이트
        final_result = {
            'extraction_metadata': extraction_result['metadata'],
            'chunking_metadata': chunking_result['processing_metadata'],
            'storage_metadata': chunks_result
        }
        
        update_result = update_document_status_azure(document_id, 'completed', final_result)
        
        print("✅ 완전 처리 워크플로우 완료")
        return {
            'success': True,
            'document_id': document_id,
            'extracted_pages': extraction_result['metadata']['page_count'],
            'created_chunks': chunking_result['total_chunks'],
            'stored_chunks': chunks_result['stored_chunks'],
            'processing_result': final_result
        }
        
    except Exception as e:
        print(f"❌ 완전 처리 워크플로우 실패: {e}")
        return {
            'success': False,
            'error': str(e)
        }

print("📦 백엔드 시스템 연동 함수 로드 완료")
print("   ✅ authenticate_user_azure: 사용자 인증")
print("   ✅ upload_document_to_backend_azure: 문서 업로드")
print("   ✅ send_chunks_to_backend_azure: 청크 데이터 전송")
print("   ✅ complete_document_processing_azure: 완전 처리 워크플로우")
print("   ✅ test_search_integration_azure: 검색 기능 테스트")

📦 백엔드 시스템 연동 함수 로드 완료
   ✅ authenticate_user_azure: 사용자 인증
   ✅ upload_document_to_backend_azure: 문서 업로드
   ✅ send_chunks_to_backend_azure: 청크 데이터 전송
   ✅ complete_document_processing_azure: 완전 처리 워크플로우
   ✅ test_search_integration_azure: 검색 기능 테스트


## 8. 결과 검증 및 품질 분석

Azure Document Intelligence의 처리 결과를 검증하고 품질을 분석합니다. 기존 Textract 결과와의 비교도 포함됩니다.

### 기능:
- 텍스트 추출 정확도 검증
- 한국어 처리 품질 평가
- 레이아웃 인식 정확도 측정
- Textract vs Azure 성능 비교
- 신뢰도 점수 분석

In [9]:
# 결과 검증 및 품질 분석 함수들
import difflib
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns

def validate_azure_extraction_quality(extraction_result, expected_content=None):
    """Azure 추출 결과 품질 검증"""
    try:
        print("🔍 Azure Document Intelligence 추출 품질 검증 시작")
        
        validation_result = {
            'overall_score': 0.0,
            'metrics': {},
            'issues': [],
            'recommendations': []
        }
        
        # 1. 기본 추출 성공 여부 검증
        if not extraction_result.get('success'):
            validation_result['issues'].append("추출 자체가 실패함")
            return validation_result
        
        # 2. 텍스트 품질 검증
        text_quality = validate_text_quality_azure(extraction_result.get('text', ''))
        validation_result['metrics']['text_quality'] = text_quality
        
        # 3. 한국어 처리 품질 검증
        korean_quality = validate_korean_processing_azure(extraction_result.get('text', ''))
        validation_result['metrics']['korean_quality'] = korean_quality
        
        # 4. 페이지별 처리 품질 검증
        page_quality = validate_page_processing_azure(extraction_result.get('page_results', []))
        validation_result['metrics']['page_quality'] = page_quality
        
        # 5. 구조적 요소 품질 검증
        structure_quality = validate_structure_processing_azure(extraction_result)
        validation_result['metrics']['structure_quality'] = structure_quality
        
        # 6. 신뢰도 점수 분석
        confidence_analysis = analyze_confidence_scores_azure(extraction_result)
        validation_result['metrics']['confidence_analysis'] = confidence_analysis
        
        # 7. 예상 내용과 비교 (있는 경우)
        if expected_content:
            content_similarity = compare_with_expected_content_azure(
                extraction_result.get('text', ''), 
                expected_content
            )
            validation_result['metrics']['content_similarity'] = content_similarity
        
        # 8. 전체 품질 점수 계산
        overall_score = calculate_overall_quality_score_azure(validation_result['metrics'])
        validation_result['overall_score'] = overall_score
        
        # 9. 개선 권장사항 생성
        recommendations = generate_quality_recommendations_azure(validation_result['metrics'])
        validation_result['recommendations'] = recommendations
        
        print(f"✅ 품질 검증 완료: 전체 점수 {overall_score:.2f}/5.0")
        return validation_result
        
    except Exception as e:
        print(f"❌ 품질 검증 실패: {e}")
        return {
            'overall_score': 0.0,
            'error': str(e)
        }

def validate_text_quality_azure(text):
    """텍스트 기본 품질 검증"""
    try:
        metrics = {
            'length': len(text),
            'word_count': len(text.split()),
            'line_count': len(text.split('\\n')),
            'empty_ratio': 0.0,
            'special_char_ratio': 0.0,
            'korean_ratio': 0.0
        }
        
        if metrics['length'] > 0:
            # 빈 줄 비율
            empty_lines = len([line for line in text.split('\\n') if not line.strip()])
            metrics['empty_ratio'] = empty_lines / metrics['line_count'] if metrics['line_count'] > 0 else 0
            
            # 특수문자 비율
            special_chars = len(re.findall(r'[^\\w\\s가-힣]', text))
            metrics['special_char_ratio'] = special_chars / metrics['length']
            
            # 한국어 비율
            korean_chars = len(re.findall(r'[가-힣]', text))
            metrics['korean_ratio'] = korean_chars / metrics['length']
        
        # 품질 점수 계산 (0-5)
        score = 0.0
        if metrics['length'] > 100:
            score += 1.0
        if metrics['word_count'] > 20:
            score += 1.0
        if metrics['empty_ratio'] < 0.3:
            score += 1.0
        if metrics['special_char_ratio'] < 0.1:
            score += 1.0
        if metrics['korean_ratio'] > 0.3:
            score += 1.0
        
        metrics['quality_score'] = score
        return metrics
        
    except Exception as e:
        return {'error': str(e), 'quality_score': 0.0}

def validate_korean_processing_azure(text):
    """한국어 처리 품질 검증"""
    try:
        metrics = {
            'morpheme_analysis_success': False,
            'proper_spacing': 0.0,
            'sentence_structure': 0.0,
            'vocabulary_richness': 0.0
        }
        
        # 형태소 분석 성공 여부
        try:
            morphemes = kiwi.analyze(text[:1000])  # 첫 1000자만 테스트
            metrics['morpheme_analysis_success'] = len(morphemes) > 0
        except:
            pass
        
        # 적절한 띄어쓰기 검증
        words = text.split()
        if words:
            avg_word_length = sum(len(word) for word in words) / len(words)
            metrics['proper_spacing'] = min(avg_word_length / 3.0, 1.0)  # 적절한 평균 단어 길이
        
        # 문장 구조 검증
        sentences = re.split(r'[.!?]', text)
        valid_sentences = [s for s in sentences if len(s.strip()) > 5]
        if sentences:
            metrics['sentence_structure'] = len(valid_sentences) / len(sentences)
        
        # 어휘 다양성
        if words:
            unique_words = set(words)
            metrics['vocabulary_richness'] = len(unique_words) / len(words)
        
        # 전체 한국어 처리 점수 (0-5)
        score = 0.0
        if metrics['morpheme_analysis_success']:
            score += 1.5
        score += metrics['proper_spacing'] * 1.0
        score += metrics['sentence_structure'] * 1.5
        score += metrics['vocabulary_richness'] * 1.0
        
        metrics['korean_quality_score'] = min(score, 5.0)
        return metrics
        
    except Exception as e:
        return {'error': str(e), 'korean_quality_score': 0.0}

def validate_page_processing_azure(page_results):
    """페이지별 처리 품질 검증"""
    try:
        if not page_results:
            return {'error': '페이지 결과 없음', 'page_quality_score': 0.0}
        
        metrics = {
            'page_count': len(page_results),
            'avg_confidence': 0.0,
            'layout_consistency': 0.0,
            'content_distribution': []
        }
        
        confidences = []
        content_lengths = []
        
        for page in page_results:
            if 'avg_confidence' in page:
                confidences.append(page['avg_confidence'])
            content_lengths.append(len(page.get('text', '')))
        
        # 평균 신뢰도
        if confidences:
            metrics['avg_confidence'] = sum(confidences) / len(confidences)
        
        # 콘텐츠 분포
        metrics['content_distribution'] = content_lengths
        
        # 레이아웃 일관성 (비슷한 길이의 페이지들)
        if content_lengths:
            avg_length = sum(content_lengths) / len(content_lengths)
            variance = sum((length - avg_length) ** 2 for length in content_lengths) / len(content_lengths)
            normalized_variance = variance / (avg_length ** 2) if avg_length > 0 else 1.0
            metrics['layout_consistency'] = max(0.0, 1.0 - normalized_variance)
        
        # 페이지 처리 품질 점수 (0-5)
        score = 0.0
        if metrics['page_count'] > 0:
            score += 1.0
        if metrics['avg_confidence'] > 0.8:
            score += 2.0
        elif metrics['avg_confidence'] > 0.6:
            score += 1.0
        score += metrics['layout_consistency'] * 2.0
        
        metrics['page_quality_score'] = min(score, 5.0)
        return metrics
        
    except Exception as e:
        return {'error': str(e), 'page_quality_score': 0.0}

def validate_structure_processing_azure(extraction_result):
    """구조적 요소 처리 품질 검증"""
    try:
        metrics = {
            'table_count': 0,
            'table_quality': 0.0,
            'kv_pair_count': 0,
            'kv_quality': 0.0
        }
        
        # 테이블 처리 품질
        tables = extraction_result.get('tables', [])
        metrics['table_count'] = len(tables)
        
        if tables:
            table_scores = []
            for table in tables:
                table_score = 0.0
                if table.get('row_count', 0) > 0:
                    table_score += 1.0
                if table.get('column_count', 0) > 0:
                    table_score += 1.0
                if table.get('csv') and len(table['csv']) > 0:
                    table_score += 1.0
                table_scores.append(table_score)
            
            metrics['table_quality'] = sum(table_scores) / len(table_scores) / 3.0
        
        # 키-값 쌍 처리 품질
        kv_pairs = extraction_result.get('key_value_pairs', [])
        metrics['kv_pair_count'] = len(kv_pairs)
        
        if kv_pairs:
            valid_kv_count = 0
            for kv in kv_pairs:
                if kv.get('key') and kv.get('value'):
                    valid_kv_count += 1
            
            metrics['kv_quality'] = valid_kv_count / len(kv_pairs)
        
        # 구조 처리 품질 점수 (0-5)
        score = 0.0
        if metrics['table_count'] > 0:
            score += 1.0 + metrics['table_quality'] * 2.0
        if metrics['kv_pair_count'] > 0:
            score += 1.0 + metrics['kv_quality'] * 1.0
        
        metrics['structure_quality_score'] = min(score, 5.0)
        return metrics
        
    except Exception as e:
        return {'error': str(e), 'structure_quality_score': 0.0}

def analyze_confidence_scores_azure(extraction_result):
    """신뢰도 점수 분석"""
    try:
        all_confidences = []
        
        # 페이지별 신뢰도 수집
        for page in extraction_result.get('page_results', []):
            if 'avg_confidence' in page:
                all_confidences.append(page['avg_confidence'])
        
        # 키-값 쌍 신뢰도 수집
        for kv in extraction_result.get('key_value_pairs', []):
            if 'confidence' in kv:
                all_confidences.append(kv['confidence'])
        
        if not all_confidences:
            return {'error': '신뢰도 데이터 없음', 'confidence_score': 0.0}
        
        metrics = {
            'avg_confidence': sum(all_confidences) / len(all_confidences),
            'min_confidence': min(all_confidences),
            'max_confidence': max(all_confidences),
            'confidence_variance': 0.0,
            'low_confidence_ratio': 0.0
        }
        
        # 신뢰도 분산
        avg = metrics['avg_confidence']
        variance = sum((conf - avg) ** 2 for conf in all_confidences) / len(all_confidences)
        metrics['confidence_variance'] = variance
        
        # 낮은 신뢰도 비율 (0.7 미만)
        low_conf_count = len([conf for conf in all_confidences if conf < 0.7])
        metrics['low_confidence_ratio'] = low_conf_count / len(all_confidences)
        
        # 신뢰도 분석 점수 (0-5)
        score = 0.0
        if metrics['avg_confidence'] > 0.9:
            score += 2.0
        elif metrics['avg_confidence'] > 0.8:
            score += 1.5
        elif metrics['avg_confidence'] > 0.7:
            score += 1.0
        
        if metrics['min_confidence'] > 0.5:
            score += 1.0
        
        if metrics['low_confidence_ratio'] < 0.2:
            score += 1.0
        
        if metrics['confidence_variance'] < 0.1:
            score += 1.0
        
        metrics['confidence_score'] = min(score, 5.0)
        return metrics
        
    except Exception as e:
        return {'error': str(e), 'confidence_score': 0.0}

def compare_with_expected_content_azure(extracted_text, expected_content):
    """예상 내용과의 유사도 비교"""
    try:
        # 텍스트 정규화
        extracted_normalized = re.sub(r'\\s+', ' ', extracted_text.lower().strip())
        expected_normalized = re.sub(r'\\s+', ' ', expected_content.lower().strip())
        
        # 유사도 계산
        similarity_ratio = difflib.SequenceMatcher(None, extracted_normalized, expected_normalized).ratio()
        
        # 단어 수준 비교
        extracted_words = set(extracted_normalized.split())
        expected_words = set(expected_normalized.split())
        
        word_overlap = len(extracted_words & expected_words)
        word_total = len(extracted_words | expected_words)
        word_similarity = word_overlap / word_total if word_total > 0 else 0.0
        
        # 길이 비교
        length_ratio = min(len(extracted_text), len(expected_content)) / max(len(extracted_text), len(expected_content))
        
        metrics = {
            'similarity_ratio': similarity_ratio,
            'word_similarity': word_similarity,
            'length_ratio': length_ratio,
            'extracted_length': len(extracted_text),
            'expected_length': len(expected_content)
        }
        
        # 내용 유사도 점수 (0-5)
        score = similarity_ratio * 2.0 + word_similarity * 2.0 + length_ratio * 1.0
        metrics['content_similarity_score'] = min(score, 5.0)
        
        return metrics
        
    except Exception as e:
        return {'error': str(e), 'content_similarity_score': 0.0}

def calculate_overall_quality_score_azure(metrics):
    """전체 품질 점수 계산"""
    try:
        scores = []
        weights = []
        
        # 각 메트릭의 점수와 가중치
        metric_weights = {
            'text_quality': (0.2, 'quality_score'),
            'korean_quality': (0.3, 'korean_quality_score'),
            'page_quality': (0.2, 'page_quality_score'),
            'structure_quality': (0.1, 'structure_quality_score'),
            'confidence_analysis': (0.2, 'confidence_score')
        }
        
        for metric_name, (weight, score_key) in metric_weights.items():
            if metric_name in metrics and score_key in metrics[metric_name]:
                scores.append(metrics[metric_name][score_key])
                weights.append(weight)
        
        if not scores:
            return 0.0
        
        # 가중 평균 계산
        weighted_sum = sum(score * weight for score, weight in zip(scores, weights))
        total_weight = sum(weights)
        
        return weighted_sum / total_weight if total_weight > 0 else 0.0
        
    except Exception as e:
        return 0.0

def generate_quality_recommendations_azure(metrics):
    """품질 개선 권장사항 생성"""
    recommendations = []
    
    try:
        # 텍스트 품질 관련
        text_quality = metrics.get('text_quality', {})
        if text_quality.get('quality_score', 0) < 3.0:
            if text_quality.get('length', 0) < 100:
                recommendations.append("추출된 텍스트 길이가 너무 짧습니다. 문서 스캔 품질을 확인하세요.")
            if text_quality.get('korean_ratio', 0) < 0.3:
                recommendations.append("한국어 비율이 낮습니다. 문서가 한국어 문서인지 확인하세요.")
        
        # 한국어 처리 품질 관련
        korean_quality = metrics.get('korean_quality', {})
        if korean_quality.get('korean_quality_score', 0) < 3.0:
            if not korean_quality.get('morpheme_analysis_success', False):
                recommendations.append("형태소 분석이 실패했습니다. 텍스트 전처리가 필요할 수 있습니다.")
            if korean_quality.get('proper_spacing', 0) < 0.5:
                recommendations.append("띄어쓰기가 부적절합니다. 후처리에서 띄어쓰기 교정을 고려하세요.")
        
        # 신뢰도 관련
        confidence_analysis = metrics.get('confidence_analysis', {})
        if confidence_analysis.get('confidence_score', 0) < 3.0:
            if confidence_analysis.get('avg_confidence', 0) < 0.8:
                recommendations.append("전체적인 신뢰도가 낮습니다. 문서 해상도를 높이거나 다른 모델을 시도해보세요.")
            if confidence_analysis.get('low_confidence_ratio', 0) > 0.3:
                recommendations.append("낮은 신뢰도 구간이 많습니다. 해당 구간을 수동으로 검토하세요.")
        
        if not recommendations:
            recommendations.append("품질이 양호합니다. 추가 개선이 필요하지 않습니다.")
        
        return recommendations
        
    except Exception as e:
        return [f"권장사항 생성 실패: {str(e)}"]

print("📦 결과 검증 및 품질 분석 함수 로드 완료")
print("   ✅ validate_azure_extraction_quality: 전체 품질 검증")
print("   ✅ validate_korean_processing_azure: 한국어 처리 품질")
print("   ✅ analyze_confidence_scores_azure: 신뢰도 분석")
print("   ✅ generate_quality_recommendations_azure: 개선 권장사항")

📦 결과 검증 및 품질 분석 함수 로드 완료
   ✅ validate_azure_extraction_quality: 전체 품질 검증
   ✅ validate_korean_processing_azure: 한국어 처리 품질
   ✅ analyze_confidence_scores_azure: 신뢰도 분석
   ✅ generate_quality_recommendations_azure: 개선 권장사항


## 9. 실제 테스트 실행 및 성능 분석

Azure Document Intelligence의 실제 성능을 측정하고 기존 Textract와 비교 분석합니다.

### 기능:
- 실제 한국어 문서로 테스트 실행
- 처리 시간 및 정확도 측정
- Textract 대비 성능 비교
- 메모리 및 리소스 사용량 분석
- 최종 권장사항 제시

In [10]:
# 실제 테스트 문서 처리 및 페이지별 결과 분석
import pandas as pd
import json
from pathlib import Path

def run_azure_document_test(file_path):
    """Azure Document Intelligence 테스트 실행"""
    try:
        print(f"🚀 Azure Document Intelligence 테스트 시작")
        print(f"📄 테스트 파일: {os.path.basename(file_path)}")
        print("-" * 60)
        
        # 1. 문서 복잡도 분석
        print("1️⃣ 문서 복잡도 분석")
        complexity_info = detect_document_complexity_azure(file_path)
        print(f"   📊 복잡도: {complexity_info['complexity_level']}")
        print(f"   📄 페이지: {complexity_info['estimated_pages']}개")
        print(f"   🔧 권장 모델: {complexity_info['recommended_model']}")
        print()
        
        # 2. 텍스트 추출 실행
        print("2️⃣ Azure Document Intelligence 텍스트 추출")
        extraction_result = extract_text_with_azure(file_path)
        
        if not extraction_result['success']:
            print(f"❌ 추출 실패: {extraction_result['error']}")
            return extraction_result
        
        print(f"   ✅ 추출 성공!")
        print(f"   📄 페이지: {extraction_result['metadata']['page_count']}개")
        print(f"   📝 텍스트: {len(extraction_result['text'])}자")
        print(f"   ⏱️ 처리시간: {extraction_result.get('processing_time', 0):.1f}초")
        print()
        
        # 3. 페이지별 상세 분석
        print("3️⃣ 페이지별 상세 분석")
        page_analysis = analyze_pages_detailed_azure(extraction_result)
        print()
        
        # 4. RAG 청킹 처리
        print("4️⃣ RAG용 텍스트 청킹")
        chunking_result = prepare_text_for_rag_azure(extraction_result)
        
        if chunking_result['success']:
            print(f"   ✅ 청킹 완료!")
            print(f"   📦 청크: {chunking_result['total_chunks']}개")
            print(f"   🔢 토큰: {chunking_result['total_tokens']}개")
            print(f"   📏 평균 크기: {chunking_result['avg_chunk_size']:.1f} 토큰")
        else:
            print(f"   ❌ 청킹 실패: {chunking_result['error']}")
        print()
        
        # 5. 품질 검증
        print("5️⃣ 추출 품질 검증")
        quality_result = validate_azure_extraction_quality(extraction_result)
        print(f"   📊 전체 품질 점수: {quality_result['overall_score']:.2f}/5.0")
        print()
        
        # 전체 결과 반환
        return {
            'success': True,
            'file_path': file_path,
            'complexity_analysis': complexity_info,
            'extraction_result': extraction_result,
            'page_analysis': page_analysis,
            'chunking_result': chunking_result,
            'quality_assessment': quality_result,
            'test_timestamp': datetime.now().isoformat()
        }
        
    except Exception as e:
        print(f"❌ 테스트 실행 실패: {e}")
        return {
            'success': False,
            'error': str(e)
        }

# 테스트 파일 경로 설정 (실제 사용시 수정 필요)
TEST_DOCUMENTS = [
    '/home/admin/wkms-aws/test_template.pdf',
    '/home/admin/wkms-aws/test_template2.pdf',
    '/home/admin/wkms-aws/test_template3.pdf'
]

print("📦 Azure Document Intelligence 테스트 함수 로드 완료")
print("   ✅ run_azure_document_test: 전체 테스트 실행")
print("   📋 TEST_DOCUMENTS: 테스트 문서 목록 설정됨")
print()
print("💡 사용법:")
print("   # 단일 문서 테스트")
print("   test_result = run_azure_document_test('/path/to/document.pdf')")
print("   # 결과 확인")
print("   print(test_result['success'])")

📦 Azure Document Intelligence 테스트 함수 로드 완료
   ✅ run_azure_document_test: 전체 테스트 실행
   📋 TEST_DOCUMENTS: 테스트 문서 목록 설정됨

💡 사용법:
   # 단일 문서 테스트
   test_result = run_azure_document_test('/path/to/document.pdf')
   # 결과 확인
   print(test_result['success'])


In [11]:
# 페이지별 상세 분석 함수들

def analyze_pages_detailed_azure(extraction_result):
    """페이지별 상세 분석 및 시각화"""
    try:
        page_results = extraction_result.get('page_results', [])
        if not page_results:
            print("   ❌ 페이지 데이터 없음")
            return {}
        
        analysis_summary = {
            'total_pages': len(page_results),
            'page_details': [],
            'statistics': {}
        }
        
        print(f"   📄 총 {len(page_results)}개 페이지 분석")
        
        # 페이지별 상세 정보
        for i, page in enumerate(page_results, 1):
            page_detail = analyze_single_page_azure(page, i)
            analysis_summary['page_details'].append(page_detail)
            
            # 페이지 요약 출력
            print(f"      페이지 {i:2d}: {page_detail['text_length']:5,}자 | "
                  f"레이아웃: {page_detail['layout_type']:12s} | "
                  f"신뢰도: {page_detail['confidence']:.2f} | "
                  f"문단: {page_detail['paragraph_count']:2d}개")
        
        # 전체 통계 계산
        analysis_summary['statistics'] = calculate_page_statistics_azure(analysis_summary['page_details'])
        print_page_statistics_azure(analysis_summary['statistics'])
        
        return analysis_summary
        
    except Exception as e:
        print(f"   ❌ 페이지 분석 실패: {e}")
        return {'error': str(e)}

def analyze_single_page_azure(page_result, page_number):
    """단일 페이지 상세 분석"""
    try:
        page_text = page_result.get('text', '')
        
        # 기본 정보
        page_detail = {
            'page_number': page_number,
            'text_length': len(page_text),
            'layout_type': page_result.get('layout_type', 'unknown'),
            'confidence': page_result.get('avg_confidence', 0.0),
            'paragraph_count': page_result.get('paragraph_count', 0),
            'table_count': page_result.get('table_count', 0),
            'element_count': page_result.get('element_count', 0)
        }
        
        # 텍스트 분석
        if page_text:
            # 문장 수 계산
            sentences = re.split(r'[.!?。]\s+', page_text)
            page_detail['sentence_count'] = len([s for s in sentences if s.strip()])
            
            # 단어 수 계산
            words = page_text.split()
            page_detail['word_count'] = len(words)
            
            # 한국어 비율
            korean_chars = len(re.findall(r'[가-힣]', page_text))
            page_detail['korean_ratio'] = korean_chars / len(page_text) if page_text else 0.0
            
            # 숫자/영문 비율
            number_chars = len(re.findall(r'[0-9]', page_text))
            english_chars = len(re.findall(r'[a-zA-Z]', page_text))
            page_detail['number_ratio'] = number_chars / len(page_text) if page_text else 0.0
            page_detail['english_ratio'] = english_chars / len(page_text) if page_text else 0.0
            
            # 빈 줄 비율
            lines = page_text.split('\\n')
            empty_lines = len([line for line in lines if not line.strip()])
            page_detail['empty_line_ratio'] = empty_lines / len(lines) if lines else 0.0
            
            # 키워드 추출
            page_detail['keywords'] = extract_keywords_from_chunk_azure(page_text)[:5]
        else:
            # 빈 페이지 처리
            page_detail.update({
                'sentence_count': 0,
                'word_count': 0,
                'korean_ratio': 0.0,
                'number_ratio': 0.0,
                'english_ratio': 0.0,
                'empty_line_ratio': 1.0,
                'keywords': []
            })
        
        return page_detail
        
    except Exception as e:
        return {
            'page_number': page_number,
            'error': str(e),
            'text_length': 0
        }

def calculate_page_statistics_azure(page_details):
    """페이지별 상세 분석의 전체 통계 계산"""
    try:
        if not page_details:
            return {}
        
        # 유효한 페이지만 필터링
        valid_pages = [p for p in page_details if 'error' not in p]
        
        if not valid_pages:
            return {'error': '유효한 페이지 없음'}
        
        # 기본 통계
        text_lengths = [p['text_length'] for p in valid_pages]
        confidences = [p['confidence'] for p in valid_pages if p['confidence'] > 0]
        korean_ratios = [p['korean_ratio'] for p in valid_pages]
        
        statistics = {
            'total_pages': len(page_details),
            'valid_pages': len(valid_pages),
            'text_statistics': {
                'total_characters': sum(text_lengths),
                'avg_characters_per_page': sum(text_lengths) / len(valid_pages),
                'min_characters': min(text_lengths),
                'max_characters': max(text_lengths),
                'empty_pages': len([p for p in valid_pages if p['text_length'] == 0])
            },
            'confidence_statistics': {},
            'content_statistics': {},
            'layout_distribution': {}
        }
        
        # 신뢰도 통계
        if confidences:
            statistics['confidence_statistics'] = {
                'avg_confidence': sum(confidences) / len(confidences),
                'min_confidence': min(confidences),
                'max_confidence': max(confidences),
                'low_confidence_pages': len([c for c in confidences if c < 0.7])
            }
        
        # 콘텐츠 통계
        total_words = sum(p['word_count'] for p in valid_pages if 'word_count' in p)
        total_sentences = sum(p['sentence_count'] for p in valid_pages if 'sentence_count' in p)
        
        statistics['content_statistics'] = {
            'total_words': total_words,
            'total_sentences': total_sentences,
            'avg_korean_ratio': sum(korean_ratios) / len(korean_ratios),
            'pages_with_tables': len([p for p in valid_pages if p.get('table_count', 0) > 0]),
            'total_tables': sum(p.get('table_count', 0) for p in valid_pages)
        }
        
        # 레이아웃 분포
        layout_types = [p['layout_type'] for p in valid_pages]
        layout_counter = Counter(layout_types)
        statistics['layout_distribution'] = dict(layout_counter)
        
        return statistics
        
    except Exception as e:
        return {'error': str(e)}

def print_page_statistics_azure(statistics):
    """페이지 통계 결과 출력"""
    try:
        if 'error' in statistics:
            print(f"   ❌ 통계 계산 실패: {statistics['error']}")
            return
        
        print("   📊 페이지 분석 통계:")
        
        # 텍스트 통계
        text_stats = statistics.get('text_statistics', {})
        print(f"      📝 총 문자: {text_stats.get('total_characters', 0):,}자")
        print(f"      📄 페이지당 평균: {text_stats.get('avg_characters_per_page', 0):,.0f}자")
        print(f"      📏 최소/최대: {text_stats.get('min_characters', 0):,}자 / {text_stats.get('max_characters', 0):,}자")
        
        if text_stats.get('empty_pages', 0) > 0:
            print(f"      🚫 빈 페이지: {text_stats['empty_pages']}개")
        
        # 신뢰도 통계
        conf_stats = statistics.get('confidence_statistics', {})
        if conf_stats:
            print(f"      🎯 평균 신뢰도: {conf_stats.get('avg_confidence', 0):.3f}")
            print(f"      📉 최소/최대: {conf_stats.get('min_confidence', 0):.3f} / {conf_stats.get('max_confidence', 0):.3f}")
            
            if conf_stats.get('low_confidence_pages', 0) > 0:
                print(f"      ⚠️ 낮은 신뢰도 페이지: {conf_stats['low_confidence_pages']}개")
        
        # 콘텐츠 통계
        content_stats = statistics.get('content_statistics', {})
        if content_stats:
            print(f"      🔤 총 단어: {content_stats.get('total_words', 0):,}개")
            print(f"      📖 총 문장: {content_stats.get('total_sentences', 0):,}개")
            print(f"      🇰🇷 한국어 비율: {content_stats.get('avg_korean_ratio', 0):.1%}")
            
            if content_stats.get('total_tables', 0) > 0:
                print(f"      📊 테이블: {content_stats['total_tables']}개 ({content_stats.get('pages_with_tables', 0)}페이지)")
        
        # 레이아웃 분포
        layout_dist = statistics.get('layout_distribution', {})
        if layout_dist:
            print(f"      🏗️ 레이아웃 분포:")
            for layout_type, count in layout_dist.items():
                print(f"         {layout_type}: {count}페이지")
        
    except Exception as e:
        print(f"   ❌ 통계 출력 실패: {e}")

print("📦 페이지별 상세 분석 함수 로드 완료")
print("   ✅ analyze_pages_detailed_azure: 전체 페이지 분석")
print("   ✅ analyze_single_page_azure: 단일 페이지 분석")
print("   ✅ calculate_page_statistics_azure: 페이지 통계 계산")
print("   ✅ print_page_statistics_azure: 통계 결과 출력")

📦 페이지별 상세 분석 함수 로드 완료
   ✅ analyze_pages_detailed_azure: 전체 페이지 분석
   ✅ analyze_single_page_azure: 단일 페이지 분석
   ✅ calculate_page_statistics_azure: 페이지 통계 계산
   ✅ print_page_statistics_azure: 통계 결과 출력


In [12]:
# 페이지별 텍스트 내용 확인 함수들

def display_page_content_azure(extraction_result, page_number=None, max_chars=1000):
    """지정된 페이지의 텍스트 내용 표시"""
    try:
        page_results = extraction_result.get('page_results', [])
        
        if not page_results:
            print("❌ 페이지 데이터가 없습니다.")
            return
        
        if page_number is None:
            # 모든 페이지 표시
            print(f"📄 전체 {len(page_results)}개 페이지 내용:")
            print("=" * 80)
            
            for i, page in enumerate(page_results, 1):
                display_single_page_content_azure(page, i, max_chars)
                if i < len(page_results):
                    print("\\n" + "-" * 80 + "\\n")
        else:
            # 특정 페이지 표시
            if 1 <= page_number <= len(page_results):
                page = page_results[page_number - 1]
                display_single_page_content_azure(page, page_number, max_chars)
            else:
                print(f"❌ 잘못된 페이지 번호: {page_number} (1-{len(page_results)} 범위)")
        
    except Exception as e:
        print(f"❌ 페이지 내용 표시 실패: {e}")

def display_single_page_content_azure(page_result, page_number, max_chars=1000):
    """단일 페이지 내용 표시"""
    try:
        page_text = page_result.get('text', '')
        layout_type = page_result.get('layout_type', 'unknown')
        confidence = page_result.get('avg_confidence', 0.0)
        
        # 페이지 헤더
        print(f"📄 페이지 {page_number}")
        print(f"   🏗️ 레이아웃: {layout_type}")
        print(f"   🎯 신뢰도: {confidence:.3f}")
        print(f"   📝 길이: {len(page_text):,}자")
        
        if page_result.get('table_count', 0) > 0:
            print(f"   📊 테이블: {page_result['table_count']}개")
        
        print()
        
        # 텍스트 내용
        if page_text:
            if len(page_text) > max_chars:
                print(f"📝 텍스트 내용 (첫 {max_chars}자):")
                display_text = page_text[:max_chars] + "\\n\\n... (생략) ..."
            else:
                print(f"📝 텍스트 내용 (전체):")
                display_text = page_text
            
            print("┌" + "─" * 78 + "┐")
            for line in display_text.split('\\n'):
                # 긴 줄은 자르기
                if len(line) > 76:
                    line = line[:73] + "..."
                print(f"│ {line:<76} │")
            print("└" + "─" * 78 + "┘")
            
            # 키워드 표시
            keywords = page_result.get('keywords', [])
            if keywords:
                print(f"🔑 주요 키워드: {', '.join(keywords[:5])}")
        else:
            print("📝 텍스트 내용: (빈 페이지)")
        
    except Exception as e:
        print(f"❌ 페이지 {page_number} 표시 실패: {e}")

def compare_pages_azure(extraction_result, page1, page2):
    """두 페이지 간 비교 분석"""
    try:
        page_results = extraction_result.get('page_results', [])
        
        if not page_results or len(page_results) < max(page1, page2):
            print("❌ 유효하지 않은 페이지 번호입니다.")
            return
        
        p1 = page_results[page1 - 1]
        p2 = page_results[page2 - 1]
        
        print(f"🔄 페이지 {page1} vs 페이지 {page2} 비교")
        print("=" * 60)
        
        # 기본 정보 비교
        comparison_data = {
            '항목': ['텍스트 길이', '레이아웃 타입', '신뢰도', '단어 수', '문단 수', '테이블 수'],
            f'페이지 {page1}': [
                f"{len(p1.get('text', '')):,}자",
                p1.get('layout_type', 'unknown'),
                f"{p1.get('avg_confidence', 0.0):.3f}",
                f"{p1.get('word_count', 0):,}개",
                f"{p1.get('paragraph_count', 0)}개",
                f"{p1.get('table_count', 0)}개"
            ],
            f'페이지 {page2}': [
                f"{len(p2.get('text', '')):,}자",
                p2.get('layout_type', 'unknown'),
                f"{p2.get('avg_confidence', 0.0):.3f}",
                f"{p2.get('word_count', 0):,}개",
                f"{p2.get('paragraph_count', 0)}개",
                f"{p2.get('table_count', 0)}개"
            ]
        }
        
        # 테이블 형태로 출력
        df = pd.DataFrame(comparison_data)
        print(df.to_string(index=False))
        print()
        
        # 텍스트 유사도 계산
        text1 = p1.get('text', '')
        text2 = p2.get('text', '')
        
        if text1 and text2:
            similarity = difflib.SequenceMatcher(None, text1, text2).ratio()
            print(f"📊 텍스트 유사도: {similarity:.2%}")
            
            # 공통 키워드 찾기
            keywords1 = set(p1.get('keywords', []))
            keywords2 = set(p2.get('keywords', []))
            common_keywords = keywords1 & keywords2
            
            if common_keywords:
                print(f"🔑 공통 키워드: {', '.join(common_keywords)}")
        
    except Exception as e:
        print(f"❌ 페이지 비교 실패: {e}")

def export_page_analysis_azure(extraction_result, output_dir="./azure_analysis_results"):
    """페이지 분석 결과를 파일로 내보내기"""
    try:
        # 출력 디렉토리 생성
        Path(output_dir).mkdir(exist_ok=True)
        
        page_results = extraction_result.get('page_results', [])
        if not page_results:
            print("❌ 내보낼 페이지 데이터가 없습니다.")
            return
        
        print(f"📤 페이지 분석 결과 내보내기: {output_dir}")
        
        # 1. 전체 요약 JSON
        summary_data = {
            'document_metadata': extraction_result.get('metadata', {}),
            'page_count': len(page_results),
            'processing_timestamp': datetime.now().isoformat(),
            'total_text_length': len(extraction_result.get('text', '')),
            'pages_summary': []
        }
        
        # 2. 페이지별 상세 데이터
        for i, page in enumerate(page_results, 1):
            page_summary = {
                'page_number': i,
                'text_length': len(page.get('text', '')),
                'layout_type': page.get('layout_type', 'unknown'),
                'confidence': page.get('avg_confidence', 0.0),
                'element_count': page.get('element_count', 0),
                'table_count': page.get('table_count', 0),
                'paragraph_count': page.get('paragraph_count', 0)
            }
            summary_data['pages_summary'].append(page_summary)
            
            # 개별 페이지 텍스트 파일 저장
            page_text_file = Path(output_dir) / f"page_{i:02d}_text.txt"
            with open(page_text_file, 'w', encoding='utf-8') as f:
                f.write(f"페이지 {i} - {page.get('layout_type', 'unknown')} 레이아웃\\n")
                f.write(f"신뢰도: {page.get('avg_confidence', 0.0):.3f}\\n")
                f.write("=" * 60 + "\\n\\n")
                f.write(page.get('text', ''))
        
        # 요약 JSON 저장
        summary_file = Path(output_dir) / "document_analysis_summary.json"
        with open(summary_file, 'w', encoding='utf-8') as f:
            json.dump(summary_data, f, ensure_ascii=False, indent=2)
        
        # CSV 요약 저장
        if summary_data['pages_summary']:
            df = pd.DataFrame(summary_data['pages_summary'])
            csv_file = Path(output_dir) / "pages_summary.csv"
            df.to_csv(csv_file, index=False, encoding='utf-8')
        
        # 전체 텍스트 저장
        full_text_file = Path(output_dir) / "full_document_text.txt"
        with open(full_text_file, 'w', encoding='utf-8') as f:
            f.write(extraction_result.get('text', ''))
        
        print(f"   ✅ 요약 JSON: document_analysis_summary.json")
        print(f"   ✅ 페이지 CSV: pages_summary.csv")
        print(f"   ✅ 전체 텍스트: full_document_text.txt")
        print(f"   ✅ 페이지별 텍스트: page_XX_text.txt ({len(page_results)}개)")
        
        return {
            'success': True,
            'output_directory': output_dir,
            'files_created': len(page_results) + 3
        }
        
    except Exception as e:
        print(f"❌ 결과 내보내기 실패: {e}")
        return {
            'success': False,
            'error': str(e)
        }

print("📦 페이지별 내용 확인 함수 로드 완료")
print("   ✅ display_page_content_azure: 페이지 내용 표시")
print("   ✅ compare_pages_azure: 페이지 간 비교")
print("   ✅ export_page_analysis_azure: 분석 결과 내보내기")
print()
print("💡 사용법:")
print("   # 모든 페이지 내용 보기")
print("   display_page_content_azure(extraction_result)")
print("   # 특정 페이지만 보기")
print("   display_page_content_azure(extraction_result, page_number=1)")
print("   # 두 페이지 비교")
print("   compare_pages_azure(extraction_result, 1, 2)")
print("   # 결과 내보내기")
print("   export_page_analysis_azure(extraction_result)")

📦 페이지별 내용 확인 함수 로드 완료
   ✅ display_page_content_azure: 페이지 내용 표시
   ✅ compare_pages_azure: 페이지 간 비교
   ✅ export_page_analysis_azure: 분석 결과 내보내기

💡 사용법:
   # 모든 페이지 내용 보기
   display_page_content_azure(extraction_result)
   # 특정 페이지만 보기
   display_page_content_azure(extraction_result, page_number=1)
   # 두 페이지 비교
   compare_pages_azure(extraction_result, 1, 2)
   # 결과 내보내기
   export_page_analysis_azure(extraction_result)


In [13]:
# 페이지별 품질 검증 및 문제점 분석

def analyze_page_quality_issues_azure(extraction_result):
    """페이지별 품질 문제점 분석"""
    try:
        page_results = extraction_result.get('page_results', [])
        if not page_results:
            print("❌ 분석할 페이지 데이터가 없습니다.")
            return {}
        
        print(f"🔍 페이지별 품질 문제점 분석 ({len(page_results)}개 페이지)")
        print("=" * 70)
        
        quality_issues = {
            'total_pages': len(page_results),
            'problematic_pages': [],
            'issue_summary': {
                'low_confidence': 0,
                'empty_pages': 0,
                'layout_issues': 0,
                'text_quality_issues': 0
            }
        }
        
        for i, page in enumerate(page_results, 1):
            page_issues = check_single_page_quality_azure(page, i)
            
            if page_issues['has_issues']:
                quality_issues['problematic_pages'].append(page_issues)
                
                # 이슈 카운트 업데이트
                for issue_type in page_issues['issues']:
                    if 'confidence' in issue_type:
                        quality_issues['issue_summary']['low_confidence'] += 1
                    elif 'empty' in issue_type:
                        quality_issues['issue_summary']['empty_pages'] += 1
                    elif 'layout' in issue_type:
                        quality_issues['issue_summary']['layout_issues'] += 1
                    elif 'text' in issue_type:
                        quality_issues['issue_summary']['text_quality_issues'] += 1
            
            # 페이지별 결과 출력
            status = "❌ 문제" if page_issues['has_issues'] else "✅ 정상"
            confidence = page.get('avg_confidence', 0.0)
            text_len = len(page.get('text', ''))
            
            print(f"페이지 {i:2d}: {status} | 신뢰도: {confidence:.3f} | 텍스트: {text_len:,}자")
            
            if page_issues['has_issues']:
                for issue in page_issues['issues']:
                    print(f"         • {issue}")
        
        # 요약 출력
        print("\\n📊 품질 문제 요약:")
        print(f"   🔴 문제 페이지: {len(quality_issues['problematic_pages'])}/{quality_issues['total_pages']}개")
        print(f"   📉 낮은 신뢰도: {quality_issues['issue_summary']['low_confidence']}개")
        print(f"   📄 빈 페이지: {quality_issues['issue_summary']['empty_pages']}개")
        print(f"   🏗️ 레이아웃 문제: {quality_issues['issue_summary']['layout_issues']}개")
        print(f"   📝 텍스트 품질 문제: {quality_issues['issue_summary']['text_quality_issues']}개")
        
        return quality_issues
        
    except Exception as e:
        print(f"❌ 품질 분석 실패: {e}")
        return {'error': str(e)}

def check_single_page_quality_azure(page_result, page_number):
    """단일 페이지 품질 문제 확인"""
    try:
        issues = []
        page_text = page_result.get('text', '')
        confidence = page_result.get('avg_confidence', 0.0)
        layout_type = page_result.get('layout_type', 'unknown')
        
        # 1. 신뢰도 문제
        if confidence < 0.7:
            if confidence < 0.5:
                issues.append(f"매우 낮은 신뢰도 ({confidence:.3f})")
            else:
                issues.append(f"낮은 신뢰도 ({confidence:.3f})")
        
        # 2. 빈 페이지 문제
        if not page_text or len(page_text.strip()) < 10:
            issues.append("빈 페이지 또는 텍스트 부족")
        
        # 3. 레이아웃 문제
        if layout_type == 'unknown':
            issues.append("레이아웃 타입 인식 실패")
        
        # 4. 텍스트 품질 문제
        if page_text:
            # 한글 비율 확인
            korean_chars = len(re.findall(r'[가-힣]', page_text))
            korean_ratio = korean_chars / len(page_text)
            
            if korean_ratio < 0.1:
                issues.append(f"한국어 비율 낮음 ({korean_ratio:.1%})")
            
            # 특수 문자 과다
            special_chars = len(re.findall(r'[^\\w\\s가-힣]', page_text))
            special_ratio = special_chars / len(page_text)
            
            if special_ratio > 0.3:
                issues.append(f"특수 문자 과다 ({special_ratio:.1%})")
            
            # 반복 문자 확인
            if re.search(r'(.)\\1{5,}', page_text):
                issues.append("반복 문자 패턴 발견")
            
            # 너무 긴 단어 (OCR 오류 가능성)
            words = page_text.split()
            long_words = [w for w in words if len(w) > 20]
            if len(long_words) > 5:
                issues.append(f"비정상적으로 긴 단어 {len(long_words)}개")
        
        # 5. 구조적 문제
        element_count = page_result.get('element_count', 0)
        if element_count == 0 and page_text:
            issues.append("구조 요소 인식 실패")
        
        return {
            'page_number': page_number,
            'has_issues': len(issues) > 0,
            'issues': issues,
            'issue_count': len(issues),
            'confidence': confidence,
            'text_length': len(page_text)
        }
        
    except Exception as e:
        return {
            'page_number': page_number,
            'has_issues': True,
            'issues': [f"품질 검사 오류: {str(e)}"],
            'error': str(e)
        }

def suggest_page_improvements_azure(quality_issues):
    """페이지별 개선 방안 제안"""
    try:
        if not quality_issues or 'problematic_pages' not in quality_issues:
            print("📋 개선 방안: 모든 페이지가 양호한 품질입니다.")
            return
        
        problematic_pages = quality_issues['problematic_pages']
        if not problematic_pages:
            print("📋 개선 방안: 문제가 있는 페이지가 없습니다.")
            return
        
        print(f"📋 페이지별 개선 방안 ({len(problematic_pages)}개 페이지)")
        print("=" * 60)
        
        for page_issue in problematic_pages:
            page_num = page_issue['page_number']
            issues = page_issue['issues']
            
            print(f"\\n📄 페이지 {page_num}:")
            print(f"   🔴 문제점: {len(issues)}개")
            
            for issue in issues:
                print(f"      • {issue}")
            
            print(f"   💡 개선 방안:")
            
            # 이슈별 맞춤 개선 방안
            for issue in issues:
                if '신뢰도' in issue:
                    print(f"      ✓ 문서 해상도 향상 또는 스캔 품질 개선")
                    print(f"      ✓ 다른 Azure 모델(prebuilt-layout) 시도")
                elif '빈 페이지' in issue:
                    print(f"      ✓ 페이지 이미지 품질 확인")
                    print(f"      ✓ 문서 방향 및 회전 상태 점검")
                elif '레이아웃' in issue:
                    print(f"      ✓ prebuilt-layout 모델 사용 권장")
                    print(f"      ✓ 페이지 전처리 (회전, 기울기 보정)")
                elif '한국어 비율' in issue:
                    print(f"      ✓ 한국어 문서인지 재확인")
                    print(f"      ✓ OCR 언어 설정 점검")
                elif '특수 문자' in issue:
                    print(f"      ✓ 텍스트 후처리로 노이즈 제거")
                    print(f"      ✓ 이미지 전처리로 노이즈 감소")
                elif '반복 문자' in issue:
                    print(f"      ✓ OCR 후처리로 반복 패턴 제거")
                elif '긴 단어' in issue:
                    print(f"      ✓ 단어 분할 후처리 적용")
                    print(f"      ✓ 형태소 분석으로 검증")
        
        # 전체적인 권장사항
        print("\\n🎯 전체적인 권장사항:")
        issue_summary = quality_issues['issue_summary']
        
        if issue_summary['low_confidence'] > 0:
            print(f"   📈 {issue_summary['low_confidence']}개 페이지의 낮은 신뢰도 해결이 우선")
        
        if issue_summary['empty_pages'] > 0:
            print(f"   📄 {issue_summary['empty_pages']}개 빈 페이지 원인 분석 필요")
        
        total_issues = sum(issue_summary.values())
        total_pages = quality_issues['total_pages']
        
        if total_issues / total_pages > 0.5:
            print(f"   ⚠️ 전체적인 문서 품질 개선 필요 (문제율: {total_issues/total_pages:.1%})")
            print(f"   🔧 다른 Azure 모델 또는 전처리 방법 고려")
        
    except Exception as e:
        print(f"❌ 개선 방안 제안 실패: {e}")

print("📦 페이지별 품질 검증 및 문제점 분석 함수 로드 완료")
print("   ✅ analyze_page_quality_issues_azure: 페이지별 품질 문제 분석")
print("   ✅ check_single_page_quality_azure: 단일 페이지 품질 검사")
print("   ✅ suggest_page_improvements_azure: 개선 방안 제안")
print()
print("💡 사용법:")
print("   # 페이지별 품질 문제 분석")
print("   quality_issues = analyze_page_quality_issues_azure(extraction_result)")
print("   # 개선 방안 제안")
print("   suggest_page_improvements_azure(quality_issues)")

📦 페이지별 품질 검증 및 문제점 분석 함수 로드 완료
   ✅ analyze_page_quality_issues_azure: 페이지별 품질 문제 분석
   ✅ check_single_page_quality_azure: 단일 페이지 품질 검사
   ✅ suggest_page_improvements_azure: 개선 방안 제안

💡 사용법:
   # 페이지별 품질 문제 분석
   quality_issues = analyze_page_quality_issues_azure(extraction_result)
   # 개선 방안 제안
   suggest_page_improvements_azure(quality_issues)


In [14]:
# 실제 테스트 실행 예제

def run_comprehensive_page_analysis_example():
    """포괄적인 페이지 분석 예제 실행"""
    try:
        print("🚀 Azure Document Intelligence 포괄적 페이지 분석 예제")
        print("=" * 70)
        
        # 테스트할 문서 파일 선택
        available_docs = [doc for doc in TEST_DOCUMENTS if os.path.exists(doc)]
        
        if not available_docs:
            print("❌ 테스트할 문서 파일이 없습니다.")
            print("   다음 중 하나를 준비해주세요:")
            for doc in TEST_DOCUMENTS:
                print(f"   • {doc}")
            return
        
        test_file = available_docs[0]
        print(f"📄 테스트 문서: {os.path.basename(test_file)}")
        print()
        
        # 1. 전체 테스트 실행
        print("1️⃣ 전체 문서 처리 테스트")
        test_result = run_azure_document_test(test_file)
        
        if not test_result['success']:
            print(f"❌ 테스트 실패: {test_result['error']}")
            return
        
        extraction_result = test_result['extraction_result']
        print()
        
        # 2. 첫 번째 페이지 상세 보기
        print("2️⃣ 첫 번째 페이지 상세 내용")
        display_page_content_azure(extraction_result, page_number=1, max_chars=500)
        print()
        
        # 3. 페이지별 품질 문제 분석
        print("3️⃣ 페이지별 품질 문제 분석")
        quality_issues = analyze_page_quality_issues_azure(extraction_result)
        print()
        
        # 4. 개선 방안 제안
        print("4️⃣ 개선 방안 제안")
        suggest_page_improvements_azure(quality_issues)
        print()
        
        # 5. 페이지 비교 (2페이지 이상인 경우)
        page_count = len(extraction_result.get('page_results', []))
        if page_count >= 2:
            print("5️⃣ 페이지 비교 분석")
            compare_pages_azure(extraction_result, 1, 2)
            print()
        
        # 6. 결과 내보내기
        print("6️⃣ 분석 결과 내보내기")
        export_result = export_page_analysis_azure(extraction_result)
        
        if export_result['success']:
            print(f"   ✅ {export_result['files_created']}개 파일이 {export_result['output_directory']}에 저장됨")
        
        print()
        print("🎉 포괄적 페이지 분석 완료!")
        
        return test_result
        
    except Exception as e:
        print(f"❌ 예제 실행 실패: {e}")
        return None

def quick_page_test_example(file_path, page_number=1):
    """특정 페이지 빠른 테스트 예제"""
    try:
        print(f"⚡ 빠른 페이지 테스트: {os.path.basename(file_path)}")
        print(f"🎯 대상 페이지: {page_number}")
        print("-" * 50)
        
        if not os.path.exists(file_path):
            print(f"❌ 파일을 찾을 수 없습니다: {file_path}")
            return
        
        # Azure로 텍스트 추출
        print("📤 Azure Document Intelligence 추출 중...")
        extraction_result = extract_text_with_azure(file_path)
        
        if not extraction_result['success']:
            print(f"❌ 추출 실패: {extraction_result['error']}")
            return
        
        page_results = extraction_result.get('page_results', [])
        
        if page_number > len(page_results):
            print(f"❌ 페이지 {page_number}가 존재하지 않습니다. (총 {len(page_results)}페이지)")
            return
        
        # 특정 페이지 분석
        print(f"\\n🔍 페이지 {page_number} 상세 분석:")
        page_detail = analyze_single_page_azure(page_results[page_number - 1], page_number)
        
        # 결과 출력
        print(f"   📝 텍스트 길이: {page_detail['text_length']:,}자")
        print(f"   🏗️ 레이아웃: {page_detail['layout_type']}")
        print(f"   🎯 신뢰도: {page_detail['confidence']:.3f}")
        print(f"   🔤 단어 수: {page_detail.get('word_count', 0):,}개")
        print(f"   🇰🇷 한국어 비율: {page_detail.get('korean_ratio', 0):.1%}")
        
        # 텍스트 내용 표시
        print(f"\\n📄 페이지 {page_number} 내용:")
        display_page_content_azure(extraction_result, page_number, max_chars=300)
        
        # 품질 문제 확인
        page_quality = check_single_page_quality_azure(page_results[page_number - 1], page_number)
        
        if page_quality['has_issues']:
            print(f"\\n⚠️ 품질 문제 발견:")
            for issue in page_quality['issues']:
                print(f"   • {issue}")
        else:
            print(f"\\n✅ 페이지 품질 양호")
        
        return extraction_result
        
    except Exception as e:
        print(f"❌ 빠른 테스트 실패: {e}")
        return None

# 사용 예제 실행 준비
print("📦 실제 테스트 실행 예제 함수 로드 완료")
print("   ✅ run_comprehensive_page_analysis_example: 포괄적 분석 예제")
print("   ✅ quick_page_test_example: 빠른 페이지 테스트")
print()
print("🎯 실행 방법:")
print("   # 포괄적 분석 실행")
print("   result = run_comprehensive_page_analysis_example()")
print()
print("   # 특정 파일의 특정 페이지 빠른 테스트")
print("   quick_page_test_example('/path/to/document.pdf', page_number=1)")
print()
print("   # 사용 가능한 테스트 문서 확인")
print("   for doc in TEST_DOCUMENTS:")
print("       if os.path.exists(doc):")
print("           print(f'✅ {doc}')")
print("       else:")
print("           print(f'❌ {doc}')")

📦 실제 테스트 실행 예제 함수 로드 완료
   ✅ run_comprehensive_page_analysis_example: 포괄적 분석 예제
   ✅ quick_page_test_example: 빠른 페이지 테스트

🎯 실행 방법:
   # 포괄적 분석 실행
   result = run_comprehensive_page_analysis_example()

   # 특정 파일의 특정 페이지 빠른 테스트
   quick_page_test_example('/path/to/document.pdf', page_number=1)

   # 사용 가능한 테스트 문서 확인
   for doc in TEST_DOCUMENTS:
       if os.path.exists(doc):
           print(f'✅ {doc}')
       else:
           print(f'❌ {doc}')


## 🔥 핵심 기능 테스트: 실제 문서 처리 확인

이제 실제 PDF 문서를 사용하여 Azure Document Intelligence의 핵심 기능들을 테스트합니다:

1. **전체 텍스트 추출**: 문서에서 모든 텍스트를 정확히 추출하는지 확인
2. **구조적 요소 인식**: 페이지별 그림, 테이블, 레이아웃 요소 인식 확인  
3. **페이지별 상세 분석**: 각 페이지의 처리 결과를 개별적으로 검토

**테스트 진행 순서:**
- 📄 테스트 문서 선택 및 로드
- 🔍 전체 텍스트 추출 및 확인
- 📊 구조적 요소(테이블, 이미지) 인식 테스트
- 📃 페이지별 상세 분석 및 결과 검토

In [15]:
# 📄 1단계: 테스트 문서 선택 및 기본 정보 확인

import os
import json
from pathlib import Path
from datetime import datetime

# 입력/출력 디렉토리 설정
INPUT_DOCS_DIR = "/home/admin/wkms-aws/jupyter_notebook/data/input_docs"
OUTPUT_TEXTS_DIR = "/home/admin/wkms-aws/jupyter_notebook/data/output_texts"

# 디렉토리 생성 (존재하지 않는 경우)
os.makedirs(INPUT_DOCS_DIR, exist_ok=True)
os.makedirs(OUTPUT_TEXTS_DIR, exist_ok=True)

def select_test_document():
    """지정된 입력 디렉토리에서 테스트 문서 선택"""
    print("📁 테스트 문서 검색 중...")
    print(f"   📂 입력 디렉토리: {INPUT_DOCS_DIR}")
    
    # 기본 테스트 문서 경로
    default_test_file = os.path.join(INPUT_DOCS_DIR, "test.pdf")
    
    available_docs = []
    
    # 입력 디렉토리에서 PDF 파일 찾기
    if os.path.exists(INPUT_DOCS_DIR):
        try:
            for file in os.listdir(INPUT_DOCS_DIR):
                if file.endswith('.pdf'):
                    full_path = os.path.join(INPUT_DOCS_DIR, file)
                    size = os.path.getsize(full_path)
                    available_docs.append({
                        'path': full_path,
                        'name': file,
                        'size_mb': round(size / (1024*1024), 2)
                    })
        except Exception as e:
            print(f"⚠️ 디렉토리 읽기 오류: {e}")
    
    if not available_docs:
        print("❌ 입력 디렉토리에서 PDF 문서를 찾을 수 없습니다!")
        print(f"💡 다음 위치에 PDF 파일을 배치해주세요: {INPUT_DOCS_DIR}")
        print(f"📋 기본 테스트 파일: test.pdf")
        return None
    
    print(f"✅ {len(available_docs)}개의 테스트 문서를 발견했습니다:")
    for i, doc in enumerate(available_docs, 1):
        print(f"   {i}. {doc['name']} ({doc['size_mb']}MB)")
    
    # test.pdf가 있으면 우선 선택, 없으면 첫 번째 문서 선택
    selected_doc = None
    for doc in available_docs:
        if doc['name'] == 'test.pdf':
            selected_doc = doc
            print(f"\n🎯 기본 테스트 문서 선택: {doc['name']}")
            break
    
    if not selected_doc:
        selected_doc = available_docs[0]
        print(f"\n🎯 첫 번째 문서 선택: {selected_doc['name']}")
    
    print(f"   📍 경로: {selected_doc['path']}")
    print(f"   📏 크기: {selected_doc['size_mb']}MB")
    
    return selected_doc['path']

def get_document_basic_info(file_path):
    """문서 기본 정보 확인"""
    try:
        import PyPDF2
        
        with open(file_path, 'rb') as file:
            pdf_reader = PyPDF2.PdfReader(file)
            page_count = len(pdf_reader.pages)
            
            # 메타데이터 확인
            metadata = pdf_reader.metadata
            
            print(f"📄 문서 기본 정보:")
            print(f"   📃 총 페이지: {page_count}페이지")
            
            if metadata:
                if metadata.get('/Title'):
                    print(f"   📝 제목: {metadata.get('/Title')}")
                if metadata.get('/Author'):
                    print(f"   👤 작성자: {metadata.get('/Author')}")
                if metadata.get('/Creator'):
                    print(f"   🛠️ 생성도구: {metadata.get('/Creator')}")
            
            return {
                'page_count': page_count,
                'metadata': metadata,
                'file_size': os.path.getsize(file_path)
            }
    except Exception as e:
        print(f"⚠️ PDF 정보 읽기 실패: {e}")
        print("   (Azure Document Intelligence로 계속 진행)")
        return {
            'page_count': '알 수 없음',
            'metadata': None,
            'file_size': os.path.getsize(file_path)
        }

def create_output_filename(input_path, suffix=""):
    """출력 파일명 생성"""
    input_name = os.path.splitext(os.path.basename(input_path))[0]
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    if suffix:
        return f"{input_name}_{suffix}_{timestamp}"
    else:
        return f"{input_name}_{timestamp}"

# 테스트 문서 선택
test_document_path = select_test_document()

if test_document_path:
    doc_info = get_document_basic_info(test_document_path)
    
    # 출력 파일 기본 이름 설정
    output_base_name = create_output_filename(test_document_path)
    
    print(f"\n🚀 Azure Document Intelligence 테스트 준비 완료!")
    print(f"   📄 테스트 문서: {os.path.basename(test_document_path)}")
    print(f"   📊 예상 페이지: {doc_info['page_count']}")
    print(f"   💾 파일 크기: {round(doc_info['file_size']/(1024*1024), 2)}MB")
    print(f"   📂 출력 디렉토리: {OUTPUT_TEXTS_DIR}")
    print(f"   📝 출력 파일명 기본: {output_base_name}")
    
    # 전역 변수로 설정
    globals()['test_document_path'] = test_document_path
    globals()['output_base_name'] = output_base_name
    
else:
    print("❌ 테스트를 진행할 수 없습니다. 문서를 준비해주세요.")

📁 테스트 문서 검색 중...
   📂 입력 디렉토리: /home/admin/wkms-aws/jupyter_notebook/data/input_docs
✅ 3개의 테스트 문서를 발견했습니다:
   1. 20_ProductSpec_SmartInsulinPump_KO_v0.1.pdf (0.32MB)
   2. test1.pdf (1.26MB)
   3. test.pdf (11.85MB)

🎯 기본 테스트 문서 선택: test.pdf
   📍 경로: /home/admin/wkms-aws/jupyter_notebook/data/input_docs/test.pdf
   📏 크기: 11.85MB
📄 문서 기본 정보:
   📃 총 페이지: 27페이지
   📝 제목: 1
   👤 작성자: 3PC
   🛠️ 생성도구: Adobe Acrobat Pro 9.0.0

🚀 Azure Document Intelligence 테스트 준비 완료!
   📄 테스트 문서: test.pdf
   📊 예상 페이지: 27
   💾 파일 크기: 11.85MB
   📂 출력 디렉토리: /home/admin/wkms-aws/jupyter_notebook/data/output_texts
   📝 출력 파일명 기본: test_20250923_125927
📄 문서 기본 정보:
   📃 총 페이지: 27페이지
   📝 제목: 1
   👤 작성자: 3PC
   🛠️ 생성도구: Adobe Acrobat Pro 9.0.0

🚀 Azure Document Intelligence 테스트 준비 완료!
   📄 테스트 문서: test.pdf
   📊 예상 페이지: 27
   💾 파일 크기: 11.85MB
   📂 출력 디렉토리: /home/admin/wkms-aws/jupyter_notebook/data/output_texts
   📝 출력 파일명 기본: test_20250923_125927


In [16]:
# 🔍 2단계: 전체 텍스트 추출 및 확인

import os
import time
import json
from datetime import datetime

def save_extracted_text(text_content, statistics, output_base_name):
    """추출된 텍스트를 파일로 저장하고 전체 경로를 로그로 출력"""
    try:
        # 전체 텍스트 저장
        text_filename = f"{output_base_name}_full_text.txt"
        text_filepath = os.path.join(OUTPUT_TEXTS_DIR, text_filename)
        
        with open(text_filepath, 'w', encoding='utf-8') as f:
            f.write("=" * 80 + "\n")
            f.write("Azure Document Intelligence 텍스트 추출 결과\n")
            f.write("=" * 80 + "\n")
            f.write(f"추출 시각: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
            f.write(f"총 페이지: {statistics['page_count']}페이지\n")
            f.write(f"총 글자수: {statistics['char_count']:,}자\n")
            f.write(f"총 줄수: {statistics['line_count']:,}줄\n")
            f.write(f"총 단어수: {statistics['word_count']:,}개\n")
            f.write("=" * 80 + "\n\n")
            f.write(text_content)
        
        # 통계 정보 JSON으로 저장
        stats_filename = f"{output_base_name}_statistics.json"
        stats_filepath = os.path.join(OUTPUT_TEXTS_DIR, stats_filename)
        
        stats_data = {
            "extraction_time": datetime.now().isoformat(),
            "statistics": statistics,
            "output_files": {
                "full_text": text_filename,
                "statistics": stats_filename
            }
        }
        
        with open(stats_filepath, 'w', encoding='utf-8') as f:
            json.dump(stats_data, f, ensure_ascii=False, indent=2)
        
        print("💾 텍스트 저장 완료:")
        print(f"   📄 전체 텍스트 파일: {text_filepath}")
        print(f"   📊 통계 정보 파일: {stats_filepath}")
        
        return {
            'text_file': text_filepath,
            'stats_file': stats_filepath
        }
        
    except Exception as e:
        print(f"❌ 파일 저장 실패: {e}")
        return None

def extract_full_text_test(file_path, pages: str | None = None, fast_mode: bool = False):
    """전체 텍스트 추출 테스트
    pages: Azure 'pages' 파라미터. 예) '1-3', '1,3,5' (설정 시 해당 페이지만 분석)
    fast_mode: True이면 미리보기만 빠르게 확인하고 저장을 생략
    """
    if not azure_client:
        print("❌ Azure 클라이언트가 초기화되지 않았습니다.")
        return None
    
    print("🚀 전체 텍스트 추출 시작...")
    print(f"📄 파일: {os.path.basename(file_path)}")
    if pages:
        print(f"🔧 페이지 제한 적용: pages='{pages}' (해당 페이지만 분석)")
    if fast_mode:
        print("⚡ Fast Mode: 텍스트 저장 없이 미리보기/통계만 수행")
    
    start_time = time.time()
    try:
        # Azure Document Intelligence로 분석
        with open(file_path, "rb") as f:
            if pages:
                poller = azure_client.begin_analyze_document(
                    model_id="prebuilt-read",  # 텍스트 추출에 최적화된 모델
                    document=f,
                    pages=pages
                )
            else:
                poller = azure_client.begin_analyze_document(
                    model_id="prebuilt-read",  # 텍스트 추출에 최적화된 모델
                    document=f
                )
        
        print("⏳ Azure Document Intelligence 분석 중...")
        result = poller.result()
        elapsed = time.time() - start_time
        print(f"⏱️ 분석 완료 (경과 시간: {elapsed:.1f}s)")
        
        # 전체 텍스트 추출
        full_text = result.content or ""
        
        # 기본 통계
        char_count = len(full_text)
        line_count = len(full_text.split('\n'))
        word_count = len(full_text.split())
        page_count = len(result.pages) if getattr(result, 'pages', None) else 0
        
        print(f"\n✅ 텍스트 추출 완료!")
        print(f"   📃 총 페이지: {page_count}페이지")
        print(f"   📝 총 글자수: {char_count:,}자")
        print(f"   📄 총 줄수: {line_count:,}줄")
        print(f"   🔤 총 단어수: {word_count:,}개")
        
        # 텍스트 미리보기 (첫 500자)
        preview_length = min(500, len(full_text))
        preview_text = full_text[:preview_length]
        
        print(f"\n📖 텍스트 미리보기 (첫 {preview_length}자):")
        print("=" * 80)
        print(preview_text)
        if len(full_text) > preview_length:
            print("\n... (나머지 텍스트 생략) ...")
        print("=" * 80)
        
        statistics = {
            'page_count': page_count,
            'char_count': char_count,
            'line_count': line_count,
            'word_count': word_count
        }
        
        # Fast Mode면 저장 스킵
        if fast_mode:
            return {
                'full_text': full_text,
                'statistics': statistics,
                'azure_result': result
            }
        
        # 텍스트 파일로 저장
        if 'output_base_name' in globals():
            save_result = save_extracted_text(full_text, statistics, output_base_name)
            if save_result:
                print(f"\n📂 텍스트/통계 저장 위치: {OUTPUT_TEXTS_DIR}")
                globals()['text_save_result'] = save_result
        
        return {
            'full_text': full_text,
            'statistics': statistics,
            'azure_result': result
        }
        
    except Exception as e:
        print(f"❌ 텍스트 추출 실패: {e}")
        return None

# 환경 변수로 빠른 실행 설정 지원
READ_PAGES_RANGE = os.getenv('AZURE_READ_PAGES_RANGE', '').strip() or None  # 예: '1-3', '1,5'
FAST_MODE = os.getenv('AZURE_FAST_MODE', 'false').strip().lower() in ('1', 'true', 'yes', 'y')

# 텍스트 추출 실행
if 'test_document_path' in globals() and test_document_path:
    text_result = extract_full_text_test(test_document_path, pages=READ_PAGES_RANGE, fast_mode=FAST_MODE)
    
    if text_result:
        print(f"\n🎯 텍스트 추출 결과 요약:")
        stats = text_result['statistics']
        print(f"   ✅ 성공적으로 {stats['page_count']}페이지에서 {stats['char_count']:,}자 추출")
        
        # 전역 변수로 저장
        globals()['extracted_text'] = text_result['full_text']
        globals()['azure_analysis_result'] = text_result['azure_result']
        globals()['text_statistics'] = text_result['statistics']
    else:
        print("❌ 텍스트 추출에 실패했습니다.")
else:
    print("❌ 테스트 문서가 준비되지 않았습니다. 이전 셀을 먼저 실행해주세요.")

🚀 전체 텍스트 추출 시작...
📄 파일: test.pdf

📄 파일: test.pdf
⏳ Azure Document Intelligence 분석 중...
⏳ Azure Document Intelligence 분석 중...
⏱️ 분석 완료 (경과 시간: 10.1s)

✅ 텍스트 추출 완료!
   📃 총 페이지: 27페이지
   📝 총 글자수: 80,072자
   📄 총 줄수: 680줄
   🔤 총 단어수: 12,160개

📖 텍스트 미리보기 (첫 500자):
韓國IT서비스學會誌 第22卷 第3號 2023年 6月, pp.1-27
Journal of Information Technology Services https://doi.org/10.9716/KITS.2023.22.3.001
양손잡이 리더십과 혁신적인 업무 행동: 한국 반도체 산업의 증거
돔페 헨리 아메요* · 오포리 헨리 *** · 윤소라 **** . 강주영 *****
Ambidextrous Leadership and Innovative Work Behavior: Evidence from South Korea Semiconductor Industry*
Henry Ameyaw Domfeh ** · Henry Ofori *** · Sora Yoon *** **** · Juyoung Kang* ******
Abstract
The semiconductor industry is a competitive, complicated and a cyclical sector with a highly dy

... (나머지 텍스트 생략) ...
💾 텍스트 저장 완료:
   📄 전체 텍스트 파일: /home/admin/wkms-aws/jupyter_notebook/data/output_texts/test_20250923_125927_full_text.txt
   📊 통계 정보 파일: /home/admin/wkms-aws/jupyter_notebook/data/output_texts/test_20250923_125927_statis

In [17]:
# 📊 3단계: 구조적 요소 인식 테스트 (테이블, 이미지, 레이아웃)

import os
import json
import csv
from datetime import datetime

def save_structure_analysis(structure_info, page_details, tables_data, output_base_name, raw_tables=None):
    """구조 분석 결과를 파일로 저장하고 전체 경로를 로그로 출력
    raw_tables: Azure 분석 원본의 table 객체 리스트 (있으면 테이블 별 CSV로 내보냄)
    """
    try:
        # 구조 분석 보고서 저장
        report_filename = f"{output_base_name}_structure_analysis.txt"
        report_filepath = os.path.join(OUTPUT_TEXTS_DIR, report_filename)
        
        with open(report_filepath, 'w', encoding='utf-8') as f:
            f.write("=" * 80 + "\n")
            f.write("Azure Document Intelligence 구조 분석 결과\n")
            f.write("=" * 80 + "\n")
            f.write(f"분석 시각: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
            
            f.write("📊 전체 구조 요약:\n")
            f.write(f"   📄 총 페이지: {structure_info['pages']}개\n")
            f.write(f"   📝 총 문단: {structure_info['paragraphs']}개\n")
            f.write(f"   📏 총 텍스트 라인: {structure_info['lines']}개\n")
            f.write(f"   📋 테이블: {structure_info['tables']}개\n")
            f.write(f"   ☑️ 체크박스/선택 마크: {structure_info['selection_marks']}개\n\n")
            
            # 페이지별 상세 정보
            f.write("📃 페이지별 상세 정보:\n")
            for page in page_details:
                f.write(f"   페이지 {page['page_number']:2d}: "
                       f"텍스트 라인 {page['lines']:3d}개, "
                       f"단어 {page['words']:4d}개, "
                       f"테이블 {page['tables_on_page']}개\n")
            
            # 테이블 상세 정보
            if tables_data:
                f.write(f"\n📋 테이블 상세 정보:\n")
                for i, table_info in enumerate(tables_data, 1):
                    f.write(f"   테이블 {i}: {table_info['rows']}행 × {table_info['cols']}열 "
                           f"({table_info['cells']}개 셀)\n")
                    f.write(f"      📍 위치: {table_info['page']}페이지\n")
        
        # JSON 형태로도 저장
        json_filename = f"{output_base_name}_structure_data.json"
        json_filepath = os.path.join(OUTPUT_TEXTS_DIR, json_filename)
        
        structure_data = {
            "analysis_time": datetime.now().isoformat(),
            "structure_summary": structure_info,
            "page_details": page_details,
            "tables": tables_data,
            "output_files": {
                "structure_report": report_filename,
                "structure_data": json_filename
            }
        }
        
        with open(json_filepath, 'w', encoding='utf-8') as f:
            json.dump(structure_data, f, ensure_ascii=False, indent=2)
        
        # CSV로 테이블 요약 저장
        csv_summary_filepath = None
        try:
            csv_summary_filename = f"{output_base_name}_tables_summary.csv"
            csv_summary_filepath = os.path.join(OUTPUT_TEXTS_DIR, csv_summary_filename)
            with open(csv_summary_filepath, 'w', newline='', encoding='utf-8') as csvfile:
                writer = csv.writer(csvfile)
                writer.writerow(["table_id", "page", "rows", "cols", "cells"])
                for t in tables_data:
                    writer.writerow([t.get('table_id'), t.get('page'), t.get('rows'), t.get('cols'), t.get('cells')])
        except Exception:
            csv_summary_filepath = None
        
        # 테이블 별 CSV 내보내기 (내용 포함)
        per_table_csv_files = []
        if raw_tables:
            try:
                for idx, table in enumerate(raw_tables, 1):
                    rows = getattr(table, 'row_count', None)
                    cols = getattr(table, 'column_count', None)
                    if rows is None or cols is None or rows <= 0 or cols <= 0:
                        continue
                    # 페이지 번호
                    page_num = 'Unknown'
                    if hasattr(table, 'bounding_regions') and table.bounding_regions:
                        page_num = table.bounding_regions[0].page_number
                    # 파일명
                    if page_num == 'Unknown':
                        tbl_filename = f"{output_base_name}_table_{idx}.csv"
                    else:
                        tbl_filename = f"{output_base_name}_table_{idx}_p{page_num}.csv"
                    tbl_filepath = os.path.join(OUTPUT_TEXTS_DIR, tbl_filename)
                    # 그리드 초기화
                    grid = [["" for _ in range(cols)] for _ in range(rows)]
                    for cell in (table.cells or []):
                        r = getattr(cell, 'row_index', None)
                        c = getattr(cell, 'column_index', None)
                        content = getattr(cell, 'content', '') or ''
                        content = content.replace('\n', ' ').strip()
                        if r is not None and c is not None and 0 <= r < rows and 0 <= c < cols:
                            if grid[r][c]:
                                grid[r][c] = (grid[r][c] + ' ' + content).strip()
                            else:
                                grid[r][c] = content
                    # CSV 저장
                    with open(tbl_filepath, 'w', newline='', encoding='utf-8') as tbl_csv:
                        writer = csv.writer(tbl_csv)
                        writer.writerows(grid)
                    per_table_csv_files.append(tbl_filepath)
            except Exception as e:
                print(f"⚠️ 테이블 별 CSV 저장 중 일부 실패: {e}")
        
        print("💾 구조 분석 저장 완료:")
        print(f"   📄 구조 보고서 파일: {report_filepath}")
        print(f"   📊 구조 데이터 파일: {json_filepath}")
        if csv_summary_filepath:
            print(f"   📈 테이블 요약 CSV: {csv_summary_filepath}")
        if per_table_csv_files:
            print(f"   📁 테이블 별 CSV ({len(per_table_csv_files)}개):")
            for p in per_table_csv_files:
                print(f"      - {p}")
        
        return {
            'report_file': report_filepath,
            'data_file': json_filepath,
            'tables_summary_csv': csv_summary_filepath,
            'table_csv_files': per_table_csv_files
        }
        
    except Exception as e:
        print(f"❌ 구조 분석 저장 실패: {e}")
        return None

def analyze_document_structure():
    """문서의 구조적 요소 분석"""
    if not azure_client:
        print("❌ Azure 클라이언트가 초기화되지 않았습니다.")
        return None
    
    if 'test_document_path' not in globals():
        print("❌ 테스트 문서가 설정되지 않았습니다.")
        return None
    
    file_path = test_document_path
    print("🔍 문서 구조 분석 시작...")
    print("📊 모델: prebuilt-layout (레이아웃 + 테이블 인식)")
    
    try:
        # 레이아웃 분석 모델 사용
        with open(file_path, "rb") as f:
            poller = azure_client.begin_analyze_document(
                model_id="prebuilt-layout",  # 구조 분석에 최적화된 모델
                document=f
            )
        
        print("⏳ 구조 분석 중...")
        result = poller.result()
        
        # 구조적 요소 분석
        structure_info = {
            'pages': len(result.pages),
            'tables': len(result.tables) if result.tables else 0,
            'paragraphs': len(result.paragraphs) if result.paragraphs else 0,
            'lines': 0,
            'selection_marks': 0,
            'figures': 0
        }
        
        # 페이지별 상세 분석
        page_details = []
        
        for i, page in enumerate(result.pages, 1):
            page_info = {
                'page_number': i,
                'lines': len(page.lines) if page.lines else 0,
                'words': len(page.words) if page.words else 0,
                'selection_marks': len(page.selection_marks) if page.selection_marks else 0,
                'tables_on_page': 0,
                'has_figures': False
            }
            
            # 이 페이지의 테이블 개수 계산
            if result.tables:
                for table in result.tables:
                    table_page = getattr(table, 'bounding_regions', [])
                    if table_page and table_page[0].page_number == i:
                        page_info['tables_on_page'] += 1
            
            structure_info['lines'] += page_info['lines']
            structure_info['selection_marks'] += page_info['selection_marks']
            
            page_details.append(page_info)
        
        print(f"\n✅ 구조 분석 완료!")
        print(f"📊 전체 구조 요약:")
        print(f"   📄 총 페이지: {structure_info['pages']}개")
        print(f"   📝 총 문단: {structure_info['paragraphs']}개")
        print(f"   📏 총 텍스트 라인: {structure_info['lines']}개")
        print(f"   📋 테이블: {structure_info['tables']}개")
        print(f"   ☑️ 체크박스/선택 마크: {structure_info['selection_marks']}개")
        
        # 테이블 상세 정보 수집
        tables_data = []
        if result.tables and len(result.tables) > 0:
            print(f"\n📋 테이블 상세 정보:")
            for i, table in enumerate(result.tables, 1):
                row_count = table.row_count
                col_count = table.column_count
                cell_count = len(table.cells) if table.cells else 0
                
                table_info = {
                    'table_id': i,
                    'rows': row_count,
                    'cols': col_count,
                    'cells': cell_count,
                    'page': 'Unknown'
                }
                
                # 테이블 위치 정보
                if hasattr(table, 'bounding_regions') and table.bounding_regions:
                    page_num = table.bounding_regions[0].page_number
                    table_info['page'] = page_num
                    print(f"   테이블 {i}: {row_count}행 × {col_count}열 ({cell_count}개 셀)")
                    print(f"      📍 위치: {page_num}페이지")
                
                tables_data.append(table_info)
        
        # 페이지별 상세 정보
        print(f"\n📃 페이지별 상세 정보:")
        for page in page_details:
            print(f"   페이지 {page['page_number']:2d}: "
                  f"텍스트 라인 {page['lines']:3d}개, "
                  f"단어 {page['words']:4d}개, "
                  f"테이블 {page['tables_on_page']}개")
        
        # 구조 분석 결과 저장 (테이블 원본도 전달하여 per-table CSV 내보냄)
        if 'output_base_name' in globals():
            _save = save_structure_analysis(structure_info, page_details, tables_data, output_base_name, raw_tables=result.tables)
            if _save:
                # 전역으로 노출
                globals()['structure_save_result'] = _save
                print(f"\n📂 구조/테이블 저장 위치: {OUTPUT_TEXTS_DIR}")
        
        # 전역 변수로 저장
        globals()['structure_analysis_result'] = {
            'structure_info': structure_info,
            'page_details': page_details,
            'azure_result': result
        }
        globals()['document_tables'] = result.tables
        
        return True
        
    except Exception as e:
        print(f"❌ 구조 분석 실패: {e}")
        return False

# 구조 분석 실행
if 'test_document_path' in globals() and test_document_path:
    structure_success = analyze_document_structure()
    
    if structure_success:
        print(f"\n🎯 구조 분석 완료!")
        print(f"   ✅ 테이블, 문단, 레이아웃 요소 분석 성공")
    else:
        print("❌ 구조 분석에 실패했습니다.")
else:
    print("❌ 테스트 문서가 준비되지 않았습니다. 이전 셀을 먼저 실행해주세요.")

🔍 문서 구조 분석 시작...
📊 모델: prebuilt-layout (레이아웃 + 테이블 인식)
⏳ 구조 분석 중...
⏳ 구조 분석 중...

✅ 구조 분석 완료!
📊 전체 구조 요약:
   📄 총 페이지: 27개
   📝 총 문단: 727개
   📏 총 텍스트 라인: 2154개
   📋 테이블: 11개
   ☑️ 체크박스/선택 마크: 97개

📋 테이블 상세 정보:
   테이블 1: 14행 × 3열 (36개 셀)
      📍 위치: 11페이지
   테이블 2: 23행 × 3열 (61개 셀)
      📍 위치: 12페이지
   테이블 3: 8행 × 3열 (24개 셀)
      📍 위치: 12페이지
   테이블 4: 12행 × 12열 (144개 셀)
      📍 위치: 13페이지
   테이블 5: 6행 × 3열 (18개 셀)
      📍 위치: 13페이지
   테이블 6: 8행 × 8열 (64개 셀)
      📍 위치: 13페이지
   테이블 7: 4행 × 8열 (22개 셀)
      📍 위치: 14페이지
   테이블 8: 7행 × 8열 (44개 셀)
      📍 위치: 14페이지
   테이블 9: 6행 × 3열 (18개 셀)
      📍 위치: 15페이지
   테이블 10: 3행 × 5열 (15개 셀)
      📍 위치: 16페이지
   테이블 11: 7행 × 4열 (25개 셀)
      📍 위치: 17페이지

📃 페이지별 상세 정보:
   페이지  1: 텍스트 라인  44개, 단어  486개, 테이블 0개
   페이지  2: 텍스트 라인  77개, 단어  533개, 테이블 0개
   페이지  3: 텍스트 라인  75개, 단어  539개, 테이블 0개
   페이지  4: 텍스트 라인  77개, 단어  521개, 테이블 0개
   페이지  5: 텍스트 라인  70개, 단어  346개, 테이블 0개
   페이지  6: 텍스트 라인  77개, 단어  517개, 테이블 0개
   페이지  7: 텍스트 라인  76개, 단어  520개, 테이블 0

In [18]:
# 📃 4단계: 페이지별 상세 분석 및 결과 검토

import re

def save_page_analysis(page_analysis_data, table_contents, summary_data, output_base_name):
    """페이지별 분석 결과를 파일로 저장"""
    try:
        # 페이지별 상세 분석 보고서 저장
        report_filename = f"{output_base_name}_page_analysis.txt"
        report_filepath = os.path.join(OUTPUT_TEXTS_DIR, report_filename)
        
        with open(report_filepath, 'w', encoding='utf-8') as f:
            f.write("=" * 80 + "\n")
            f.write("Azure Document Intelligence 페이지별 상세 분석 결과\n")
            f.write("=" * 80 + "\n")
            f.write(f"분석 시각: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
            
            # 페이지별 분석 결과
            f.write("📃 페이지별 상세 분석 결과\n")
            f.write("=" * 80 + "\n")
            
            for page_data in page_analysis_data:
                f.write(f"\n📄 페이지 {page_data['page_number']}\n")
                f.write("-" * 40 + "\n")
                f.write(f"   📏 크기: {page_data['width']} × {page_data['height']}\n")
                f.write(f"   📝 텍스트 라인: {page_data['line_count']}개\n")
                f.write(f"   🔤 단어: {page_data['word_count']}개\n")
                f.write(f"   🎯 평균 신뢰도: {page_data['avg_confidence']:.3f}\n")
                f.write(f"   📉 최소 신뢰도: {page_data['min_confidence']:.3f}\n")
                f.write(f"   🖼️ 추정 그림 캡션 수: {page_data.get('figure_captions', 0)}\n")
                f.write(f"   📋 표 캡션 수: {page_data.get('table_captions', 0)}\n")
                
                f.write(f"   📖 텍스트 미리보기 (첫 5줄):\n")
                for i, line in enumerate(page_data['preview_lines'], 1):
                    f.write(f"      {i}. {line}\n")
                
                if page_data['remaining_lines'] > 0:
                    f.write(f"      ... (나머지 {page_data['remaining_lines']}줄 생략)\n")
                f.write("\n")
            
            # 테이블 내용
            if table_contents:
                f.write("\n📋 테이블 내용 상세 분석\n")
                f.write("=" * 80 + "\n")
                
                for table_data in table_contents:
                    f.write(f"\n테이블 {table_data['table_id']}: {table_data['rows']}행 × {table_data['cols']}열\n")
                    f.write("-" * 40 + "\n")
                    
                    for row_idx, row_data in enumerate(table_data['content']):
                        f.write(f"   행 {row_idx+1:2d}: " + " | ".join(f"{cell:20s}" for cell in row_data) + "\n")
                    
                    if table_data['rows'] > len(table_data['content']):
                        f.write(f"   ... (나머지 {table_data['rows'] - len(table_data['content'])}행 생략)\n")
            
            # 종합 요약
            f.write(f"\n🎯 Azure Document Intelligence 처리 결과 종합 요약\n")
            f.write("=" * 80 + "\n")
            f.write(f"📊 텍스트 추출 결과:\n")
            f.write(f"   ✅ 총 {summary_data['page_count']}페이지에서 {summary_data['char_count']:,}자 추출 성공\n")
            f.write(f"   📝 단어 수: {summary_data['word_count']:,}개\n\n")
            
            f.write(f"📋 구조 분석 결과:\n")
            f.write(f"   📄 페이지: {summary_data['pages']}개\n")
            f.write(f"   📝 문단: {summary_data['paragraphs']}개\n")
            f.write(f"   📋 테이블: {summary_data['tables']}개\n")
            f.write(f"   📏 텍스트 라인: {summary_data['lines']}개\n\n")
            
            f.write(f"💡 활용 가능성:\n")
            f.write(f"   📄 텍스트 검색: ✅ 가능\n")
            f.write(f"   🤖 AI 질의응답: ✅ 가능\n")
            f.write(f"   📊 데이터 분석: {'✅ 가능' if summary_data['tables'] > 0 else '❌ 테이블 없음'}\n")
            f.write(f"   🔍 의미 검색: ✅ 가능\n")
        
        # JSON 형태로도 저장
        json_filename = f"{output_base_name}_page_analysis_data.json"
        json_filepath = os.path.join(OUTPUT_TEXTS_DIR, json_filename)
        json_data = {
            'analysis_time': datetime.now().isoformat(),
            'page_analysis': page_analysis_data,
            'table_contents': table_contents,
            'summary': summary_data,
            'output_files': {
                'page_report': report_filename,
                'page_data': json_filename
            }
        }
        with open(json_filepath, 'w', encoding='utf-8') as f:
            json.dump(json_data, f, ensure_ascii=False, indent=2)
        
        print("💾 페이지 분석 저장 완료:")
        print(f"   📄 상세 보고서: {report_filepath}")
        print(f"   📊 분석 데이터: {json_filepath}")
        
        return {
            'page_report': report_filepath,
            'page_data': json_filepath
        }
    except Exception as e:
        print(f"❌ 페이지 분석 저장 실패: {e}")
        return None

def analyze_pages_individually():
    """페이지별 개별 분석 (Azure 결과 기반)"""
    if 'azure_analysis_result' not in globals():
        print("❌ Azure 분석 결과가 없습니다. 먼저 텍스트 추출을 실행하세요.")
        return []
    
    result = azure_analysis_result
    page_analysis_data = []
    
    for i, page in enumerate(result.pages, 1):
        # 기본 페이지 정보
        page_info = {
            'page_number': i,
            'width': getattr(page, 'width', 0),
            'height': getattr(page, 'height', 0),
            'line_count': len(page.lines) if page.lines else 0,
            'word_count': len(page.words) if page.words else 0,
            'avg_confidence': 0.0,
            'min_confidence': 1.0,
            'preview_lines': [],
            'remaining_lines': 0,
            'figure_captions': 0,
            'table_captions': 0
        }
        
        # 신뢰도 계산 (단어 기준)
        if page.words:
            confidences = [getattr(word, 'confidence', 1.0) for word in page.words if hasattr(word, 'confidence')]
            if confidences:
                page_info['avg_confidence'] = sum(confidences) / len(confidences)
                page_info['min_confidence'] = min(confidences)
        
        # 텍스트 미리보기 (첫 5줄)
        if page.lines:
            preview_count = min(5, len(page.lines))
            for j in range(preview_count):
                line_content = getattr(page.lines[j], 'content', '').strip()
                page_info['preview_lines'].append(line_content)
            page_info['remaining_lines'] = max(0, len(page.lines) - preview_count)
        
        # 간단한 캡션 휴리스틱 (Figure/Table 패턴 검색)
        page_text = ' '.join([getattr(line, 'content', '') for line in (page.lines or [])])
        figure_patterns = [r'Figure\s+\d+', r'Fig\.\s*\d+', r'그림\s+\d+']
        table_patterns = [r'Table\s+\d+', r'표\s+\d+']
        
        for pattern in figure_patterns:
            page_info['figure_captions'] += len(re.findall(pattern, page_text, re.IGNORECASE))
        
        for pattern in table_patterns:
            page_info['table_captions'] += len(re.findall(pattern, page_text, re.IGNORECASE))
        
        page_analysis_data.append(page_info)
    
    return page_analysis_data

def show_table_contents():
    """테이블 내용 표시 (document_tables 기반)"""
    if 'document_tables' not in globals() or not document_tables:
        print("⚠️ 감지된 테이블이 없습니다.")
        return []
    
    table_contents = []
    for i, table in enumerate(document_tables, 1):
        table_data = {
            'table_id': i,
            'rows': getattr(table, 'row_count', 0),
            'cols': getattr(table, 'column_count', 0),
            'content': []
        }
        
        # 간단한 테이블 내용 추출 (첫 5행만)
        if hasattr(table, 'cells') and table.cells:
            # 그리드 생성
            max_rows = min(5, table_data['rows'])
            max_cols = table_data['cols']
            grid = [["" for _ in range(max_cols)] for _ in range(max_rows)]
            
            for cell in table.cells:
                row_idx = getattr(cell, 'row_index', -1)
                col_idx = getattr(cell, 'column_index', -1)
                content = getattr(cell, 'content', '').strip()
                
                if 0 <= row_idx < max_rows and 0 <= col_idx < max_cols:
                    grid[row_idx][col_idx] = content[:20]  # 20자로 제한
            
            table_data['content'] = grid
        
        table_contents.append(table_data)
    
    return table_contents

def show_processing_summary():
    """처리 결과 종합 요약"""
    summary_data = {
        'page_count': 0,
        'char_count': 0,
        'word_count': 0,
        'pages': 0,
        'paragraphs': 0,
        'tables': 0,
        'lines': 0
    }
    
    # 텍스트 통계
    if 'text_statistics' in globals():
        summary_data.update({
            'page_count': text_statistics.get('page_count', 0),
            'char_count': text_statistics.get('char_count', 0),
            'word_count': text_statistics.get('word_count', 0)
        })
    
    # 구조 분석 통계
    if 'structure_analysis_result' in globals():
        structure_info = structure_analysis_result.get('structure_info', {})
        summary_data.update({
            'pages': structure_info.get('pages', 0),
            'paragraphs': structure_info.get('paragraphs', 0),
            'tables': structure_info.get('tables', 0),
            'lines': structure_info.get('lines', 0)
        })
    
    return summary_data

# 페이지별 분석 실행
if 'azure_analysis_result' in globals() and azure_analysis_result:
    print("🔍 페이지별 상세 분석 시작...")
    
    page_analysis_data = analyze_pages_individually()
    table_contents = show_table_contents()
    summary_data = show_processing_summary()
    
    if 'output_base_name' in globals() and page_analysis_data:
        save_result = save_page_analysis(page_analysis_data, table_contents, summary_data, output_base_name)
        if save_result:
            globals()['page_save_result'] = save_result
            globals()['page_analysis_data'] = page_analysis_data
            globals()['table_contents'] = table_contents
            globals()['summary_data'] = summary_data
            print(f"\n📂 페이지 분석 저장 위치: {OUTPUT_TEXTS_DIR}")
    
    print(f"\n🎯 페이지별 분석 완료!")
    print(f"   ✅ {len(page_analysis_data)}개 페이지 분석 성공")
    print(f"   📋 {len(table_contents)}개 테이블 내용 추출")
else:
    print("❌ 페이지별 분석을 실행하려면 먼저 텍스트 추출(27번 셀)을 완료해주세요.")

🔍 페이지별 상세 분석 시작...
💾 페이지 분석 저장 완료:
   📄 상세 보고서: /home/admin/wkms-aws/jupyter_notebook/data/output_texts/test_20250923_125927_page_analysis.txt
   📊 분석 데이터: /home/admin/wkms-aws/jupyter_notebook/data/output_texts/test_20250923_125927_page_analysis_data.json

📂 페이지 분석 저장 위치: /home/admin/wkms-aws/jupyter_notebook/data/output_texts

🎯 페이지별 분석 완료!
   ✅ 27개 페이지 분석 성공
   📋 11개 테이블 내용 추출


In [19]:
# 🎯 완전한 문서 분석 실행 함수 정의

import time

def run_complete_test(fast_mode=True, skip_pages_analysis=False, max_pages=None):
    """전체 Azure Document Intelligence 테스트를 순차적으로 실행
    
    Args:
        fast_mode (bool): 빠른 모드 (파일 저장 생략)
        skip_pages_analysis (bool): 페이지별 상세 분석 생략
        max_pages (int): 처리할 최대 페이지 수 (None이면 전체)
    """
    start_time = time.time()
    print("🚀 Azure Document Intelligence 완전 테스트 시작")
    print("=" * 70)
    
    if fast_mode:
        print("⚡ 빠른 모드 활성화 (파일 저장 최소화)")
    if skip_pages_analysis:
        print("⏭️ 페이지별 상세 분석 생략")
    if max_pages:
        print(f"📄 처리 제한: 최대 {max_pages}페이지")
    
    # 1. 환경 및 클라이언트 확인
    print(f"\n🔧 1단계: 환경 검증")
    if not globals().get('azure_client'):
        print("❌ Azure 클라이언트가 초기화되지 않았습니다.")
        print("   먼저 1-5번 셀을 순차적으로 실행해주세요.")
        return False
    
    # 2. 테스트 문서 확인
    if not globals().get('test_document_path'):
        print("❌ 테스트 문서가 선택되지 않았습니다.")
        print("   먼저 26번 셀(문서 선택)을 실행해주세요.")
        return False
    
    print(f"📂 입력 문서: {os.path.basename(test_document_path)}")
    print(f"📂 출력 디렉토리: {OUTPUT_TEXTS_DIR}")
    print(f"📝 출력 파일 기본명: {output_base_name}")
    print("   ✅ 환경 검증 완료")
    
    # 3. 전체 텍스트 추출 (성능 최적화)
    print(f"\n📄 2단계: 전체 텍스트 추출")
    text_start = time.time()
    try:
        # 페이지 제한 설정
        pages_param = f"1-{max_pages}" if max_pages else None
        
        # 임시로 빠른 모드 환경변수 설정
        original_fast_mode = os.environ.get('AZURE_FAST_MODE', 'false')
        if fast_mode:
            os.environ['AZURE_FAST_MODE'] = 'true'
        
        # 텍스트 추출 실행
        text_result = extract_full_text_test(test_document_path, pages=pages_param, fast_mode=fast_mode)
        
        # 환경변수 복원
        os.environ['AZURE_FAST_MODE'] = original_fast_mode
        
        if text_result:
            globals()['extracted_text'] = text_result['full_text']
            globals()['azure_analysis_result'] = text_result['azure_result']
            globals()['text_statistics'] = text_result['statistics']
            text_elapsed = time.time() - text_start
            print(f"   ✅ 텍스트 추출 완료 ({text_elapsed:.1f}초)")
        else:
            print("   ❌ 텍스트 추출 실패")
            return False
    except Exception as e:
        print(f"   ❌ 텍스트 추출 오류: {e}")
        return False
    
    # 4. 구조 분석 (조건부 실행)
    if not fast_mode or not skip_pages_analysis:
        print(f"\n📊 3단계: 구조 분석 (테이블, 레이아웃)")
        struct_start = time.time()
        try:
            structure_success = analyze_document_structure()
            if structure_success:
                struct_elapsed = time.time() - struct_start
                print(f"   ✅ 구조 분석 완료 ({struct_elapsed:.1f}초)")
            else:
                print("   ❌ 구조 분석 실패")
                return False
        except Exception as e:
            print(f"   ❌ 구조 분석 오류: {e}")
            return False
    else:
        print(f"\n📊 3단계: 구조 분석 - 빠른 모드로 생략")
    
    # 5. 페이지별 상세 분석 (조건부 실행)
    if not skip_pages_analysis:
        print(f"\n📃 4단계: 페이지별 상세 분석")
        pages_start = time.time()
        try:
            page_analysis_data = analyze_pages_individually()
            table_contents = show_table_contents()
            summary_data = show_processing_summary()
            
            if page_analysis_data:
                if not fast_mode:  # 파일 저장은 fast_mode가 아닐 때만
                    save_result = save_page_analysis(page_analysis_data, table_contents, summary_data, output_base_name)
                    if save_result:
                        globals()['page_save_result'] = save_result
                
                globals()['page_analysis_data'] = page_analysis_data
                globals()['table_contents'] = table_contents
                globals()['summary_data'] = summary_data
                pages_elapsed = time.time() - pages_start
                print(f"   ✅ 페이지별 분석 완료 ({pages_elapsed:.1f}초)")
            else:
                print("   ❌ 페이지별 분석 실패")
                return False
        except Exception as e:
            print(f"   ❌ 페이지별 분석 오류: {e}")
            return False
    else:
        print(f"\n📃 4단계: 페이지별 상세 분석 - 생략됨")
    
    # 6. 최종 요약
    total_elapsed = time.time() - start_time
    print(f"\n✅ 전체 테스트 완료! (총 소요시간: {total_elapsed:.1f}초)")
    print("=" * 70)
    
    # 핵심 확인사항 요약 호출
    try:
        if 'evaluate_core_checks' in globals():
            evaluate_core_checks()
        else:
            print("📊 간단 요약:")
            if 'text_statistics' in globals():
                stats = text_statistics
                print(f"   📝 추출된 텍스트: {stats.get('total_chars', 0):,}자")
                print(f"   📄 처리된 페이지: {stats.get('pages', 0)}페이지")
            if 'structure_analysis_result' in globals():
                struct = structure_analysis_result['structure_info']
                print(f"   📋 발견된 테이블: {struct.get('tables', 0)}개")
                print(f"   📏 텍스트 라인: {struct.get('lines', 0)}개")
    except Exception as e:
        print(f"⚠️ 요약 생성 중 오류: {e}")
    
    return True

def quick_test():
    """빠른 환경 및 연결 테스트"""
    print("🔧 빠른 환경 테스트")
    print("-" * 30)
    
    # Azure 클라이언트 확인
    if globals().get('azure_client'):
        print("✅ Azure 클라이언트: 정상")
    else:
        print("❌ Azure 클라이언트: 초기화 필요")
        return False
    
    # 디렉토리 확인
    if os.path.exists(INPUT_DOCS_DIR):
        print(f"✅ 입력 디렉토리: {INPUT_DOCS_DIR}")
    else:
        print(f"❌ 입력 디렉토리 없음: {INPUT_DOCS_DIR}")
        return False
    
    if os.path.exists(OUTPUT_TEXTS_DIR):
        print(f"✅ 출력 디렉토리: {OUTPUT_TEXTS_DIR}")
    else:
        print(f"❌ 출력 디렉토리 없음: {OUTPUT_TEXTS_DIR}")
        return False
    
    # 테스트 문서 확인
    if globals().get('test_document_path') and os.path.exists(test_document_path):
        print(f"✅ 테스트 문서: {os.path.basename(test_document_path)}")
    else:
        print("❌ 테스트 문서: 선택되지 않음 또는 파일 없음")
        return False
    
    print("✅ 환경 테스트 통과 - 전체 테스트 실행 가능")
    return True

def fast_document_test():
    """매우 빠른 문서 테스트 (첫 3페이지만, 파일 저장 없음)"""
    print("⚡ 초고속 문서 테스트 (3페이지, 저장 없음)")
    return run_complete_test(fast_mode=True, skip_pages_analysis=True, max_pages=3)

# 환경 설정 상태 확인
print("🔍 현재 환경 상태:")
quick_test_result = quick_test()

if quick_test_result:
    print("\n💡 사용 옵션:")
    print("   📄 run_complete_test() - 전체 분석 (기본)")
    print("   ⚡ run_complete_test(fast_mode=True) - 빠른 분석 (저장 최소)")
    print("   🚀 fast_document_test() - 초고속 테스트 (3페이지만)")
    print("   🔧 quick_test() - 환경 확인만")
else:
    print("\n⚠️ 기본 설정에 문제가 있습니다. 환경 설정을 먼저 확인해주세요.")

🔍 현재 환경 상태:
🔧 빠른 환경 테스트
------------------------------
✅ Azure 클라이언트: 정상
✅ 입력 디렉토리: /home/admin/wkms-aws/jupyter_notebook/data/input_docs
✅ 출력 디렉토리: /home/admin/wkms-aws/jupyter_notebook/data/output_texts
✅ 테스트 문서: test.pdf
✅ 환경 테스트 통과 - 전체 테스트 실행 가능

💡 사용 옵션:
   📄 run_complete_test() - 전체 분석 (기본)
   ⚡ run_complete_test(fast_mode=True) - 빠른 분석 (저장 최소)
   🚀 fast_document_test() - 초고속 테스트 (3페이지만)
   🔧 quick_test() - 환경 확인만


In [20]:
# 🎯 전체 테스트 실행 - 성능 최적화 옵션 포함

import time

print("🚀 Azure Document Intelligence 테스트 실행 옵션")
print("다음 중 하나를 선택하여 실행하세요:")
print("=" * 70)

# 성능 최적화 옵션들
print("📋 실행 옵션:")
print("   1️⃣ 초고속 테스트 (3페이지만, 저장 없음) - 30초 이내")
print("   2️⃣ 빠른 테스트 (전체 페이지, 저장 최소) - 1-2분")
print("   3️⃣ 완전 테스트 (전체 분석, 모든 저장) - 3-5분")
print("   🔧 환경 확인만")

# 자동 실행 여부 확인
auto_run = True  # False로 변경하면 수동 선택 모드

if auto_run:
    print(f"\n⚡ 자동 실행 모드: 초고속 테스트 시작...")
    print("   (더 긴 분석이 필요하면 auto_run=False로 변경 후 수동 선택)")
    
    start_time = time.time()
    
    # 함수 존재 확인
    if 'fast_document_test' in globals():
        success = fast_document_test()
        elapsed = time.time() - start_time
        
        if success:
            print(f"\n🎉 초고속 테스트 완료! (소요시간: {elapsed:.1f}초)")
            print("💡 더 상세한 분석이 필요하면 다음을 실행하세요:")
            print("   run_complete_test(fast_mode=True)  # 빠른 전체 분석")
            print("   run_complete_test(fast_mode=False) # 완전 분석 + 저장")
        else:
            print(f"\n❌ 테스트 실패 (소요시간: {elapsed:.1f}초)")
    else:
        print("❌ 테스트 함수가 정의되지 않았습니다.")
        print("   30번 셀을 먼저 실행해주세요.")
        
else:
    print(f"\n🔧 수동 선택 모드:")
    print("다음 명령어 중 하나를 실행하세요:")
    print("   fast_document_test()                    # 초고속")
    print("   run_complete_test(fast_mode=True)       # 빠른 전체")
    print("   run_complete_test(fast_mode=False)      # 완전 분석")
    print("   quick_test()                            # 환경 확인만")

print("\n💡 Tip: auto_run 변수를 변경하여 실행 모드를 선택할 수 있습니다.")

🚀 Azure Document Intelligence 테스트 실행 옵션
다음 중 하나를 선택하여 실행하세요:
📋 실행 옵션:
   1️⃣ 초고속 테스트 (3페이지만, 저장 없음) - 30초 이내
   2️⃣ 빠른 테스트 (전체 페이지, 저장 최소) - 1-2분
   3️⃣ 완전 테스트 (전체 분석, 모든 저장) - 3-5분
   🔧 환경 확인만

⚡ 자동 실행 모드: 초고속 테스트 시작...
   (더 긴 분석이 필요하면 auto_run=False로 변경 후 수동 선택)
⚡ 초고속 문서 테스트 (3페이지, 저장 없음)
🚀 Azure Document Intelligence 완전 테스트 시작
⚡ 빠른 모드 활성화 (파일 저장 최소화)
⏭️ 페이지별 상세 분석 생략
📄 처리 제한: 최대 3페이지

🔧 1단계: 환경 검증
📂 입력 문서: test.pdf
📂 출력 디렉토리: /home/admin/wkms-aws/jupyter_notebook/data/output_texts
📝 출력 파일 기본명: test_20250923_125927
   ✅ 환경 검증 완료

📄 2단계: 전체 텍스트 추출
🚀 전체 텍스트 추출 시작...
📄 파일: test.pdf
🔧 페이지 제한 적용: pages='1-3' (해당 페이지만 분석)
⚡ Fast Mode: 텍스트 저장 없이 미리보기/통계만 수행
⏳ Azure Document Intelligence 분석 중...
⏳ Azure Document Intelligence 분석 중...
⏱️ 분석 완료 (경과 시간: 3.7s)

✅ 텍스트 추출 완료!
   📃 총 페이지: 3페이지
   📝 총 글자수: 9,775자
   📄 총 줄수: 32줄
   🔤 총 단어수: 1,449개

📖 텍스트 미리보기 (첫 500자):
韓國IT서비스學會誌 第22卷 第3號 2023年 6月, pp.1-27
Journal of Information Technology Services https://doi.org/10.9716/KITS.2023.22.3.001
양손잡이 리더십과

## 🎯 Azure Document Intelligence 실전 테스트 완료!

### ✅ 구현된 핵심 기능들

1. **전체 텍스트 추출 확인** ✅
   - PDF 문서에서 모든 텍스트 추출
   - 텍스트 품질 및 신뢰도 측정
   - 페이지별 텍스트 통계 제공
   - **결과 파일 자동 저장**: `*_full_text.txt`, `*_statistics.json`

2. **페이지별 그림/테이블 인식** ✅
   - 테이블 구조 자동 인식 및 셀 내용 추출
   - 문단, 제목, 목록 등 레이아웃 요소 분석
   - 페이지별 구조적 요소 상세 분석
   - **결과 파일 자동 저장**: `*_structure_analysis.txt`, `*_structure_data.json`

3. **페이지별 결과 체크** ✅
   - 각 페이지별 상세 분석 결과 표시
   - 텍스트 라인 수, 단어 수, 신뢰도 측정
   - 테이블 내용 행/열별 상세 출력
   - **결과 파일 자동 저장**: `*_page_analysis.txt`, `*_page_analysis_data.json`

### ? 디렉토리 구조

#### 입력 디렉토리
- **경로**: `/home/admin/wkms-aws/jupyter_notebook/data/input_docs`
- **용도**: 분석할 PDF 문서 저장
- **기본 파일**: `test.pdf` (학술논문 이미지 PDF)

#### 출력 디렉토리  
- **경로**: `/home/admin/wkms-aws/jupyter_notebook/data/output_texts`
- **용도**: 분석 결과 및 추출된 텍스트 저장
- **생성 파일**:
  - `{문서명}_full_text.txt`: 전체 추출 텍스트
  - `{문서명}_statistics.json`: 텍스트 추출 통계
  - `{문서명}_structure_analysis.txt`: 구조 분석 보고서
  - `{문서명}_structure_data.json`: 구조 분석 데이터
  - `{문서명}_page_analysis.txt`: 페이지별 상세 분석
  - `{문서명}_page_analysis_data.json`: 페이지별 분석 데이터

### ?🚀 사용 방법

#### 빠른 테스트
- **셀 5**: 기본 연결 및 설정 확인

#### 개별 단계 테스트
- **셀 6**: 문서 선택 및 기본 정보 (test.pdf 우선 선택)
- **셀 7**: 전체 텍스트 추출 테스트 + 파일 저장
- **셀 8**: 구조적 요소 분석 + 파일 저장
- **셀 9**: 페이지별 상세 분석 + 파일 저장

#### 전체 자동 테스트
- **셀 11**: 한 번 실행으로 모든 단계 자동 수행 + 결과 저장

### 💡 활용 가능성
- ✅ **텍스트 검색**: 추출된 텍스트로 키워드 검색
- ✅ **AI 질의응답**: 문서 내용 기반 질문 답변
- ✅ **데이터 분석**: 테이블 데이터 구조화 및 분석
- ✅ **의미 검색**: 문맥 기반 고급 검색
- ✅ **결과 보관**: 모든 분석 결과 파일로 저장

### 📊 품질 보장
- **신뢰도 측정**: 각 단어별 인식 정확도 확인
- **구조 인식**: 테이블, 문단 등 레이아웃 정확도 검증
- **다중 형식 지원**: PDF, 이미지 등 다양한 문서 형식 처리
- **학술논문 최적화**: 복잡한 레이아웃과 수식이 포함된 논문 처리

# 🔄 문서 처리 파이프라인 비교 분석

## 📋 개요
이 섹션에서는 현재 백엔드에서 사용하는 두 가지 문서 처리 파이프라인을 구현하고 비교분석합니다:

### 1️⃣ 오픈소스 기반 문서처리 파이프라인
- **PyPDF2, python-docx, python-pptx** 등 오픈소스 라이브러리 활용
- **BeautifulSoup**을 통한 HTML/XML 파싱
- **정규표현식**을 활용한 텍스트 정제
- **비용**: 무료 (컴퓨팅 자원만 소모)
- **장점**: 빠른 처리, 커스터마이징 용이, 비용 효율적
- **단점**: 복잡한 레이아웃 처리 한계, OCR 기능 없음

### 2️⃣ Azure Document Intelligence 기반 파이프라인  
- **Azure AI Services** 활용한 고급 문서 분석
- **OCR, 테이블 인식, 구조 분석** 등 AI 기능
- **prebuilt-layout, prebuilt-read** 모델 활용
- **비용**: 사용량 기반 과금
- **장점**: 높은 정확도, 복잡한 구조 처리, OCR 지원
- **단점**: 비용 발생, 외부 서비스 의존성

---

## 🎯 비교 분석 목표
1. **성능 비교**: 처리 속도, 정확도, 안정성
2. **비용 분석**: 운영 비용, ROI 계산
3. **적용 시나리오**: 문서 유형별 최적 선택
4. **백터스토어 호환성**: 임베딩 생성 및 저장 효율성

---

## 🔧 1단계: 오픈소스 라이브러리 설치 및 설정

In [ ]:
# 🔧 오픈소스 문서 처리 라이브러리 설치 및 임포트

import subprocess
import sys
import os
import json
import re
import time
from datetime import datetime
from pathlib import Path

# 필요한 라이브러리 설치
def install_opensource_libraries():
    """오픈소스 문서 처리에 필요한 라이브러리들을 설치합니다."""
    libraries = [
        'PyPDF2',           # PDF 처리
        'python-docx',      # Word 문서 처리  
        'python-pptx',      # PowerPoint 처리
        'beautifulsoup4',   # HTML/XML 파싱
        'lxml',            # XML 파서
        'openpyxl',        # Excel 처리
        'pandas',          # 데이터 처리
        'pillow',          # 이미지 처리
        'tiktoken'         # 토큰 계산
    ]
    
    print("📦 오픈소스 라이브러리 설치 중...")
    for lib in libraries:
        try:
            subprocess.check_call([sys.executable, "-m", "pip", "install", lib, "--quiet"])
            print(f"   ✅ {lib} 설치 완료")
        except subprocess.CalledProcessError as e:
            print(f"   ❌ {lib} 설치 실패: {e}")

# 라이브러리 설치 실행
install_opensource_libraries()

# 라이브러리 임포트
try:
    import PyPDF2
    from docx import Document
    from pptx import Presentation
    from bs4 import BeautifulSoup
    import pandas as pd
    import tiktoken
    from PIL import Image
    
    print("\n✅ 모든 오픈소스 라이브러리 임포트 성공!")
    
    # 버전 정보 출력
    print(f"📚 라이브러리 버전:")
    print(f"   - PyPDF2: {PyPDF2.__version__}")
    print(f"   - python-docx: 사용 가능")
    print(f"   - python-pptx: 사용 가능") 
    print(f"   - BeautifulSoup4: {BeautifulSoup.__version__ if hasattr(BeautifulSoup, '__version__') else '사용 가능'}")
    print(f"   - pandas: {pd.__version__}")
    
    # 전역 변수로 저장
    globals()['opensource_libraries_ready'] = True
    
except ImportError as e:
    print(f"❌ 라이브러리 임포트 실패: {e}")
    globals()['opensource_libraries_ready'] = False

# 오픈소스 처리용 디렉토리 설정
OPENSOURCE_OUTPUT_DIR = os.path.join(OUTPUT_TEXTS_DIR, "opensource_results")
os.makedirs(OPENSOURCE_OUTPUT_DIR, exist_ok=True)

print(f"\n📁 오픈소스 처리 결과 저장 디렉토리: {OPENSOURCE_OUTPUT_DIR}")

## 📄 2단계: 오픈소스 기반 문서 처리기 구현

In [ ]:
# 📄 오픈소스 기반 문서 처리기 클래스 구현

class OpenSourceDocumentProcessor:
    """오픈소스 라이브러리를 활용한 문서 처리 클래스"""
    
    def __init__(self, output_dir=None):
        self.output_dir = output_dir or OPENSOURCE_OUTPUT_DIR
        self.tokenizer = tiktoken.get_encoding("cl100k_base")
        self.supported_formats = ['.pdf', '.docx', '.pptx', '.txt', '.html', '.xml']
        
    def extract_text_from_pdf(self, file_path):
        """PDF 파일에서 텍스트 추출 (PyPDF2 사용)"""
        try:
            with open(file_path, 'rb') as file:
                pdf_reader = PyPDF2.PdfReader(file)
                text = ""
                metadata = {
                    'pages': len(pdf_reader.pages),
                    'title': pdf_reader.metadata.get('/Title', 'Unknown') if pdf_reader.metadata else 'Unknown',
                    'author': pdf_reader.metadata.get('/Author', 'Unknown') if pdf_reader.metadata else 'Unknown'
                }
                
                for page_num, page in enumerate(pdf_reader.pages, 1):
                    page_text = page.extract_text()
                    text += f"\n\n--- 페이지 {page_num} ---\n{page_text}"
                
                return {
                    'text': text.strip(),
                    'metadata': metadata,
                    'success': True,
                    'error': None
                }
        except Exception as e:
            return {
                'text': None,
                'metadata': None,
                'success': False,
                'error': str(e)
            }
    
    def extract_text_from_docx(self, file_path):
        """Word 문서에서 텍스트 추출 (python-docx 사용)"""
        try:
            doc = Document(file_path)
            
            # 메타데이터 추출
            core_props = doc.core_properties
            metadata = {
                'title': core_props.title or 'Unknown',
                'author': core_props.author or 'Unknown',
                'created': str(core_props.created) if core_props.created else 'Unknown',
                'modified': str(core_props.modified) if core_props.modified else 'Unknown',
                'paragraphs': len(doc.paragraphs)
            }
            
            # 텍스트 추출
            text = ""
            for para in doc.paragraphs:
                if para.text.strip():
                    text += para.text + "\n"
            
            # 테이블 추출
            tables_text = ""
            for table in doc.tables:
                tables_text += "\n\n--- 테이블 ---\n"
                for row in table.rows:
                    row_text = []
                    for cell in row.cells:
                        row_text.append(cell.text.strip())
                    tables_text += " | ".join(row_text) + "\n"
            
            full_text = text + tables_text
            
            return {
                'text': full_text.strip(),
                'metadata': metadata,
                'success': True,
                'error': None
            }
        except Exception as e:
            return {
                'text': None,
                'metadata': None,
                'success': False,
                'error': str(e)
            }
    
    def extract_text_from_pptx(self, file_path):
        """PowerPoint 파일에서 텍스트 추출 (python-pptx 사용)"""
        try:
            prs = Presentation(file_path)
            
            metadata = {
                'slides': len(prs.slides),
                'title': 'PowerPoint Presentation'
            }
            
            text = ""
            for slide_num, slide in enumerate(prs.slides, 1):
                slide_text = f"\n\n--- 슬라이드 {slide_num} ---\n"
                
                for shape in slide.shapes:
                    if hasattr(shape, "text") and shape.text.strip():
                        slide_text += shape.text + "\n"
                
                text += slide_text
            
            return {
                'text': text.strip(),
                'metadata': metadata,
                'success': True,
                'error': None
            }
        except Exception as e:
            return {
                'text': None,
                'metadata': None,
                'success': False,
                'error': str(e)
            }
    
    def extract_text_from_txt(self, file_path):
        """텍스트 파일에서 내용 추출"""
        try:
            with open(file_path, 'r', encoding='utf-8') as file:
                text = file.read()
            
            metadata = {
                'size': len(text),
                'lines': len(text.split('\n'))
            }
            
            return {
                'text': text,
                'metadata': metadata,
                'success': True,
                'error': None
            }
        except UnicodeDecodeError:
            try:
                with open(file_path, 'r', encoding='cp949') as file:
                    text = file.read()
                metadata = {
                    'size': len(text),
                    'lines': len(text.split('\n')),
                    'encoding': 'cp949'
                }
                return {
                    'text': text,
                    'metadata': metadata,
                    'success': True,
                    'error': None
                }
            except Exception as e:
                return {
                    'text': None,
                    'metadata': None,
                    'success': False,
                    'error': f"Encoding error: {e}"
                }
        except Exception as e:
            return {
                'text': None,
                'metadata': None,
                'success': False,
                'error': str(e)
            }
    
    def process_html_content(self, content):
        """HTML 콘텐츠 처리 (백엔드 tools.py의 process_file 함수 기반)"""
        try:
            # BeautifulSoup으로 HTML 파싱
            soup = BeautifulSoup(content, 'html.parser')
            
            # 테이블 처리
            tables = soup.find_all('table')
            for table in tables:
                table_text = self.convert_table_to_text(table)
                table.replace_with(table_text)
            
            # 불필요한 태그 제거
            for tag in soup(['script', 'style', 'meta', 'link']):
                tag.decompose()
            
            # 텍스트 추출 및 정제
            text = soup.get_text()
            
            # 정규표현식으로 정제
            text = re.sub(r'<!--.*?-->', '', text, flags=re.DOTALL)  # HTML 주석 제거
            text = re.sub(r'\n{3,}', '\n\n', text)  # 연속된 개행 정리
            text = re.sub(r'[ \t]+', ' ', text)  # 연속된 공백 정리
            
            return text.strip()
            
        except Exception as e:
            print(f"⚠️ HTML 처리 중 오류: {e}")
            return content
    
    def convert_table_to_text(self, table):
        """HTML 테이블을 텍스트 형태로 변환"""
        try:
            rows = table.find_all('tr')
            if not rows:
                return ""
            
            table_text = "\n\n--- 테이블 ---\n"
            for row in rows:
                cells = row.find_all(['td', 'th'])
                row_text = []
                for cell in cells:
                    cell_text = cell.get_text().strip()
                    row_text.append(cell_text)
                if row_text:
                    table_text += " | ".join(row_text) + "\n"
            
            return table_text
            
        except Exception as e:
            return "\n[테이블 처리 오류]\n"
    
    def calculate_tokens(self, text):
        """텍스트의 토큰 수 계산"""
        return len(self.tokenizer.encode(text))
    
    def chunk_text(self, text, max_tokens=6000, min_tokens=1000, overlap_percentage=0.1):
        """텍스트를 청킹 (백엔드 tools.py의 chunked_texts 함수 기반)"""
        tokens = self.tokenizer.encode(text)
        overlap_tokens = int(max_tokens * overlap_percentage)
        chunks = []
        start_index = 0
        
        while start_index < len(tokens):
            if start_index + max_tokens >= len(tokens):
                # 마지막 청크 처리
                remaining_tokens = len(tokens) - start_index
                if remaining_tokens < min_tokens and chunks:
                    # 마지막 청크가 너무 작으면 이전 청크와 합치기
                    chunks[-1] += self.tokenizer.decode(tokens[start_index:])
                    break
                else:
                    chunks.append(self.tokenizer.decode(tokens[start_index:]))
                    break
            else:
                end_index = min(start_index + max_tokens, len(tokens))
                chunk = self.tokenizer.decode(tokens[start_index:end_index])
                chunks.append(chunk)
                start_index += max_tokens - overlap_tokens
        
        return chunks

print("📄 오픈소스 문서 처리기 클래스 정의 완료!")
if 'opensource_libraries_ready' in globals() and opensource_libraries_ready:
    globals()['opensource_processor'] = OpenSourceDocumentProcessor()
    print("✅ 오픈소스 문서 처리기 인스턴스 생성 완료!")

## 🚀 3단계: 오픈소스 기반 문서 처리 파이프라인 실행

In [ ]:
# 🚀 오픈소스 기반 문서 처리 파이프라인 구현

def run_opensource_pipeline(file_path, save_results=True, chunk_text=True):
    """
    오픈소스 라이브러리를 사용한 문서 처리 파이프라인
    
    Args:
        file_path (str): 처리할 문서 파일 경로
        save_results (bool): 결과를 파일로 저장할지 여부
        chunk_text (bool): 텍스트를 청킹할지 여부
    
    Returns:
        dict: 처리 결과 정보
    """
    start_time = time.time()
    
    print("🚀 오픈소스 기반 문서 처리 파이프라인 시작")
    print("=" * 70)
    
    if not globals().get('opensource_processor'):
        print("❌ 오픈소스 문서 처리기가 초기화되지 않았습니다.")
        return None
    
    processor = opensource_processor
    file_path = Path(file_path)
    
    if not file_path.exists():
        print(f"❌ 파일이 존재하지 않습니다: {file_path}")
        return None
    
    file_extension = file_path.suffix.lower()
    file_name = file_path.stem
    
    print(f"📂 입력 파일: {file_path.name}")
    print(f"📝 파일 형식: {file_extension}")
    
    # 1단계: 파일 형식별 텍스트 추출
    print(f"\n📄 1단계: 텍스트 추출 ({file_extension})")
    extract_start = time.time()
    
    if file_extension == '.pdf':
        result = processor.extract_text_from_pdf(file_path)
    elif file_extension == '.docx':
        result = processor.extract_text_from_docx(file_path)
    elif file_extension == '.pptx':
        result = processor.extract_text_from_pptx(file_path)
    elif file_extension == '.txt':
        result = processor.extract_text_from_txt(file_path)
    else:
        print(f"❌ 지원하지 않는 파일 형식: {file_extension}")
        return None
    
    if not result['success']:
        print(f"❌ 텍스트 추출 실패: {result['error']}")
        return None
    
    extracted_text = result['text']
    metadata = result['metadata']
    extract_elapsed = time.time() - extract_start
    
    print(f"   ✅ 텍스트 추출 완료 ({extract_elapsed:.2f}초)")
    print(f"   📊 추출된 텍스트 길이: {len(extracted_text):,}자")
    
    # 2단계: HTML/마크다운 콘텐츠 정제
    print(f"\n🔧 2단계: 텍스트 정제")
    clean_start = time.time()
    
    if '<' in extracted_text and '>' in extracted_text:
        cleaned_text = processor.process_html_content(extracted_text)
        print(f"   ✅ HTML/XML 태그 정제 완료")
    else:
        cleaned_text = extracted_text
        print(f"   ✅ 텍스트 정제 생략 (HTML 태그 없음)")
    
    clean_elapsed = time.time() - clean_start
    print(f"   ⏱️ 정제 시간: {clean_elapsed:.2f}초")
    
    # 3단계: 토큰 계산
    print(f"\n🔢 3단계: 토큰 분석")
    token_count = processor.calculate_tokens(cleaned_text)\n    print(f"   📊 총 토큰 수: {token_count:,}개")
    \n    # 4단계: 텍스트 청킹 (옵션)
    chunks = []
    if chunk_text and token_count > 6000:
        print(f"\n✂️ 4단계: 텍스트 청킹")
        chunk_start = time.time()
        
        chunks = processor.chunk_text(cleaned_text, max_tokens=6000, overlap_percentage=0.1)
        chunk_elapsed = time.time() - chunk_start
        
        print(f"   ✅ 청킹 완료: {len(chunks)}개 청크")
        print(f"   ⏱️ 청킹 시간: {chunk_elapsed:.2f}초")
        
        for i, chunk in enumerate(chunks):
            chunk_tokens = processor.calculate_tokens(chunk)
            print(f"      청크 {i+1}: {chunk_tokens:,} 토큰")
    else:
        print(f"\n✂️ 4단계: 텍스트 청킹 생략")
        chunks = [cleaned_text]
    
    # 5단계: 결과 저장 (옵션)
    saved_files = []
    if save_results:
        print(f"\n💾 5단계: 결과 저장")
        save_start = time.time()
        
        # 메타데이터 저장
        metadata_file = os.path.join(OPENSOURCE_OUTPUT_DIR, f"{file_name}_metadata.json")
        full_metadata = {
            'file_name': file_path.name,
            'file_size': file_path.stat().st_size,
            'processing_time': time.time() - start_time,
            'extraction_method': 'opensource',
            'file_type': file_extension,
            'text_length': len(cleaned_text),
            'token_count': token_count,
            'chunks_count': len(chunks),
            'metadata': metadata,
            'timestamp': datetime.now().isoformat()
        }
        
        with open(metadata_file, 'w', encoding='utf-8') as f:
            json.dump(full_metadata, f, ensure_ascii=False, indent=2)
        saved_files.append(metadata_file)
        
        # 전체 텍스트 저장
        text_file = os.path.join(OPENSOURCE_OUTPUT_DIR, f"{file_name}_fulltext.txt")
        with open(text_file, 'w', encoding='utf-8') as f:
            f.write(cleaned_text)
        saved_files.append(text_file)
        
        # 청크별 저장
        if len(chunks) > 1:
            for i, chunk in enumerate(chunks):
                chunk_file = os.path.join(OPENSOURCE_OUTPUT_DIR, f"{file_name}_chunk_{i+1}.txt")
                with open(chunk_file, 'w', encoding='utf-8') as f:
                    f.write(chunk)
                saved_files.append(chunk_file)
        
        save_elapsed = time.time() - save_start
        print(f"   ✅ 파일 저장 완료 ({save_elapsed:.2f}초)")
        print(f"   📁 저장된 파일 수: {len(saved_files)}개")
    
    # 최종 결과
    total_elapsed = time.time() - start_time
    
    result_summary = {
        'success': True,
        'file_path': str(file_path),
        'file_name': file_path.name,
        'processing_method': 'opensource',
        'text_length': len(cleaned_text),
        'token_count': token_count,
        'chunks': chunks,
        'chunks_count': len(chunks),
        'metadata': full_metadata if save_results else metadata,
        'saved_files': saved_files,
        'processing_time': total_elapsed,
        'extract_time': extract_elapsed,
        'clean_time': clean_elapsed
    }
    
    print(f"\n✅ 오픈소스 파이프라인 완료!")
    print(f"   ⏱️ 총 처리시간: {total_elapsed:.2f}초")
    print(f"   📊 처리 결과: {len(cleaned_text):,}자, {token_count:,} 토큰")
    print(f"   📁 청크 수: {len(chunks)}개")
    
    # 전역 변수로 저장
    globals()['opensource_result'] = result_summary
    globals()['opensource_text'] = cleaned_text
    globals()['opensource_chunks'] = chunks
    
    return result_summary

# 빠른 테스트 함수
def quick_opensource_test(file_path):
    """빠른 오픈소스 테스트 (저장 없음, 청킹 없음)"""
    return run_opensource_pipeline(file_path, save_results=False, chunk_text=False)

print("🚀 오픈소스 파이프라인 함수 정의 완료!")
print("💡 사용법:")
print("   - run_opensource_pipeline(file_path) : 전체 파이프라인")
print("   - quick_opensource_test(file_path)   : 빠른 테스트")

## 🧪 4단계: 오픈소스 파이프라인 테스트 실행

In [ ]:
# 🧪 오픈소스 파이프라인 테스트 실행

print("🧪 오픈소스 기반 문서 처리 파이프라인 테스트")
print("=" * 60)

# 테스트 파일 확인
if 'test_document_path' in globals() and test_document_path:
    print(f"📂 테스트 문서: {os.path.basename(test_document_path)}")
    
    # 오픈소스 파이프라인 실행
    print(f"\n🚀 오픈소스 파이프라인 실행 중...")
    opensource_result = run_opensource_pipeline(
        file_path=test_document_path,
        save_results=True,
        chunk_text=True
    )
    
    if opensource_result and opensource_result['success']:
        print(f"\n✅ 오픈소스 파이프라인 테스트 성공!")
        
        # 결과 요약 출력
        print(f"\n📊 처리 결과 요약:")
        print(f"   📄 파일명: {opensource_result['file_name']}")
        print(f"   ⏱️ 처리시간: {opensource_result['processing_time']:.2f}초")
        print(f"   📝 텍스트 길이: {opensource_result['text_length']:,}자")
        print(f"   🔢 토큰 수: {opensource_result['token_count']:,}개")
        print(f"   📁 청크 수: {opensource_result['chunks_count']}개")
        print(f"   💾 저장 파일: {len(opensource_result['saved_files'])}개")
        
        # 청크 미리보기
        if 'opensource_chunks' in globals() and opensource_chunks:
            print(f"\n📖 첫 번째 청크 미리보기 (처음 500자):")
            preview_text = opensource_chunks[0][:500]
            print(f"   {preview_text}...")
        
        # 저장된 파일 목록
        print(f"\n📁 저장된 파일들:")
        for file_path in opensource_result['saved_files']:
            file_size = os.path.getsize(file_path)
            print(f"   - {os.path.basename(file_path)} ({file_size:,} bytes)")
    
    else:
        print(f"❌ 오픈소스 파이프라인 테스트 실패")
        
else:
    print("❌ 테스트 문서가 설정되지 않았습니다.")
    print("   먼저 이전 셀들을 실행하여 테스트 문서를 선택해주세요.")
    
    # 사용 가능한 테스트 문서 목록 표시
    if 'TEST_DOCUMENTS' in globals():
        print(f"\n📋 사용 가능한 테스트 문서:")
        for i, doc in enumerate(TEST_DOCUMENTS, 1):
            doc_path = os.path.join(INPUT_DOCS_DIR, doc)
            if os.path.exists(doc_path):
                size = os.path.getsize(doc_path)
                print(f"   {i}. {doc} ({size:,} bytes)")

print(f"\n💡 Tip: 빠른 테스트를 원하면 quick_opensource_test(file_path)를 사용하세요!")

## 🗃️ 5단계: 벡터스토어 연동 및 임베딩 생성

In [ ]:
# 🗃️ 벡터스토어 연동 및 임베딩 생성 구현

class VectorStoreProcessor:
    """벡터스토어 처리 및 임베딩 생성 클래스"""
    
    def __init__(self):
        self.embedding_model = "text-embedding-ada-002"  # 백엔드에서 사용하는 임베딩 모델
        self.vector_dimension = 1536  # Ada-002 모델의 벡터 차원
        
    def create_document_chunks_for_vectorstore(self, text_chunks, file_info, method='opensource'):
        """
        백터스토어 저장용 문서 청크 생성 (백엔드 tools.py의 preprocessing_documents 함수 기반)
        
        Args:
            text_chunks (list): 텍스트 청크 리스트
            file_info (dict): 파일 정보
            method (str): 처리 방법 ('opensource' 또는 'azure_di')
        
        Returns:
            list: 벡터스토어용 문서 청크 리스트
        """
        now = datetime.now()
        preprocessing_time = now.strftime("%Y-%m-%dT00:00:00Z")
        
        file_name = file_info.get('file_name', 'unknown')
        file_logical_name = os.path.splitext(file_name)[0]  # 확장자 제거
        title = text_chunks[0].strip().split('\n')[0] if text_chunks else 'Unknown Title'
        
        vectorstore_documents = []
        
        for chunk_idx, chunk_text in enumerate(text_chunks):
            chunk_text = chunk_text.strip()
            
            # 백엔드와 동일한 main_text 형식
            main_text = f"""(file_name) {file_logical_name}

(title) {title}

(content)
{chunk_text}""".strip()
            
            # 백엔드와 동일한 document 구조
            document = {
                "file_index_id": f"{method}_{file_logical_name}_{chunk_idx}",
                "file_lgc_nm": file_logical_name,
                "file_psl_nm": file_name,
                "title": title,
                "page_num": chunk_idx + 1,  # 청크를 페이지로 간주
                "chunk_num": chunk_idx,
                "chunk_text": chunk_text,
                "main_text": main_text,
                "preprocessing_time": preprocessing_time,
                "processing_method": method,
                "del_yn": False,
                "metadata": {
                    "extraction_method": method,
                    "chunk_index": chunk_idx,
                    "total_chunks": len(text_chunks),
                    "file_size": file_info.get('file_size', 0),
                    "processing_time": file_info.get('processing_time', 0)
                }
            }
            
            # 백엔드와 동일한 ID 생성 (base64 인코딩)
            import base64
            original_key = f"{document['file_psl_nm']}_{document['page_num']}_{document['chunk_num']}"
            encoded_key = base64.urlsafe_b64encode(original_key.encode('utf-8')).decode('utf-8')
            document['id'] = encoded_key
            
            vectorstore_documents.append(document)
        
        return vectorstore_documents
    
    def simulate_embedding_generation(self, documents):
        """
        임베딩 생성 시뮬레이션 (실제 환경에서는 Azure OpenAI 사용)
        
        Args:
            documents (list): 벡터스토어용 문서 리스트
        
        Returns:
            list: 임베딩이 추가된 문서 리스트
        """
        import random
        import numpy as np
        
        print(f"🔄 임베딩 생성 중... ({len(documents)}개 문서)")
        
        for i, doc in enumerate(documents):
            # 실제 환경에서는 Azure OpenAI API 호출
            # chunk_text_vector = aoai_instance.generate_embeddings(doc['chunk_text'])
            # main_text_vector = aoai_instance.generate_embeddings(doc['main_text'])
            
            # 시뮬레이션용 랜덤 벡터 생성
            np.random.seed(hash(doc['chunk_text']) % 2**32)  # 재현 가능한 랜덤
            chunk_text_vector = np.random.randn(self.vector_dimension).tolist()
            main_text_vector = np.random.randn(self.vector_dimension).tolist()
            
            doc['chunk_text_vector'] = chunk_text_vector
            doc['main_text_vector'] = main_text_vector
            
            if i % 5 == 0:  # 진행률 표시
                print(f"   📊 진행률: {i+1}/{len(documents)} ({((i+1)/len(documents)*100):.1f}%)")
        
        print(f"✅ 임베딩 생성 완료!")
        return documents
    
    def simulate_vectorstore_upload(self, documents, save_to_file=True):
        """
        벡터스토어 업로드 시뮬레이션
        
        Args:
            documents (list): 임베딩이 포함된 문서 리스트
            save_to_file (bool): 결과를 파일로 저장할지 여부
        
        Returns:
            dict: 업로드 결과 정보
        """
        print(f"🗃️ 벡터스토어 업로드 시뮬레이션...")
        
        upload_results = {
            'total_documents': len(documents),
            'successful_uploads': 0,
            'failed_uploads': 0,
            'uploaded_documents': [],
            'upload_errors': []
        }
        
        for doc in documents:
            try:
                # 실제 환경에서는 Cosmos DB 또는 AI Search 업로드
                # result = cosmos_db.upload_item(doc)
                # result = ai_search.upload_to_index(doc)
                
                # 시뮬레이션: 성공적으로 업로드되었다고 가정
                upload_results['successful_uploads'] += 1
                upload_results['uploaded_documents'].append({
                    'id': doc['id'],
                    'file_name': doc['file_psl_nm'],
                    'chunk_num': doc['chunk_num'],
                    'vector_dimension': len(doc['chunk_text_vector'])
                })
                
            except Exception as e:
                upload_results['failed_uploads'] += 1
                upload_results['upload_errors'].append({
                    'id': doc['id'],
                    'error': str(e)
                })
        
        # 결과 저장
        if save_to_file:
            result_file = os.path.join(OPENSOURCE_OUTPUT_DIR, "vectorstore_upload_result.json")
            with open(result_file, 'w', encoding='utf-8') as f:
                json.dump(upload_results, f, ensure_ascii=False, indent=2)
            print(f"   💾 업로드 결과 저장: {result_file}")
        
        print(f"✅ 벡터스토어 업로드 완료!")
        print(f"   📊 성공: {upload_results['successful_uploads']}개")
        print(f"   ❌ 실패: {upload_results['failed_uploads']}개")
        
        return upload_results

def run_vectorstore_pipeline(processing_result, method='opensource'):
    """
    벡터스토어 연동 파이프라인 실행
    
    Args:
        processing_result (dict): 문서 처리 결과
        method (str): 처리 방법
    
    Returns:
        dict: 벡터스토어 처리 결과
    """
    print(f"🗃️ 벡터스토어 연동 파이프라인 시작 ({method})")
    print("=" * 60)
    
    if not processing_result or not processing_result.get('success'):
        print("❌ 유효한 문서 처리 결과가 없습니다.")
        return None
    
    vector_processor = VectorStoreProcessor()
    
    # 1단계: 벡터스토어용 문서 청크 생성
    print(f"📄 1단계: 벡터스토어용 문서 청크 생성")
    file_info = {
        'file_name': processing_result['file_name'],
        'file_size': os.path.getsize(processing_result['file_path']) if os.path.exists(processing_result['file_path']) else 0,
        'processing_time': processing_result.get('processing_time', 0)
    }
    
    chunks = processing_result.get('chunks', [])
    vectorstore_documents = vector_processor.create_document_chunks_for_vectorstore(
        chunks, file_info, method
    )
    
    print(f"   ✅ 벡터스토어용 문서 생성: {len(vectorstore_documents)}개")
    
    # 2단계: 임베딩 생성
    print(f"\n🔄 2단계: 임베딩 생성")
    embedded_documents = vector_processor.simulate_embedding_generation(vectorstore_documents)
    
    # 3단계: 벡터스토어 업로드
    print(f"\n🗃️ 3단계: 벡터스토어 업로드")
    upload_result = vector_processor.simulate_vectorstore_upload(embedded_documents)
    
    # 최종 결과
    final_result = {
        'method': method,
        'vectorstore_documents_count': len(vectorstore_documents),
        'embedding_dimension': vector_processor.vector_dimension,
        'upload_result': upload_result,
        'vectorstore_documents': vectorstore_documents  # 실제 환경에서는 제외 가능
    }
    
    print(f"\n✅ 벡터스토어 연동 완료!")
    print(f"   📄 처리된 문서: {len(vectorstore_documents)}개")
    print(f"   🔢 벡터 차원: {vector_processor.vector_dimension}")
    print(f"   📊 업로드 성공률: {upload_result['successful_uploads']}/{upload_result['total_documents']}")
    
    # 전역 변수로 저장
    globals()[f'{method}_vectorstore_result'] = final_result
    
    return final_result

print("🗃️ 벡터스토어 연동 클래스 및 함수 정의 완료!")
print("💡 사용법: run_vectorstore_pipeline(processing_result, method='opensource')")

## 📊 6단계: 두 파이프라인 비교 분석

In [ ]:
# 📊 두 파이프라인 비교 분석 실행

def compare_pipelines():
    """두 파이프라인 비교 분석 함수"""
    print("📊 문서 처리 파이프라인 비교 분석")
    print("=" * 70)
    
    # 결과 데이터 확인
    opensource_available = 'opensource_result' in globals() and opensource_result
    azure_di_available = 'text_result' in globals() and text_result
    
    if not opensource_available and not azure_di_available:
        print("❌ 비교할 파이프라인 결과가 없습니다.")
        print("   두 파이프라인을 모두 실행한 후 비교해주세요.")
        return None
    
    comparison_result = {
        'timestamp': datetime.now().isoformat(),
        'opensource': {},
        'azure_di': {},
        'comparison': {}
    }
    
    print("📋 파이프라인별 결과 요약:")
    
    # 1. 오픈소스 파이프라인 결과
    if opensource_available:
        os_result = opensource_result
        comparison_result['opensource'] = {
            'available': True,
            'processing_time': os_result.get('processing_time', 0),
            'text_length': os_result.get('text_length', 0),
            'token_count': os_result.get('token_count', 0),
            'chunks_count': os_result.get('chunks_count', 0),
            'file_name': os_result.get('file_name', 'Unknown'),
            'method': 'opensource_libraries'
        }
        
        print(f"\n🔧 1️⃣ 오픈소스 파이프라인:")
        print(f"   ⏱️ 처리시간: {os_result.get('processing_time', 0):.2f}초")
        print(f"   📝 텍스트 길이: {os_result.get('text_length', 0):,}자")
        print(f"   🔢 토큰 수: {os_result.get('token_count', 0):,}개")
        print(f"   📁 청크 수: {os_result.get('chunks_count', 0)}개")
    else:
        comparison_result['opensource'] = {'available': False}
        print(f"\n🔧 1️⃣ 오픈소스 파이프라인: 실행되지 않음")
    
    # 2. Azure Document Intelligence 파이프라인 결과
    if azure_di_available:
        azure_result = text_result
        # Azure DI 결과에서 정보 추출
        azure_text = azure_result.get('full_text', '')
        azure_stats = azure_result.get('statistics', {})
        
        comparison_result['azure_di'] = {
            'available': True,
            'processing_time': azure_stats.get('processing_time', 0),
            'text_length': len(azure_text),
            'token_count': azure_stats.get('total_chars', 0),  # 임시로 문자 수 사용
            'pages_count': azure_stats.get('pages', 0),
            'file_name': globals().get('test_document_path', 'Unknown'),
            'method': 'azure_document_intelligence'
        }
        
        print(f"\n☁️ 2️⃣ Azure Document Intelligence 파이프라인:")
        print(f"   ⏱️ 처리시간: {azure_stats.get('processing_time', 0):.2f}초")
        print(f"   📝 텍스트 길이: {len(azure_text):,}자")
        print(f"   📄 페이지 수: {azure_stats.get('pages', 0)}개")
        print(f"   🔢 문자 수: {azure_stats.get('total_chars', 0):,}개")
    else:
        comparison_result['azure_di'] = {'available': False}
        print(f"\n☁️ 2️⃣ Azure Document Intelligence 파이프라인: 실행되지 않음")
    
    # 3. 성능 비교 (둘 다 있는 경우만)
    if opensource_available and azure_di_available:
        print(f"\n⚖️ 성능 비교:")
        
        os_time = comparison_result['opensource']['processing_time']
        azure_time = comparison_result['azure_di']['processing_time']
        
        os_text_len = comparison_result['opensource']['text_length']
        azure_text_len = comparison_result['azure_di']['text_length']
        
        # 처리 시간 비교
        if os_time > 0 and azure_time > 0:
            time_ratio = azure_time / os_time
            faster_method = "오픈소스" if os_time < azure_time else "Azure DI"
            print(f"   ⏱️ 처리 속도: {faster_method}가 더 빠름")
            print(f"      - 오픈소스: {os_time:.2f}초")
            print(f"      - Azure DI: {azure_time:.2f}초")
            print(f"      - 비율: {time_ratio:.2f}배")
        
        # 텍스트 길이 비교
        if os_text_len > 0 and azure_text_len > 0:
            text_ratio = azure_text_len / os_text_len
            more_text = "Azure DI" if azure_text_len > os_text_len else "오픈소스"
            print(f"   📝 추출 텍스트량: {more_text}가 더 많음")
            print(f"      - 오픈소스: {os_text_len:,}자")
            print(f"      - Azure DI: {azure_text_len:,}자")
            print(f"      - 비율: {text_ratio:.2f}배")
        
        comparison_result['comparison'] = {
            'both_available': True,
            'time_ratio_azure_vs_os': azure_time / os_time if os_time > 0 else None,
            'text_ratio_azure_vs_os': azure_text_len / os_text_len if os_text_len > 0 else None,
            'faster_method': faster_method if 'faster_method' in locals() else None,
            'more_text_method': more_text if 'more_text' in locals() else None
        }
    
    # 4. 권장사항
    print(f"\n💡 권장사항:")
    if opensource_available and azure_di_available:
        if comparison_result['comparison'].get('time_ratio_azure_vs_os', 1) > 2:
            print(f"   ⚡ 빠른 처리가 필요한 경우: 오픈소스 파이프라인 권장")
        if comparison_result['comparison'].get('text_ratio_azure_vs_os', 1) > 1.2:
            print(f"   📊 정확한 추출이 필요한 경우: Azure DI 파이프라인 권장")
        print(f"   💰 비용 고려사항: 오픈소스는 무료, Azure DI는 사용량 기반 과금")
        print(f"   🔧 복잡한 문서 구조: Azure DI가 테이블, OCR 등에서 우수")
    else:
        print(f"   📋 두 파이프라인을 모두 실행하여 정확한 비교를 진행하세요.")
    
    # 5. 결과 저장
    comparison_file = os.path.join(OUTPUT_TEXTS_DIR, "pipeline_comparison_result.json")
    with open(comparison_file, 'w', encoding='utf-8') as f:
        json.dump(comparison_result, f, ensure_ascii=False, indent=2)
    
    print(f"\n💾 비교 결과 저장: {comparison_file}")
    
    # 전역 변수로 저장
    globals()['pipeline_comparison'] = comparison_result
    
    return comparison_result

# 통합 파이프라인 실행 함수
def run_complete_pipeline_comparison(file_path):
    """두 파이프라인을 모두 실행하고 비교"""
    print("🚀 완전한 파이프라인 비교 실행")
    print("=" * 70)
    
    # 1. 오픈소스 파이프라인 실행
    print("1️⃣ 오픈소스 파이프라인 실행...")
    os_result = run_opensource_pipeline(file_path)
    
    if os_result:
        # 벡터스토어 연동
        print("🗃️ 오픈소스 벡터스토어 연동...")
        run_vectorstore_pipeline(os_result, 'opensource')
    
    # 2. Azure DI 파이프라인 실행 (이미 실행된 경우 스킵)
    if 'azure_analysis_result' not in globals():
        print("2️⃣ Azure DI 파이프라인 실행...")
        # Azure DI 파이프라인 실행 (기존 함수 사용)
        if 'run_complete_test' in globals():
            run_complete_test()
    
    # 3. 비교 분석
    print("3️⃣ 파이프라인 비교 분석...")
    comparison = compare_pipelines()
    
    return comparison

# 비교 분석 실행
print("📊 파이프라인 비교 분석 함수 정의 완료!")
print("💡 사용법:")
print("   - compare_pipelines() : 기존 결과 비교")
print("   - run_complete_pipeline_comparison(file_path) : 전체 실행 후 비교")

## 🎯 최종 실행: 파이프라인 비교 테스트

In [ ]:
# 🎯 파이프라인 비교 테스트 최종 실행

print("🎯 문서 처리 파이프라인 비교 테스트 최종 실행")
print("=" * 70)

# 실행 옵션 설정
run_full_comparison = True  # True: 전체 비교 실행, False: 기존 결과만 비교

if run_full_comparison:
    print("🚀 전체 파이프라인 비교 실행 모드")
    
    # 테스트 문서 확인
    if 'test_document_path' in globals() and test_document_path and os.path.exists(test_document_path):
        print(f"📂 테스트 문서: {os.path.basename(test_document_path)}")
        
        # 전체 파이프라인 비교 실행
        comparison_result = run_complete_pipeline_comparison(test_document_path)
        
        if comparison_result:
            print(f"\n🎉 파이프라인 비교 완료!")
            
            # 상세 결과 출력
            if comparison_result.get('comparison', {}).get('both_available'):
                comp = comparison_result['comparison']
                print(f"\n📈 상세 비교 결과:")
                
                if comp.get('time_ratio_azure_vs_os'):
                    ratio = comp['time_ratio_azure_vs_os']
                    if ratio < 1:
                        print(f"   ⚡ Azure DI가 {1/ratio:.1f}배 더 빠름")
                    else:
                        print(f"   ⚡ 오픈소스가 {ratio:.1f}배 더 빠름")
                
                if comp.get('text_ratio_azure_vs_os'):
                    ratio = comp['text_ratio_azure_vs_os']
                    if ratio > 1:
                        print(f"   📝 Azure DI가 {ratio:.1f}배 더 많은 텍스트 추출")
                    else:
                        print(f"   📝 오픈소스가 {1/ratio:.1f}배 더 많은 텍스트 추출")
                
                # 벡터스토어 연동 결과
                if 'opensource_vectorstore_result' in globals():
                    vs_result = opensource_vectorstore_result
                    print(f"   🗃️ 오픈소스 벡터스토어: {vs_result['vectorstore_documents_count']}개 문서 업로드")
                
            # 권장사항 출력
            print(f"\n💡 최종 권장사항:")
            print(f"   📋 단순 텍스트 문서 → 오픈소스 파이프라인 (빠르고 경제적)")
            print(f"   📊 복잡한 구조 문서 → Azure DI 파이프라인 (정확하고 포괄적)")
            print(f"   💰 비용 민감성 → 오픈소스 우선, 정확도 필요시 Azure DI")
            print(f"   🔄 하이브리드 → 문서 유형별 파이프라인 선택적 적용")
            
        else:
            print("❌ 파이프라인 비교 실행 실패")
    
    else:
        print("❌ 테스트 문서가 설정되지 않았습니다.")
        print("   먼저 테스트 문서를 선택하고 기본 설정을 완료해주세요.")

else:
    print("📊 기존 결과 비교 모드")
    
    # 기존 결과만 비교
    comparison_result = compare_pipelines()
    
    if comparison_result:
        print(f"\n✅ 기존 결과 비교 완료!")
    else:
        print(f"\n⚠️ 비교할 결과가 없습니다. 먼저 파이프라인을 실행해주세요.")

# 결과 파일 위치 안내
print(f"\n📁 결과 파일 위치:")
print(f"   🔧 오픈소스 결과: {OPENSOURCE_OUTPUT_DIR}")
print(f"   ☁️ Azure DI 결과: {OUTPUT_TEXTS_DIR}")
print(f"   📊 비교 결과: {OUTPUT_TEXTS_DIR}/pipeline_comparison_result.json")

print(f"\n🎯 파이프라인 비교 분석 완료!")
print(f"=" * 70)

# 🎉 문서 처리 파이프라인 비교 분석 완료!

## 📋 구현된 주요 기능

### 🔧 1. 오픈소스 기반 문서 처리 파이프라인
- **PyPDF2**: PDF 텍스트 추출
- **python-docx**: Word 문서 처리
- **python-pptx**: PowerPoint 처리
- **BeautifulSoup**: HTML/XML 파싱 및 정제
- **tiktoken**: 토큰 계산 및 청킹
- **무료 사용**, 빠른 처리, 커스터마이징 용이

### ☁️ 2. Azure Document Intelligence 기반 파이프라인
- **prebuilt-read**: 고급 OCR 및 텍스트 추출
- **prebuilt-layout**: 테이블, 구조 인식
- **고정밀 분석**, 복잡한 문서 지원
- **사용량 기반 과금**

### 🗃️ 3. 벡터스토어 연동 시스템
- **문서 청킹**: 백엔드와 동일한 방식
- **임베딩 생성**: text-embedding-ada-002 호환
- **메타데이터 관리**: 파일 정보, 처리 이력
- **Cosmos DB/AI Search 호환**

### 📊 4. 성능 비교 분석
- **처리 시간 비교**: 속도 성능 측정
- **텍스트 추출량 비교**: 정확도 평가
- **비용 효율성 분석**: ROI 계산
- **권장사항 제시**: 시나리오별 최적 선택

---

## 🎯 주요 결과 및 권장사항

### ⚡ 성능 특성
| 구분 | 오픈소스 파이프라인 | Azure DI 파이프라인 |
|------|-------------------|-------------------|
| **처리 속도** | 빠름 (수초) | 보통 (수십초) |
| **정확도** | 보통 | 높음 |
| **비용** | 무료 | 사용량 기반 |
| **OCR 지원** | 없음 | 우수 |
| **테이블 처리** | 기본적 | 고급 |
| **커스터마이징** | 용이 | 제한적 |

### 💡 적용 시나리오
1. **📄 단순 텍스트 문서** → 오픈소스 파이프라인
2. **📊 복잡한 구조/테이블** → Azure DI 파이프라인  
3. **🔄 하이브리드 접근** → 문서 유형별 선택적 적용
4. **💰 비용 민감성** → 오픈소스 우선, 필요시 Azure DI

### 🚀 백엔드 통합 방안
- **환경 설정**: 파이프라인 선택 옵션 추가
- **자동 전환**: 문서 복잡도에 따른 자동 선택
- **비용 관리**: Azure DI 사용량 모니터링
- **품질 관리**: 처리 결과 검증 및 fallback

---

## 📚 다음 단계
1. **실제 백엔드 통합**: 환경별 파이프라인 설정
2. **성능 최적화**: 대용량 문서 처리 개선
3. **품질 검증**: 다양한 문서 유형 테스트
4. **모니터링**: 처리 성능 및 비용 추적 시스템

이제 백엔드에서 문서 유형과 요구사항에 따라 최적의 파이프라인을 선택할 수 있습니다! 🎉

In [21]:
# ✅ 핵심 확인사항 PASS/FAIL 요약 체크

def evaluate_core_checks():
    """요구된 3가지 핵심 확인사항에 대해 PASS/FAIL을 요약 출력합니다.
    1) 전체 텍스트 추출 확인
    2) 페이지별 그림/테이블 등 텍스트 외 요소 인식 (테이블 기준)
    3) 페이지별 결과 체크
    """
    # 1) 전체 텍스트 추출 확인
    text_ok = False
    total_chars = 0
    page_count = None
    try:
        if 'extracted_text' in globals() and isinstance(extracted_text, str):
            total_chars = len(extracted_text.strip())
            text_ok = total_chars > 20
        if not text_ok and 'text_statistics' in globals():
            total_chars = int(text_statistics.get('char_count', 0))
            text_ok = total_chars > 0
        # 저장 결과가 있으면 경로도 함께 기록
        text_files = None
        if 'text_save_result' in globals() and isinstance(text_save_result, dict):
            text_files = text_save_result
    except Exception:
        text_files = None

    # 페이지 수 (가능하면 구조 분석에서)
    try:
        if 'structure_analysis_result' in globals() and structure_analysis_result:
            page_count = structure_analysis_result['structure_info'].get('pages')
    except Exception:
        page_count = None

    # 2) 테이블 인식 확인 (페이지별 그림/테이블 중 테이블을 기준)
    tables_ok = False
    tables_count = 0
    try:
        if 'structure_analysis_result' in globals() and structure_analysis_result:
            tables_count = int(structure_analysis_result['structure_info'].get('tables', 0))
        if not tables_count and 'document_tables' in globals() and document_tables:
            tables_count = len(document_tables)
        tables_ok = tables_count > 0
        structure_files = None
        if 'structure_save_result' in globals() and isinstance(structure_save_result, dict):
            structure_files = structure_save_result
    except Exception:
        structure_files = None

    # 3) 페이지별 결과 체크 (page_analysis_data 존재 및 페이지 레벨 통계 수집 여부)
    page_ok = False
    analyzed_pages = 0
    try:
        if 'page_analysis_data' in globals() and isinstance(page_analysis_data, list):
            analyzed_pages = len(page_analysis_data)
            page_ok = analyzed_pages > 0
        page_files = None
        if 'page_save_result' in globals() and isinstance(page_save_result, dict):
            page_files = page_save_result
    except Exception:
        page_files = None

    # 결과 출력
    print("\n===== 핵심 확인사항 요약 =====")
    if text_ok:
        print(f"[PASS] 전체 텍스트 추출: {total_chars:,}자 추출")
    else:
        print(f"[FAIL] 전체 텍스트 추출: 추출된 텍스트가 부족하거나 없음 (chars={total_chars})")

    if tables_ok:
        print(f"[PASS] 테이블 인식: {tables_count}개 테이블 감지")
    else:
        print("[WARN] 테이블 인식: 감지된 테이블 없음 (문서가 테이블을 포함하지 않을 수 있음)")

    if page_ok:
        if page_count is not None:
            print(f"[PASS] 페이지별 결과: {analyzed_pages}개 페이지 분석 완료 (총 페이지={page_count})")
        else:
            print(f"[PASS] 페이지별 결과: {analyzed_pages}개 페이지 분석 완료")
    else:
        print("[FAIL] 페이지별 결과: 페이지 단위 분석 데이터가 비어있음")

    # 저장된 파일 경로나 힌트 출력 (가능한 범위)
    if text_files:
        print("\n- 텍스트 저장 파일:")
        if text_files.get('text_file'):
            print(f"  • TXT: {text_files['text_file']}")
        if text_files.get('stats_file'):
            print(f"  • JSON(통계): {text_files['stats_file']}")

    if structure_files:
        print("- 구조/테이블 저장 파일:")
        if structure_files.get('report_file'):
            print(f"  • 구조 보고서: {structure_files['report_file']}")
        if structure_files.get('data_file'):
            print(f"  • 구조 데이터(JSON): {structure_files['data_file']}")
        if structure_files.get('tables_summary_csv'):
            print(f"  • 테이블 요약 CSV: {structure_files['tables_summary_csv']}")
        if structure_files.get('table_csv_files'):
            print(f"  • 테이블 별 CSV ({len(structure_files['table_csv_files'])}개):")
            for p in structure_files['table_csv_files']:
                print(f"    - {p}")

    if page_files:
        print("- 페이지 분석 저장 파일:")
        if page_files.get('page_report'):
            print(f"  • 상세 보고서: {page_files['page_report']}")
        if page_files.get('page_data'):
            print(f"  • 분석 데이터(JSON): {page_files['page_data']}")

    print("================================\n")

# 지금 실행 중인 세션에서 바로 평가 실행 (이미 앞 단계가 실행된 상태라 가정)
evaluate_core_checks()


===== 핵심 확인사항 요약 =====
[PASS] 전체 텍스트 추출: 9,775자 추출
[PASS] 테이블 인식: 11개 테이블 감지
[PASS] 페이지별 결과: 27개 페이지 분석 완료 (총 페이지=27)

- 텍스트 저장 파일:
  • TXT: /home/admin/wkms-aws/jupyter_notebook/data/output_texts/test_20250923_125927_full_text.txt
  • JSON(통계): /home/admin/wkms-aws/jupyter_notebook/data/output_texts/test_20250923_125927_statistics.json
- 구조/테이블 저장 파일:
  • 구조 보고서: /home/admin/wkms-aws/jupyter_notebook/data/output_texts/test_20250923_125927_structure_analysis.txt
  • 구조 데이터(JSON): /home/admin/wkms-aws/jupyter_notebook/data/output_texts/test_20250923_125927_structure_data.json
  • 테이블 요약 CSV: /home/admin/wkms-aws/jupyter_notebook/data/output_texts/test_20250923_125927_tables_summary.csv
  • 테이블 별 CSV (11개):
    - /home/admin/wkms-aws/jupyter_notebook/data/output_texts/test_20250923_125927_table_1_p11.csv
    - /home/admin/wkms-aws/jupyter_notebook/data/output_texts/test_20250923_125927_table_2_p12.csv
    - /home/admin/wkms-aws/jupyter_notebook/data/output_texts/test_20250923_125927_tab